In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# <span style="color:red"> Step.1_PDF (과실비율인정기준) 데이터 추출 </span>

In [2]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출
    - 사고유형코드, 기본과실비율, 사고유형명, 관련판례 등 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
                
            # 페이지별 데이터 추출 시도
            page_data = extract_page_data_standards(text, page_num + 1)
            if page_data:
                extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def extract_page_data_standards(text, page_num):
    """
    과실비율인정기준 PDF의 각 페이지에서 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용 원본 텍스트 일부
    }
    
    lines = text.split('\n')
    
    # 1. 사고유형코드 찾기 (예: "보1", "차1" 등)
    for line in lines:
        # "보1", "보2", "차1", "차2" 패턴 찾기
        code_match = re.search(r'([보차]\d+)', line)
        if code_match:
            result['accident_code'] = code_match.group(1)
            # 사고유형명도 같은 줄에서 찾기
            # 예: "보1  보행자 직선신호 횡단게시, 직선신호 충격 사고"
            parts = line.split()
            if len(parts) > 1:
                result['accident_name'] = ' '.join(parts[1:])
            break
    
    # 2. 기본과실비율 찾기 (숫자 패턴)
    for line in lines:
        # "70", "기본과실비율 70" 같은 패턴
        ratio_match = re.search(r'기본과실비율.*?(\d+)', line)
        if ratio_match:
            result['basic_fault_ratio'] = ratio_match.group(1)
            break
        # 단순히 숫자만 있는 경우도 체크
        elif re.search(r'^\s*\d{1,2}\s*$', line):
            result['basic_fault_ratio'] = line.strip()
    
    # 3. 수정요소 찾기 (①, ②, ③ 패턴)
    modification_list = []
    for line in lines:
        mod_match = re.search(r'[①②③④⑤]\s*(.+?)\s*([+\-]\d+)', line)
        if mod_match:
            factor = mod_match.group(1).strip()
            value = mod_match.group(2)
            modification_list.append(f"{factor}:{value}")
    
    if modification_list:
        result['modification_factors'] = '; '.join(modification_list)
    
    # 4. 관련판례 찾기 (대법원 YYYY.MM.DD 패턴)
    case_pattern = r'대법원\s*(\d{4}\.\d{1,2}\.\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)'
    case_matches = re.findall(case_pattern, text)
    if case_matches:
        result['related_cases'] = '; '.join(case_matches)
    
    # 데이터가 있으면 반환, 없으면 None
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    return None

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 결과 확인
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        print("\n📊 추출된 데이터 미리보기:")
        print(df_temp[['accident_code', 'accident_name', 'basic_fault_ratio']].head())
        
        # 임시 저장
        df_temp.to_csv("temp_step1_standards.csv", index=False, encoding='utf-8-sig')
        print("💾 Step1 결과가 temp_step1_standards.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 

# 1. 수정보완(1)

In [3]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 개선된 버전
    - 실제 사고유형 상세 페이지만 추출 (목차 제외)
    - 기본과실비율, 수정요소, 관련판례 정확히 추출
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 목차나 설명 페이지 제외 - 실제 사고유형 상세 페이지만 처리
            if is_detail_page(text):
                page_data = extract_page_data_standards_improved(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_detail_page(text):
    """
    실제 사고유형 상세 페이지인지 판단
    - 목차, 설명, 서론 페이지 제외
    - 기본과실비율, 수정요소가 있는 실제 상세 페이지만 선택
    """
    # 제외할 페이지 키워드
    exclude_keywords = [
        '목차', '서론', '적용 범위', '용어 정의',
        '제3편', '제1장', '제2장', '제3장',
        '............', '.........',  # 목차의 점선들
        'Page', '002', '003', '004'  # 페이지 번호만 있는 경우
    ]
    
    # 포함되어야 할 키워드 (실제 상세 페이지)
    include_keywords = [
        '기본과실비율', '수정요소', '과실비율',
        '①', '②', '③', '④', '⑤',  # 수정요소 번호
        '대법원', '판결',  # 관련판례
        '%'  # 비율 표시
    ]
    
    # 제외 키워드가 많으면 목차 페이지
    exclude_count = sum(1 for keyword in exclude_keywords if keyword in text)
    if exclude_count >= 3:
        return False
    
    # 포함 키워드가 있으면 상세 페이지
    include_count = sum(1 for keyword in include_keywords if keyword in text)
    return include_count >= 2

def extract_page_data_standards_improved(text, page_num):
    """
    개선된 과실비율인정기준 PDF 데이터 추출
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:500]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    
    # 1. 사고유형코드 찾기 (개선된 패턴)
    accident_code_found = False
    for i, line in enumerate(clean_lines):
        # [보1], [차1] 같은 패턴으로 둘러싸인 코드 우선 찾기
        bracket_match = re.search(r'\[([보차이]\d+)\]', line)
        if bracket_match:
            result['accident_code'] = bracket_match.group(1)
            accident_code_found = True
            
            # 같은 줄 또는 근처에서 사고유형명 찾기
            name_candidates = [line.replace(bracket_match.group(0), '').strip()]
            
            # 앞뒤 줄도 확인
            if i > 0:
                name_candidates.append(clean_lines[i-1])
            if i < len(clean_lines) - 1:
                name_candidates.append(clean_lines[i+1])
            
            # 가장 적절한 사고유형명 선택
            for candidate in name_candidates:
                if candidate and not re.search(r'[\d\.]{3,}|페이지|Page', candidate):
                    # 불필요한 부분 제거
                    clean_name = re.sub(r'\[.*?\]|\.{3,}|\d{3,}', '', candidate).strip()
                    if len(clean_name) > 5:  # 너무 짧은 건 제외
                        result['accident_name'] = clean_name[:100]  # 길이 제한
                        break
            break
    
    # 단순 패턴도 시도 (대비책)
    if not accident_code_found:
        for line in clean_lines:
            simple_match = re.search(r'^([보차이]\d+)', line)
            if simple_match:
                result['accident_code'] = simple_match.group(1)
                # 같은 줄에서 사고유형명 추출
                remaining = line[simple_match.end():].strip()
                if remaining:
                    result['accident_name'] = remaining[:100]
                break
    
    # 2. 기본과실비율 찾기 (개선된 패턴)
    ratio_patterns = [
        r'기본과실비율.*?(\d+)',  # "기본과실비율 70"
        r'보행자.*?기본.*?과실비율.*?(\d+)',  # "보행자 기본 과실비율 70"
        r'과실비율.*?(\d{1,2})(?:\s*%)?',  # "과실비율 70%" 또는 "과실비율 70"
        r'(\d{1,2})\s*%',  # 단순히 "70%"
        r'기본.*?(\d{1,2})',  # "기본 70"
    ]
    
    text_joined = ' '.join(clean_lines)
    for pattern in ratio_patterns:
        ratio_match = re.search(pattern, text_joined, re.IGNORECASE)
        if ratio_match:
            ratio_value = int(ratio_match.group(1))
            if 0 <= ratio_value <= 100:  # 유효한 비율만
                result['basic_fault_ratio'] = str(ratio_value)
                break
    
    # 3. 수정요소 찾기 (개선된 패턴)
    modification_list = []
    modification_patterns = [
        r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩]+?)(?:[+\-]?\d+|비례)',
        r'(\d+)\.\s*([^0-9\n]+?)(?:[+\-]?\d+|비례)',
        r'([가나다라마바사아자차카타파하])\s*([^가나다라마바사아자차카타파하]+?)(?:[+\-]?\d+|비례)',
    ]
    
    for line in clean_lines:
        for pattern in modification_patterns:
            matches = re.finditer(pattern, line)
            for match in matches:
                if len(match.groups()) >= 2:
                    factor_desc = match.group(2).strip()
                    # 점수 추출
                    score_match = re.search(r'([+\-]?\d+)', line[match.end()-10:match.end()+10])
                    score = score_match.group(1) if score_match else ''
                    
                    if factor_desc and len(factor_desc) > 3:
                        modification_list.append(f"{factor_desc}:{score}")
    
    # 중복 제거 및 정리
    seen = set()
    unique_modifications = []
    for mod in modification_list:
        if mod not in seen and len(mod) > 5:
            seen.add(mod)
            unique_modifications.append(mod)
    
    result['modification_factors'] = '; '.join(unique_modifications[:10])  # 최대 10개
    
    # 4. 관련판례 찾기 (개선된 패턴)
    case_patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',  # 간단한 형태
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'(\d{4}년.*?판결)',
    ]
    
    case_matches = []
    for pattern in case_patterns:
        matches = re.findall(pattern, text_joined)
        case_matches.extend(matches)
    
    if case_matches:
        # 중복 제거하고 최대 3개까지
        unique_cases = list(dict.fromkeys(case_matches))[:3]
        result['related_cases'] = '; '.join(unique_cases)
    
    # 5. 데이터 유효성 검사
    # 최소한 사고코드나 기본과실비율 중 하나는 있어야 유효한 데이터
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

# 실행 및 검증 함수
def validate_extracted_data(data_list):
    """추출된 데이터 품질 검증"""
    if not data_list:
        return
    
    print(f"\n📊 데이터 품질 검증 보고서")
    print("="*50)
    
    total = len(data_list)
    has_code = sum(1 for d in data_list if d['accident_code'])
    has_name = sum(1 for d in data_list if d['accident_name'])
    has_ratio = sum(1 for d in data_list if d['basic_fault_ratio'])
    has_modification = sum(1 for d in data_list if d['modification_factors'])
    has_cases = sum(1 for d in data_list if d['related_cases'])
    
    print(f"📈 전체 데이터: {total}건")
    print(f"   - 사고코드 있음: {has_code}건 ({has_code/total*100:.1f}%)")
    print(f"   - 사고유형명 있음: {has_name}건 ({has_name/total*100:.1f}%)")
    print(f"   - 기본과실비율 있음: {has_ratio}건 ({has_ratio/total*100:.1f}%)")
    print(f"   - 수정요소 있음: {has_modification}건 ({has_modification/total*100:.1f}%)")
    print(f"   - 관련판례 있음: {has_cases}건 ({has_cases/total*100:.1f}%)")
    
    # 상위 5개 샘플 출력
    print(f"\n📋 추출된 데이터 샘플:")
    for i, item in enumerate(data_list[:5]):
        print(f"\n{i+1}. 사고코드: {item['accident_code']}")
        print(f"   사고유형명: {item['accident_name'][:50]}...")
        print(f"   기본과실비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행 예시
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # Step 1 실행
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    # 데이터 품질 검증
    validate_extracted_data(data_standards)
    
    # 결과 저장
    if data_standards:
        df_temp = pd.DataFrame(data_standards)
        df_temp.to_csv("temp_step1_standards_improved.csv", index=False, encoding='utf-8-sig')
        print(f"\n💾 개선된 Step1 결과가 temp_step1_standards_improved.csv로 저장되었습니다.")
    else:
        print("❌ 추출된 데이터가 없습니다. 패턴을 다시 확인해주세요.")

📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 페이지 48 처리 중...
   📄 페

   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 처리 중...
   📄 페이지 425 처리 중...
   📄 페이지 426 처리 중...
   📄 페이지 427 처리 중...
   📄 페이지 428 처리 중...
   📄 페이지 429 

# 2. 수정개선보완(2)

In [5]:
import pdfplumber
import pandas as pd
import re
import os

def extract_fault_standards_pdf(pdf_path):
    """
    첫 번째 PDF (과실비율인정기준)에서 데이터 추출 - 최종 개선 버전
    - 사고유형명, 관련판례 추출 대폭 개선
    - 페이지별 상세 분석으로 정확도 극대화
    """
    print(f"📖 첫 번째 PDF 처리 시작: {os.path.basename(pdf_path)}")
    
    extracted_data = []
    
    with pdfplumber.open(pdf_path) as pdf:
        for page_num, page in enumerate(pdf.pages):
            print(f"   📄 페이지 {page_num + 1} 처리 중...")
            
            text = page.extract_text()
            if not text:
                continue
            
            # 실제 사고유형 상세 페이지만 처리
            if is_accident_detail_page(text):
                page_data = extract_page_data_final(text, page_num + 1)
                if page_data:
                    extracted_data.append(page_data)
    
    print(f"✅ 첫 번째 PDF 완료: 총 {len(extracted_data)}개 사례 추출")
    return extracted_data

def is_accident_detail_page(text):
    """
    사고유형 상세 페이지 판별 - 더 정교한 필터링
    """
    # 반드시 제외할 페이지들
    exclude_strong = [
        '목차', '서론', '적용 범위', '용어 정의', 'Page',
        '제3편', '제1장', '제2장', '제3장',
        '..........................'  # 목차 점선
    ]
    
    # 제외할 가능성이 높은 페이지
    exclude_weak = [
        '002', '003', '004', '005',  # 페이지 번호만 있는 경우
        '자동차사고 과실비율 인정기준',
        '과실비율 적용기준'
    ]
    
    # 반드시 포함되어야 할 요소들
    must_have = [
        r'[보차이]\d+',  # 사고코드
        r'\d{1,2}(?:\s*%)?',  # 비율
    ]
    
    # 포함되면 좋은 요소들
    good_to_have = [
        '기본과실비율', '기본 과실비율', '수정요소',
        '①', '②', '③', '④', '⑤',
        '대법원', '판결', '사고 상황',
        '+', '-', '%'
    ]
    
    # 강력 제외 조건
    for exclude in exclude_strong:
        if exclude in text and len(text) < 1000:  # 짧은 텍스트에서 발견되면 제외
            return False
    
    # 필수 조건 확인
    must_count = sum(1 for pattern in must_have if re.search(pattern, text))
    if must_count < 1:
        return False
    
    # 선호 조건 확인
    good_count = sum(1 for keyword in good_to_have if keyword in text)
    weak_count = sum(1 for keyword in exclude_weak if keyword in text)
    
    # 점수 기반 판별
    score = good_count * 2 - weak_count
    return score >= 3

def extract_page_data_final(text, page_num):
    """
    최종 개선된 데이터 추출 함수
    """
    result = {
        'source_pdf': '과실비율인정기준',
        'page_number': page_num,
        'accident_code': '',
        'accident_name': '',
        'basic_fault_ratio': '',
        'modification_factors': '',
        'related_cases': '',
        'raw_text': text[:300]  # 디버깅용
    }
    
    lines = text.split('\n')
    clean_lines = [line.strip() for line in lines if line.strip()]
    text_joined = ' '.join(clean_lines)
    
    # === 1. 사고유형코드 추출 (최우선) ===
    accident_code = extract_accident_code(clean_lines, text_joined)
    if accident_code:
        result['accident_code'] = accident_code
    
    # === 2. 사고유형명 추출 (대폭 개선) ===
    accident_name = extract_accident_name(clean_lines, text_joined, accident_code)
    if accident_name:
        result['accident_name'] = accident_name
    
    # === 3. 기본과실비율 추출 ===
    basic_ratio = extract_basic_fault_ratio(clean_lines, text_joined)
    if basic_ratio:
        result['basic_fault_ratio'] = basic_ratio
    
    # === 4. 수정요소 추출 ===
    modifications = extract_modification_factors(clean_lines, text_joined)
    if modifications:
        result['modification_factors'] = modifications
    
    # === 5. 관련판례 추출 (개선) ===
    cases = extract_related_cases(clean_lines, text_joined)
    if cases:
        result['related_cases'] = cases
    
    # 최소 조건: 사고코드 또는 기본과실비율이 있어야 유효
    if result['accident_code'] or result['basic_fault_ratio']:
        return result
    
    return None

def extract_accident_code(lines, text_joined):
    """사고유형코드 추출"""
    # 패턴 1: [보1], [차1] 형태 (최우선)
    bracket_match = re.search(r'\[([보차이]\d+)\]', text_joined)
    if bracket_match:
        return bracket_match.group(1)
    
    # 패턴 2: 보1), 차1) 형태 
    paren_match = re.search(r'([보차이]\d+)\)', text_joined)
    if paren_match:
        return paren_match.group(1)
    
    # 패턴 3: 줄 시작에 보1, 차1 형태
    for line in lines:
        if re.match(r'^([보차이]\d+)', line):
            return re.match(r'^([보차이]\d+)', line).group(1)
    
    # 패턴 4: 단독으로 존재하는 코드
    standalone_match = re.search(r'\b([보차이]\d+)\b', text_joined)
    if standalone_match:
        return standalone_match.group(1)
    
    return None

def extract_accident_name(lines, text_joined, accident_code):
    """사고유형명 추출 - 대폭 개선"""
    if not accident_code:
        return None
    
    candidates = []
    
    # === 방법 1: 코드 근처 텍스트에서 추출 ===
    code_patterns = [
        rf'\[{accident_code}\]([^[\n]+)',  # [보1] 직진 대 직진
        rf'{accident_code}\)([^)\n]+)',    # 보1) 직진 대 직진  
        rf'{accident_code}([^0-9\n]+)',    # 보1 직진 대 직진
    ]
    
    for pattern in code_patterns:
        match = re.search(pattern, text_joined)
        if match:
            name_candidate = match.group(1).strip()
            # 정제
            name_candidate = re.sub(r'[\.\-\s]{3,}.*', '', name_candidate)  # 점선 제거
            name_candidate = re.sub(r'\d{3,}.*', '', name_candidate)       # 페이지번호 제거
            if 10 <= len(name_candidate) <= 100:  # 적절한 길이
                candidates.append(name_candidate)
    
    # === 방법 2: 사고 설명 패턴 찾기 ===
    accident_patterns = [
        r'(.*?(?:직진|좌회전|우회전|후진|주차).*?(?:대|사고).*?)',
        r'(.*?(?:보행자|자동차|이륜차).*?(?:횡단|충돌|접촉).*?)',
        r'(.*?(?:신호|교차로|횡단보도).*?(?:통과|진입).*?)',
        r'(\([^)]+\)\s*\([^)]+\))',  # (신호등 있음) (직진)
    ]
    
    for line in lines:
        for pattern in accident_patterns:
            match = re.search(pattern, line, re.IGNORECASE)
            if match:
                name_candidate = match.group(1).strip()
                # 불필요한 부분 제거
                name_candidate = re.sub(r'^\d+[\.\)]\s*', '', name_candidate)  # 번호 제거
                name_candidate = re.sub(r'\[.*?\]', '', name_candidate)        # 괄호 제거
                name_candidate = re.sub(r'\.{3,}.*', '', name_candidate)       # 점선 제거
                if 8 <= len(name_candidate) <= 80:
                    candidates.append(name_candidate)
    
    # === 방법 3: 구조화된 텍스트에서 추출 ===
    structure_patterns = [
        r'사고\s*상황[:\s]*([^:\n]+)',
        r'사고\s*유형[:\s]*([^:\n]+)', 
        r'적용\s*기준[:\s]*([^:\n]+)',
    ]
    
    for pattern in structure_patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            name_candidate = match.group(1).strip()
            if 10 <= len(name_candidate) <= 100:
                candidates.append(name_candidate)
    
    # === 최적 후보 선택 ===
    if candidates:
        # 중복 제거
        unique_candidates = list(dict.fromkeys(candidates))
        
        # 점수 매기기
        scored_candidates = []
        for candidate in unique_candidates:
            score = 0
            # 키워드 포함 점수
            keywords = ['직진', '좌회전', '우회전', '보행자', '자동차', '교차로', '횡단', '신호', '사고']
            score += sum(3 for keyword in keywords if keyword in candidate)
            
            # 길이 점수 (적당한 길이 선호)
            if 15 <= len(candidate) <= 50:
                score += 5
            elif 10 <= len(candidate) <= 60:
                score += 3
            
            # 특수문자 패널티
            if re.search(r'[\.]{3,}|\d{3,}', candidate):
                score -= 5
            
            scored_candidates.append((candidate, score))
        
        # 가장 높은 점수의 후보 선택
        if scored_candidates:
            best_candidate = max(scored_candidates, key=lambda x: x[1])
            if best_candidate[1] > 0:  # 최소 점수 이상
                return best_candidate[0]
    
    return None

def extract_basic_fault_ratio(lines, text_joined):
    """기본과실비율 추출"""
    patterns = [
        r'기본\s*과실\s*비율[:\s]*(\d{1,2})',
        r'기본[:\s]*(\d{1,2})\s*%?',
        r'과실\s*비율[:\s]*(\d{1,2})',
        r'(\d{1,2})\s*%',
        r'비율[:\s]*(\d{1,2})',
    ]
    
    for pattern in patterns:
        match = re.search(pattern, text_joined, re.IGNORECASE)
        if match:
            ratio = int(match.group(1))
            if 0 <= ratio <= 100:
                return str(ratio)
    
    return None

def extract_modification_factors(lines, text_joined):
    """수정요소 추출"""
    factors = []
    
    # 패턴 1: ①, ②, ③ 형태
    circle_patterns = r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(circle_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 2: 가. 나. 다. 형태
    korean_patterns = r'[가나다라마바사아자차카타파하]\.\s*([^가나다라마바사아자차카타파하\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(korean_patterns, text_joined)
    for match in matches:
        desc = match.group(1).strip()
        score = match.group(2) if match.group(2) else ''
        if len(desc) > 3:
            factors.append(f"{desc}:{score}")
    
    # 패턴 3: 1), 2), 3) 형태
    number_patterns = r'(\d+)\)\s*([^\d\n]+?)(?:([+\-]\d+)|비례)'
    matches = re.finditer(number_patterns, text_joined)
    for match in matches:
        desc = match.group(2).strip()
        score = match.group(3) if match.group(3) else ''
        if len(desc) > 3 and not re.search(r'페이지|Page', desc):
            factors.append(f"{desc}:{score}")
    
    # 중복 제거 및 정리
    unique_factors = []
    seen = set()
    for factor in factors:
        if factor not in seen and len(factor) > 5:
            seen.add(factor)
            unique_factors.append(factor)
    
    return '; '.join(unique_factors[:8]) if unique_factors else None

def extract_related_cases(lines, text_joined):
    """관련판례 추출 - 개선"""
    cases = []
    
    # 패턴들 (더 다양하게)
    patterns = [
        r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
        r'대법원\s*(\d{4}-\d+-\d+)',
        r'대법원\s*(\d{4}\.?\d{1,2}\.?\d{1,2})',  
        r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일\s*판결)',
        r'판결\s*(\d{4}\.\d{1,2}\.\d{1,2})',
        r'선고\s*(\d+[가-힣]\d+)',
    ]
    
    for pattern in patterns:
        matches = re.findall(pattern, text_joined)
        cases.extend(matches)
    
    # 판례번호 패턴도 찾기
    case_num_patterns = [
        r'(\d+[가-힣]\d+)',  # 2019다12345
        r'(\d{4}[가-힣]\d+)', # 2019가단12345  
    ]
    
    for pattern in case_num_patterns:
        matches = re.findall(pattern, text_joined)
        for match in matches:
            if not any(match in case for case in cases):  # 중복 방지
                cases.append(match)
    
    # 정리 및 중복 제거
    unique_cases = list(dict.fromkeys(cases))[:5]  # 최대 5개
    return '; '.join(unique_cases) if unique_cases else None

# 검증 및 실행 함수
def validate_and_save_data(data_list, filename):
    """데이터 검증 및 저장"""
    if not data_list:
        print("❌ 추출된 데이터가 없습니다.")
        return
    
    df = pd.DataFrame(data_list)
    df.to_csv(filename, index=False, encoding='utf-8-sig')
    
    # 상세 분석
    total = len(data_list)
    metrics = {
        'accident_code': sum(1 for d in data_list if d['accident_code']),
        'accident_name': sum(1 for d in data_list if d['accident_name']),
        'basic_fault_ratio': sum(1 for d in data_list if d['basic_fault_ratio']),
        'modification_factors': sum(1 for d in data_list if d['modification_factors']),
        'related_cases': sum(1 for d in data_list if d['related_cases']),
    }
    
    print(f"\n📊 최종 데이터 품질 보고서")
    print("="*50)
    print(f"전체 데이터: {total}건")
    for field, count in metrics.items():
        print(f"{field}: {count}건 ({count/total*100:.1f}%)")
    
    # 베스트 샘플 출력
    print(f"\n🏆 추출 품질이 좋은 상위 5개 샘플:")
    
    # 품질 점수 계산
    scored_data = []
    for item in data_list:
        score = 0
        if item['accident_code']: score += 20
        if item['accident_name']: score += 25
        if item['basic_fault_ratio']: score += 20
        if item['modification_factors']: score += 20
        if item['related_cases']: score += 15
        scored_data.append((item, score))
    
    # 상위 5개 출력
    top_samples = sorted(scored_data, key=lambda x: x[1], reverse=True)[:5]
    for i, (item, score) in enumerate(top_samples):
        print(f"\n{i+1}. [품질점수: {score}/100]")
        print(f"   코드: {item['accident_code']}")
        print(f"   사고명: {item['accident_name'][:60]}...")
        print(f"   기본비율: {item['basic_fault_ratio']}")
        print(f"   수정요소: {item['modification_factors'][:50]}...")
        print(f"   관련판례: {item['related_cases'][:50]}...")

# 실행
if __name__ == "__main__":
    pdf_path_1 = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("🚀 Step 1 최종 개선 버전 실행")
    data_standards = extract_fault_standards_pdf(pdf_path_1)
    
    validate_and_save_data(data_standards, "temp_step1_standards_final.csv")
    print(f"\n💾 최종 결과가 temp_step1_standards_final.csv로 저장되었습니다.")

🚀 Step 1 최종 개선 버전 실행
📖 첫 번째 PDF 처리 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1 처리 중...
   📄 페이지 2 처리 중...
   📄 페이지 3 처리 중...
   📄 페이지 4 처리 중...
   📄 페이지 5 처리 중...
   📄 페이지 6 처리 중...
   📄 페이지 7 처리 중...
   📄 페이지 8 처리 중...
   📄 페이지 9 처리 중...
   📄 페이지 10 처리 중...
   📄 페이지 11 처리 중...
   📄 페이지 12 처리 중...
   📄 페이지 13 처리 중...
   📄 페이지 14 처리 중...
   📄 페이지 15 처리 중...
   📄 페이지 16 처리 중...
   📄 페이지 17 처리 중...
   📄 페이지 18 처리 중...
   📄 페이지 19 처리 중...
   📄 페이지 20 처리 중...
   📄 페이지 21 처리 중...
   📄 페이지 22 처리 중...
   📄 페이지 23 처리 중...
   📄 페이지 24 처리 중...
   📄 페이지 25 처리 중...
   📄 페이지 26 처리 중...
   📄 페이지 27 처리 중...
   📄 페이지 28 처리 중...
   📄 페이지 29 처리 중...
   📄 페이지 30 처리 중...
   📄 페이지 31 처리 중...
   📄 페이지 32 처리 중...
   📄 페이지 33 처리 중...
   📄 페이지 34 처리 중...
   📄 페이지 35 처리 중...
   📄 페이지 36 처리 중...
   📄 페이지 37 처리 중...
   📄 페이지 38 처리 중...
   📄 페이지 39 처리 중...
   📄 페이지 40 처리 중...
   📄 페이지 41 처리 중...
   📄 페이지 42 처리 중...
   📄 페이지 43 처리 중...
   📄 페이지 44 처리 중...
   📄 페이지 45 처리 중...
   📄 페이지 46 처리 중...
   📄 페이지 47 처리 중...
   📄 

   📄 페이지 377 처리 중...
   📄 페이지 378 처리 중...
   📄 페이지 379 처리 중...
   📄 페이지 380 처리 중...
   📄 페이지 381 처리 중...
   📄 페이지 382 처리 중...
   📄 페이지 383 처리 중...
   📄 페이지 384 처리 중...
   📄 페이지 385 처리 중...
   📄 페이지 386 처리 중...
   📄 페이지 387 처리 중...
   📄 페이지 388 처리 중...
   📄 페이지 389 처리 중...
   📄 페이지 390 처리 중...
   📄 페이지 391 처리 중...
   📄 페이지 392 처리 중...
   📄 페이지 393 처리 중...
   📄 페이지 394 처리 중...
   📄 페이지 395 처리 중...
   📄 페이지 396 처리 중...
   📄 페이지 397 처리 중...
   📄 페이지 398 처리 중...
   📄 페이지 399 처리 중...
   📄 페이지 400 처리 중...
   📄 페이지 401 처리 중...
   📄 페이지 402 처리 중...
   📄 페이지 403 처리 중...
   📄 페이지 404 처리 중...
   📄 페이지 405 처리 중...
   📄 페이지 406 처리 중...
   📄 페이지 407 처리 중...
   📄 페이지 408 처리 중...
   📄 페이지 409 처리 중...
   📄 페이지 410 처리 중...
   📄 페이지 411 처리 중...
   📄 페이지 412 처리 중...
   📄 페이지 413 처리 중...
   📄 페이지 414 처리 중...
   📄 페이지 415 처리 중...
   📄 페이지 416 처리 중...
   📄 페이지 417 처리 중...
   📄 페이지 418 처리 중...
   📄 페이지 419 처리 중...
   📄 페이지 420 처리 중...
   📄 페이지 421 처리 중...
   📄 페이지 422 처리 중...
   📄 페이지 423 처리 중...
   📄 페이지 424 

# 3. 수정개선보완(EasyOCR)

In [1]:
import pdfplumber
import pandas as pd
import re
import os
import easyocr
import numpy as np
from PIL import Image
import time

class EnhancedFaultExtractor:
    def __init__(self, use_gpu=True):
        """
        EasyOCR 우선 과실비율 PDF 추출기
        """
        print("🚀 EasyOCR 기반 과실비율 추출기 초기화 중...")
        
        # EasyOCR 리더 초기화 (한국어 + 영어)
        try:
            self.reader = easyocr.Reader(['ko', 'en'], gpu=use_gpu)
            print(f"✅ EasyOCR 초기화 완료 (GPU: {use_gpu})")
        except Exception as e:
            print(f"⚠️  GPU 초기화 실패, CPU 모드로 전환: {e}")
            self.reader = easyocr.Reader(['ko', 'en'], gpu=False)
        
        self.extraction_stats = {
            'total_pages': 0,
            'ocr_success': 0,
            'pdfplumber_success': 0,
            'hybrid_success': 0,
            'extraction_method': []
        }

    def extract_fault_standards_pdf(self, pdf_path, max_pages=None):
        """
        EasyOCR 우선 전략으로 PDF 데이터 추출
        """
        print(f"📖 EasyOCR 우선 추출 시작: {os.path.basename(pdf_path)}")
        
        extracted_data = []
        
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if max_pages:
                total_pages = min(total_pages, max_pages)
            
            self.extraction_stats['total_pages'] = total_pages
            
            for page_num in range(total_pages):
                page = pdf.pages[page_num]
                print(f"   📄 페이지 {page_num + 1}/{total_pages} 처리 중...")
                
                # 1단계: EasyOCR 우선 시도
                text_ocr = self.extract_text_with_easyocr(page, page_num + 1)
                
                # 2단계: pdfplumber 백업 추출
                text_pdf = self.extract_text_with_pdfplumber(page)
                
                # 3단계: 최적 텍스트 선택
                best_text, method = self.select_best_text(text_ocr, text_pdf, page_num + 1)
                
                # 4단계: 데이터 추출 시도
                if best_text and self.is_accident_detail_page(best_text):
                    page_data = self.extract_page_data_enhanced(best_text, page_num + 1, method)
                    if page_data:
                        extracted_data.append(page_data)
                        print(f"      ✅ 데이터 추출 성공 ({method})")
                    else:
                        print(f"      ⚠️  텍스트는 있으나 데이터 추출 실패")
        
        self.print_extraction_stats()
        print(f"✅ 추출 완료: 총 {len(extracted_data)}개 사례")
        return extracted_data

    def extract_text_with_easyocr(self, page, page_num):
        """EasyOCR로 텍스트 추출"""
        try:
            start_time = time.time()
            
            # 고해상도 이미지 변환
            pil_image = page.to_image(resolution=300).original
            image_array = np.array(pil_image)
            
            # EasyOCR 실행
            results = self.reader.readtext(image_array)
            
            # 결과를 텍스트로 조합 (위치 정보 고려)
            if results:
                # 위치 기반 정렬 (위에서 아래로, 왼쪽에서 오른쪽으로)
                sorted_results = sorted(results, key=lambda x: (x[0][0][1], x[0][0][0]))
                extracted_text = '\n'.join([result[1] for result in sorted_results])
                
                elapsed = time.time() - start_time
                print(f"      🔍 EasyOCR: {len(extracted_text)}자 추출 ({elapsed:.1f}초)")
                self.extraction_stats['ocr_success'] += 1
                return extracted_text
            else:
                print(f"      ❌ EasyOCR: 텍스트 없음")
                return ""
                
        except Exception as e:
            print(f"      ❌ EasyOCR 오류: {e}")
            return ""

    def extract_text_with_pdfplumber(self, page):
        """pdfplumber로 텍스트 추출 (백업용)"""
        try:
            text = page.extract_text()
            if text:
                print(f"      📄 pdfplumber: {len(text)}자 추출")
                self.extraction_stats['pdfplumber_success'] += 1
                return text
            else:
                print(f"      ❌ pdfplumber: 텍스트 없음")
                return ""
        except Exception as e:
            print(f"      ❌ pdfplumber 오류: {e}")
            return ""

    def select_best_text(self, text_ocr, text_pdf, page_num):
        """최적의 텍스트 선택 로직"""
        
        # 품질 점수 계산
        score_ocr = self.calculate_text_quality(text_ocr, "OCR")
        score_pdf = self.calculate_text_quality(text_pdf, "PDF")
        
        print(f"      📊 품질점수 - OCR: {score_ocr}, PDF: {score_pdf}")
        
        # 선택 로직
        if score_ocr > score_pdf * 1.3:  # OCR이 확실히 우수
            self.extraction_stats['extraction_method'].append('OCR')
            return text_ocr, 'EasyOCR'
        elif score_pdf > score_ocr * 1.3:  # PDF가 확실히 우수
            self.extraction_stats['extraction_method'].append('PDF')
            return text_pdf, 'pdfplumber'
        elif score_ocr > 0 and score_pdf > 0:  # 둘 다 있으면 하이브리드
            hybrid_text = self.create_hybrid_text(text_ocr, text_pdf)
            self.extraction_stats['hybrid_success'] += 1
            self.extraction_stats['extraction_method'].append('Hybrid')
            return hybrid_text, 'Hybrid'
        elif score_ocr > score_pdf:  # OCR 우선
            self.extraction_stats['extraction_method'].append('OCR')
            return text_ocr, 'EasyOCR'
        else:  # PDF 우선
            self.extraction_stats['extraction_method'].append('PDF')
            return text_pdf, 'pdfplumber'

    def calculate_text_quality(self, text, method_name):
        """텍스트 품질 점수 계산"""
        if not text:
            return 0
        
        score = 0
        
        # 기본 점수: 텍스트 길이
        score += min(len(text) / 100, 10)  # 최대 10점
        
        # 핵심 키워드 점수
        key_patterns = [
            (r'[보차이]\d+', 15),        # 사고코드 (15점)
            (r'기본.*과실.*비율', 10),      # 기본과실비율 (10점)
            (r'\d{1,2}\s*%', 8),         # 비율 표시 (8점)
            (r'[①②③④⑤⑥⑦⑧⑨⑩]', 5),   # 번호 매김 (5점)
            (r'수정.*요소', 5),           # 수정요소 (5점)
            (r'대법원.*판결', 8),         # 판례 (8점)
            (r'직진|좌회전|우회전', 3),    # 사고유형 (3점)
            (r'교차로|횡단보도', 3),       # 장소 (3점)
        ]
        
        for pattern, points in key_patterns:
            matches = len(re.findall(pattern, text))
            score += min(matches * points, points * 2)  # 중복 보너스 제한
        
        # 한국어 비율 점수 (OCR에 유리)
        korean_chars = len(re.findall(r'[가-힣]', text))
        total_chars = len(text)
        if total_chars > 0:
            korean_ratio = korean_chars / total_chars
            if method_name == "OCR":
                score += korean_ratio * 10  # OCR은 한국어 비율이 높을수록 좋음
        
        # 구조화 점수
        lines = text.split('\n')
        non_empty_lines = [line for line in lines if line.strip()]
        if len(non_empty_lines) > 5:
            score += 5  # 충분한 구조화
        
        # 노이즈 패널티
        if re.search(r'[^\w\s가-힣①-⑩\[\]\(\)\.,%\-\+]', text):
            score -= 3  # 이상한 문자 패널티
        
        return max(score, 0)

    def create_hybrid_text(self, text_ocr, text_pdf):
        """OCR과 PDF 텍스트를 결합하여 최적화"""
        
        # 방법 1: 키워드 기반 선택적 결합
        key_sections = {}
        
        # OCR에서 강점이 있는 부분 추출
        ocr_strengths = [
            (r'[보차이]\d+.*?(?=\n|$)', 'accident_codes'),
            (r'.*?[①②③④⑤⑥⑦⑧⑨⑩].*?(?=\n|$)', 'numbered_items'),
            (r'.*?기본.*과실.*비율.*?(?=\n|$)', 'fault_ratios'),
        ]
        
        for pattern, section in ocr_strengths:
            matches = re.findall(pattern, text_ocr, re.MULTILINE)
            if matches:
                key_sections[section] = matches
        
        # PDF에서 강점이 있는 부분 추출
        pdf_strengths = [
            (r'대법원.*?판결.*?(?=\n|$)', 'court_cases'),
            (r'.*?수정.*요소.*?(?=\n|$)', 'modification_factors'),
        ]
        
        for pattern, section in pdf_strengths:
            matches = re.findall(pattern, text_pdf, re.MULTILINE)
            if matches and section not in key_sections:
                key_sections[section] = matches
        
        # 결합된 텍스트 생성
        hybrid_parts = []
        
        # 주요 섹션들을 순서대로 추가
        section_order = ['accident_codes', 'fault_ratios', 'numbered_items', 'modification_factors', 'court_cases']
        
        for section in section_order:
            if section in key_sections:
                hybrid_parts.extend(key_sections[section])
        
        # 나머지 중요한 내용 추가 (OCR 우선)
        remaining_ocr = text_ocr
        for section, matches in key_sections.items():
            for match in matches:
                remaining_ocr = remaining_ocr.replace(match, '')
        
        # 정리된 텍스트 추가
        cleaned_remaining = '\n'.join([line.strip() for line in remaining_ocr.split('\n') 
                                     if line.strip() and len(line.strip()) > 5])
        if cleaned_remaining:
            hybrid_parts.append(cleaned_remaining)
        
        return '\n'.join(hybrid_parts)

    def is_accident_detail_page(self, text):
        """사고유형 상세 페이지 판별 - 기존 로직 유지"""
        exclude_strong = [
            '목차', '서론', '적용 범위', '용어 정의', 'Page',
            '제3편', '제1장', '제2장', '제3장',
            '..........................'
        ]
        
        exclude_weak = [
            '002', '003', '004', '005',
            '자동차사고 과실비율 인정기준',
            '과실비율 적용기준'
        ]
        
        must_have = [
            r'[보차이]\d+',
            r'\d{1,2}(?:\s*%)?',
        ]
        
        good_to_have = [
            '기본과실비율', '기본 과실비율', '수정요소',
            '①', '②', '③', '④', '⑤',
            '대법원', '판결', '사고 상황',
            '+', '-', '%'
        ]
        
        # 강력 제외 조건
        for exclude in exclude_strong:
            if exclude in text and len(text) < 1000:
                return False
        
        # 필수 조건 확인
        must_count = sum(1 for pattern in must_have if re.search(pattern, text))
        if must_count < 1:
            return False
        
        # 선호 조건 확인
        good_count = sum(1 for keyword in good_to_have if keyword in text)
        weak_count = sum(1 for keyword in exclude_weak if keyword in text)
        
        score = good_count * 2 - weak_count
        return score >= 3

    def extract_page_data_enhanced(self, text, page_num, extraction_method):
        """향상된 데이터 추출 - 기존 로직 활용"""
        result = {
            'source_pdf': '과실비율인정기준',
            'page_number': page_num,
            'extraction_method': extraction_method,
            'accident_code': '',
            'accident_name': '',
            'basic_fault_ratio': '',
            'modification_factors': '',
            'related_cases': '',
            'text_quality_score': self.calculate_text_quality(text, extraction_method),
            'raw_text': text[:300]
        }
        
        lines = text.split('\n')
        clean_lines = [line.strip() for line in lines if line.strip()]
        text_joined = ' '.join(clean_lines)
        
        # 기존 추출 로직들 활용
        result['accident_code'] = self.extract_accident_code(clean_lines, text_joined)
        result['accident_name'] = self.extract_accident_name(clean_lines, text_joined, result['accident_code'])
        result['basic_fault_ratio'] = self.extract_basic_fault_ratio(clean_lines, text_joined)
        result['modification_factors'] = self.extract_modification_factors(clean_lines, text_joined)
        result['related_cases'] = self.extract_related_cases(clean_lines, text_joined)
        
        # 최소 조건 확인
        if result['accident_code'] or result['basic_fault_ratio']:
            return result
        
        return None

    def extract_accident_code(self, lines, text_joined):
        """사고유형코드 추출 - 기존 로직"""
        bracket_match = re.search(r'\[([보차이]\d+)\]', text_joined)
        if bracket_match:
            return bracket_match.group(1)
        
        paren_match = re.search(r'([보차이]\d+)\)', text_joined)
        if paren_match:
            return paren_match.group(1)
        
        for line in lines:
            if re.match(r'^([보차이]\d+)', line):
                return re.match(r'^([보차이]\d+)', line).group(1)
        
        standalone_match = re.search(r'\b([보차이]\d+)\b', text_joined)
        if standalone_match:
            return standalone_match.group(1)
        
        return None

    def extract_accident_name(self, lines, text_joined, accident_code):
        """사고유형명 추출 - 기존 로직 활용"""
        if not accident_code:
            return None
        
        candidates = []
        
        # 코드 근처 텍스트 추출
        code_patterns = [
            rf'\[{accident_code}\]([^[\n]+)',
            rf'{accident_code}\)([^)\n]+)',
            rf'{accident_code}([^0-9\n]+)',
        ]
        
        for pattern in code_patterns:
            match = re.search(pattern, text_joined)
            if match:
                name_candidate = match.group(1).strip()
                name_candidate = re.sub(r'[\.\-\s]{3,}.*', '', name_candidate)
                name_candidate = re.sub(r'\d{3,}.*', '', name_candidate)
                if 10 <= len(name_candidate) <= 100:
                    candidates.append(name_candidate)
        
        if candidates:
            return candidates[0]  # 첫 번째 후보 반환
        
        return None

    def extract_basic_fault_ratio(self, lines, text_joined):
        """기본과실비율 추출 - 기존 로직"""
        patterns = [
            r'기본\s*과실\s*비율[:\s]*(\d{1,2})',
            r'기본[:\s]*(\d{1,2})\s*%?',
            r'과실\s*비율[:\s]*(\d{1,2})',
            r'(\d{1,2})\s*%',
            r'비율[:\s]*(\d{1,2})',
        ]
        
        for pattern in patterns:
            match = re.search(pattern, text_joined, re.IGNORECASE)
            if match:
                ratio = int(match.group(1))
                if 0 <= ratio <= 100:
                    return str(ratio)
        
        return None

    def extract_modification_factors(self, lines, text_joined):
        """수정요소 추출 - 기존 로직"""
        factors = []
        
        patterns = [
            r'[①②③④⑤⑥⑦⑧⑨⑩]\s*([^①②③④⑤⑥⑦⑧⑨⑩\n]+?)(?:([+\-]\d+)|비례)',
            r'[가나다라마바사아자차카타파하]\.\s*([^가나다라마바사아자차카타파하\n]+?)(?:([+\-]\d+)|비례)',
            r'(\d+)\)\s*([^\d\n]+?)(?:([+\-]\d+)|비례)'
        ]
        
        for pattern in patterns:
            matches = re.finditer(pattern, text_joined)
            for match in matches:
                if len(match.groups()) >= 2:
                    desc = match.group(1).strip() if match.group(1) else match.group(2).strip()
                    score = match.group(-1) if match.group(-1) else ''
                    if len(desc) > 3:
                        factors.append(f"{desc}:{score}")
        
        unique_factors = list(dict.fromkeys(factors))[:8]
        return '; '.join(unique_factors) if unique_factors else None

    def extract_related_cases(self, lines, text_joined):
        """관련판례 추출 - 기존 로직"""
        cases = []
        
        patterns = [
            r'대법원\s*(\d{4}\.\s*\d{1,2}\.\s*\d{1,2}\.?\s*선고\s*\d+[가-힣]\d+\s*판결)',
            r'대법원\s*(\d{4}-\d+-\d+)',
            r'대법원\s*(\d{4}\.?\d{1,2}\.?\d{1,2})',
            r'(\d{4}년\s*\d{1,2}월\s*\d{1,2}일\s*판결)',
            r'선고\s*(\d+[가-힣]\d+)',
        ]
        
        for pattern in patterns:
            matches = re.findall(pattern, text_joined)
            cases.extend(matches)
        
        unique_cases = list(dict.fromkeys(cases))[:5]
        return '; '.join(unique_cases) if unique_cases else None

    def print_extraction_stats(self):
        """추출 통계 출력"""
        stats = self.extraction_stats
        print(f"\n📊 추출 방법별 통계:")
        print(f"   📄 전체 페이지: {stats['total_pages']}")
        print(f"   🔍 EasyOCR 성공: {stats['ocr_success']} ({stats['ocr_success']/stats['total_pages']*100:.1f}%)")
        print(f"   📋 pdfplumber 성공: {stats['pdfplumber_success']} ({stats['pdfplumber_success']/stats['total_pages']*100:.1f}%)")
        print(f"   🔄 하이브리드 생성: {stats['hybrid_success']}")
        
        # 최종 선택 방법 통계
        method_counts = {}
        for method in stats['extraction_method']:
            method_counts[method] = method_counts.get(method, 0) + 1
        
        print(f"   🏆 최종 선택된 방법:")
        for method, count in method_counts.items():
            print(f"      {method}: {count}회")

def main():
    """메인 실행 함수"""
    print("🚀 EasyOCR 우선 과실비율 PDF 추출기 시작")
    print("="*60)
    
    # PDF 파일 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    # 추출기 초기화 및 실행
    extractor = EnhancedFaultExtractor(use_gpu=True)
    
    # 전체 추출 (테스트 시에는 max_pages=10 정도로 제한)
    data_list = extractor.extract_fault_standards_pdf(pdf_path, max_pages=20)
    
    # 결과 저장 및 분석
    if data_list:
        df = pd.DataFrame(data_list)
        filename = "easyocr_enhanced_fault_extraction.csv"
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        
        print(f"\n✅ 결과 저장 완료: {filename}")
        print(f"📊 총 {len(data_list)}건의 데이터 추출")
        
        # 추출 방법별 성공률
        method_success = df.groupby('extraction_method').size()
        print(f"\n📈 추출 방법별 성공 데이터:")
        for method, count in method_success.items():
            print(f"   {method}: {count}건")
        
        # 품질 점수 분석
        avg_quality = df['text_quality_score'].mean()
        print(f"\n⭐ 평균 텍스트 품질 점수: {avg_quality:.1f}")
        
    else:
        print("❌ 추출된 데이터가 없습니다.")

if __name__ == "__main__":
    main()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🚀 EasyOCR 우선 과실비율 PDF 추출기 시작
🚀 EasyOCR 기반 과실비율 추출기 초기화 중...
Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |███████-------------------------------------------| 15.7% Complete

Progress: |████████████████----------------------------------| 32.6% Complete

Progress: |████████████████████████--------------------------| 48.4% Complete

...

Progress: |█████████████████████████████████████████---------| 82.0% Complete

Progress: |█████████████████████████████████████████████████-| 98.2% Complete

Progress: |██████████████████████████████████████████████████| 100.1% Complete✅ EasyOCR 초기화 완료 (GPU: True)
📖 EasyOCR 우선 추출 시작: 231107_과실비율인정기준_온라인용.pdf
   📄 페이지 1/20 처리 중...


C:\Users\kmj11\anaconda3\envs\gpudm\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


      🔍 EasyOCR: 30자 추출 (6.4초)
      📄 pdfplumber: 49자 추출
      📊 품질점수 - OCR: 6.633333333333333, PDF: 5.49
   📄 페이지 2/20 처리 중...
      🔍 EasyOCR: 390자 추출 (7.7초)
      📄 pdfplumber: 2441자 추출
      📊 품질점수 - OCR: 29.310256410256407, PDF: 30
   📄 페이지 3/20 처리 중...
      🔍 EasyOCR: 624자 추출 (8.3초)
      📄 pdfplumber: 2621자 추출
      📊 품질점수 - OCR: 59.93551282051282, PDF: 59
   📄 페이지 4/20 처리 중...
      🔍 EasyOCR: 816자 추출 (9.2초)
      📄 pdfplumber: 2826자 추출
      📊 품질점수 - OCR: 62.294803921568615, PDF: 62
   📄 페이지 5/20 처리 중...
      🔍 EasyOCR: 572자 추출 (8.0초)
      📄 pdfplumber: 2629자 추출
      📊 품질점수 - OCR: 59.545174825174826, PDF: 62
   📄 페이지 6/20 처리 중...
      🔍 EasyOCR: 671자 추출 (8.2초)
      📄 pdfplumber: 2867자 추출
      📊 품질점수 - OCR: 25.68764530551416, PDF: 24
   📄 페이지 7/20 처리 중...
      🔍 EasyOCR: 21자 추출 (5.4초)
      📄 pdfplumber: 21자 추출
      📊 품질점수 - OCR: 7.829047619047619, PDF: 0.21
   📄 페이지 8/20 처리 중...
      🔍 EasyOCR: 895자 추출 (8.8초)
      📄 pdfplumber: 902자 추출
      📊 품질점수 - OCR: 18.145530

In [2]:
import pdfplumber
import pandas as pd
import os
import easyocr
import numpy as np
from PIL import Image
import time
import re

class SimpleTextExtractor:
    def __init__(self, use_gpu=True):
        """
        단순한 텍스트 추출기 - 복잡한 필터링 없이 모든 텍스트 저장
        """
        print("🚀 단순 텍스트 추출기 초기화 중...")
        
        try:
            self.reader = easyocr.Reader(['ko', 'en'], gpu=use_gpu)
            print(f"✅ EasyOCR 초기화 완료 (GPU: {use_gpu})")
        except Exception as e:
            print(f"⚠️  GPU 초기화 실패, CPU 모드로 전환: {e}")
            self.reader = easyocr.Reader(['ko', 'en'], gpu=False)

    def extract_all_text(self, pdf_path, output_format='both', max_pages=None):
        """
        모든 페이지에서 텍스트 추출하여 저장
        
        Args:
            pdf_path: PDF 파일 경로
            output_format: 'csv', 'txt', 'both' 중 선택
            max_pages: 처리할 최대 페이지 수 (None이면 전체)
        """
        print(f"📖 전체 텍스트 추출 시작: {os.path.basename(pdf_path)}")
        
        results = []
        
        with pdfplumber.open(pdf_path) as pdf:
            total_pages = len(pdf.pages)
            if max_pages:
                total_pages = min(total_pages, max_pages)
            
            print(f"📄 총 {total_pages}페이지 처리 예정")
            
            for page_num in range(total_pages):
                page = pdf.pages[page_num]
                print(f"   📄 페이지 {page_num + 1}/{total_pages} 처리 중...")
                
                # EasyOCR 추출
                text_ocr = self.extract_with_easyocr(page, page_num + 1)
                
                # pdfplumber 추출
                text_pdf = self.extract_with_pdfplumber(page)
                
                # 최적 텍스트 선택
                best_text, method = self.choose_better_text(text_ocr, text_pdf)
                
                # 결과 저장
                page_result = {
                    'page_number': page_num + 1,
                    'extraction_method': method,
                    'text_length': len(best_text),
                    'ocr_text': text_ocr,
                    'pdf_text': text_pdf,
                    'selected_text': best_text,
                    'has_korean': len(re.findall(r'[가-힣]', best_text)) > 0,
                    'has_numbers': len(re.findall(r'\d', best_text)) > 0,
                    'contains_keywords': self.check_keywords(best_text)
                }
                
                results.append(page_result)
                
                print(f"      ✅ 완료 - {method}: {len(best_text)}자")
        
        # 결과 저장
        base_filename = os.path.splitext(os.path.basename(pdf_path))[0]
        
        if output_format in ['csv', 'both']:
            self.save_to_csv(results, f"{base_filename}_extracted_text.csv")
        
        if output_format in ['txt', 'both']:
            self.save_to_txt(results, f"{base_filename}_extracted_text.txt")
        
        # 통계 출력
        self.print_statistics(results)
        
        return results

    def extract_with_easyocr(self, page, page_num):
        """EasyOCR로 텍스트 추출"""
        try:
            start_time = time.time()
            
            # 고해상도 이미지 변환
            pil_image = page.to_image(resolution=300).original
            image_array = np.array(pil_image)
            
            # EasyOCR 실행
            results = self.reader.readtext(image_array)
            
            if results:
                # 위치 기반 정렬 (위→아래, 왼쪽→오른쪽)
                sorted_results = sorted(results, key=lambda x: (x[0][0][1], x[0][0][0]))
                extracted_text = '\n'.join([result[1] for result in sorted_results])
                
                elapsed = time.time() - start_time
                print(f"      🔍 EasyOCR: {len(extracted_text)}자 ({elapsed:.1f}초)")
                return extracted_text
            else:
                print(f"      ❌ EasyOCR: 텍스트 없음")
                return ""
                
        except Exception as e:
            print(f"      ❌ EasyOCR 오류: {e}")
            return ""

    def extract_with_pdfplumber(self, page):
        """pdfplumber로 텍스트 추출"""
        try:
            text = page.extract_text()
            if text:
                print(f"      📄 pdfplumber: {len(text)}자")
                return text
            else:
                print(f"      ❌ pdfplumber: 텍스트 없음")
                return ""
        except Exception as e:
            print(f"      ❌ pdfplumber 오류: {e}")
            return ""

    def choose_better_text(self, text_ocr, text_pdf):
        """더 나은 텍스트 선택"""
        
        # 둘 다 비어있으면
        if not text_ocr and not text_pdf:
            return "", "None"
        
        # 하나만 있으면 그것 선택
        if not text_ocr:
            return text_pdf, "pdfplumber"
        if not text_pdf:
            return text_ocr, "EasyOCR"
        
        # 둘 다 있으면 품질 비교
        score_ocr = self.calculate_simple_score(text_ocr)
        score_pdf = self.calculate_simple_score(text_pdf)
        
        # 길이도 고려 (너무 짧으면 점수 감점)
        if len(text_ocr) < 50:
            score_ocr *= 0.5
        if len(text_pdf) < 50:
            score_pdf *= 0.5
        
        if score_ocr > score_pdf:
            return text_ocr, "EasyOCR"
        else:
            return text_pdf, "pdfplumber"

    def calculate_simple_score(self, text):
        """간단한 텍스트 품질 점수"""
        if not text:
            return 0
        
        score = 0
        
        # 기본 점수: 텍스트 길이
        score += len(text) / 100
        
        # 한국어 있으면 보너스
        korean_count = len(re.findall(r'[가-힣]', text))
        score += korean_count / 10
        
        # 숫자 있으면 보너스
        number_count = len(re.findall(r'\d', text))
        score += number_count / 20
        
        # 특수 키워드 보너스
        keywords = ['보1', '보2', '차1', '차2', '과실', '비율', '%', '대법원', '판결']
        for keyword in keywords:
            if keyword in text:
                score += 5
        
        return score

    def check_keywords(self, text):
        """주요 키워드 포함 여부 확인"""
        keywords = [
            '과실', '비율', '사고', '보행자', '자동차', '교차로', 
            '직진', '좌회전', '우회전', '신호', '횡단보도',
            '보1', '보2', '보3', '차1', '차2', '차3',
            '대법원', '판결', '①', '②', '③'
        ]
        
        found_keywords = [kw for kw in keywords if kw in text]
        return ', '.join(found_keywords) if found_keywords else 'None'

    def save_to_csv(self, results, filename):
        """CSV 파일로 저장"""
        print(f"\n💾 CSV 저장 중: {filename}")
        
        # DataFrame 생성
        df_data = []
        for result in results:
            df_data.append({
                'page_number': result['page_number'],
                'extraction_method': result['extraction_method'],
                'text_length': result['text_length'],
                'has_korean': result['has_korean'],
                'has_numbers': result['has_numbers'],
                'contains_keywords': result['contains_keywords'],
                'selected_text': result['selected_text'],
                'ocr_text': result['ocr_text'],
                'pdf_text': result['pdf_text']
            })
        
        df = pd.DataFrame(df_data)
        df.to_csv(filename, index=False, encoding='utf-8-sig')
        print(f"✅ CSV 저장 완료: {filename}")

    def save_to_txt(self, results, filename):
        """TXT 파일로 저장 (읽기 편한 형태)"""
        print(f"\n💾 TXT 저장 중: {filename}")
        
        with open(filename, 'w', encoding='utf-8') as f:
            f.write("=" * 80 + "\n")
            f.write("PDF 텍스트 추출 결과\n")
            f.write("=" * 80 + "\n\n")
            
            for result in results:
                f.write(f"📄 페이지 {result['page_number']}\n")
                f.write(f"📊 추출방법: {result['extraction_method']}\n")
                f.write(f"📏 텍스트 길이: {result['text_length']}자\n")
                f.write(f"🔍 포함 키워드: {result['contains_keywords']}\n")
                f.write("-" * 60 + "\n")
                f.write(result['selected_text'])
                f.write("\n\n" + "=" * 80 + "\n\n")
        
        print(f"✅ TXT 저장 완료: {filename}")

    def print_statistics(self, results):
        """추출 통계 출력"""
        total_pages = len(results)
        
        # 추출 방법별 통계
        method_counts = {}
        for result in results:
            method = result['extraction_method']
            method_counts[method] = method_counts.get(method, 0) + 1
        
        # 텍스트 품질 통계
        text_lengths = [r['text_length'] for r in results]
        avg_length = sum(text_lengths) / len(text_lengths) if text_lengths else 0
        
        korean_pages = sum(1 for r in results if r['has_korean'])
        keyword_pages = sum(1 for r in results if r['contains_keywords'] != 'None')
        
        print(f"\n📊 최종 추출 통계")
        print("=" * 50)
        print(f"📄 총 페이지: {total_pages}")
        print(f"📝 평균 텍스트 길이: {avg_length:.0f}자")
        print(f"🇰🇷 한국어 포함 페이지: {korean_pages}페이지 ({korean_pages/total_pages*100:.1f}%)")
        print(f"🔍 키워드 포함 페이지: {keyword_pages}페이지 ({keyword_pages/total_pages*100:.1f}%)")
        
        print(f"\n🏆 추출 방법별 선택 횟수:")
        for method, count in method_counts.items():
            print(f"   {method}: {count}회 ({count/total_pages*100:.1f}%)")
        
        # 최고 품질 페이지 Top 5
        top_pages = sorted(results, key=lambda x: x['text_length'], reverse=True)[:5]
        print(f"\n🏅 텍스트 길이 상위 5페이지:")
        for i, page in enumerate(top_pages):
            keywords = page['contains_keywords'][:50] + "..." if len(page['contains_keywords']) > 50 else page['contains_keywords']
            print(f"   {i+1}. 페이지 {page['page_number']}: {page['text_length']}자 ({page['extraction_method']}) - {keywords}")

def main():
    """메인 실행 함수"""
    print("🚀 단순 텍스트 추출기 시작")
    print("="*60)
    
    # PDF 파일 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    # 추출기 초기화
    extractor = SimpleTextExtractor(use_gpu=True)
    
    # 선택 메뉴
    print("\n📋 추출 옵션을 선택하세요:")
    print("1. 전체 페이지 추출 (CSV + TXT)")
    print("2. 처음 20페이지만 테스트 (CSV + TXT)")
    print("3. CSV만 저장")
    print("4. TXT만 저장")
    
    choice = input("\n선택 (1-4): ").strip()
    
    if choice == '1':
        results = extractor.extract_all_text(pdf_path, output_format='both')
    elif choice == '2':
        results = extractor.extract_all_text(pdf_path, output_format='both', max_pages=20)
    elif choice == '3':
        results = extractor.extract_all_text(pdf_path, output_format='csv')
    elif choice == '4':
        results = extractor.extract_all_text(pdf_path, output_format='txt')
    else:
        print("기본값으로 처음 20페이지 테스트를 실행합니다.")
        results = extractor.extract_all_text(pdf_path, output_format='both', max_pages=20)
    
    print(f"\n🎉 추출 완료!")
    print(f"📁 생성된 파일들을 확인하세요.")

if __name__ == "__main__":
    main()

Neither CUDA nor MPS are available - defaulting to CPU. Note: This module is much faster with a GPU.


🚀 단순 텍스트 추출기 시작
🚀 단순 텍스트 추출기 초기화 중...
✅ EasyOCR 초기화 완료 (GPU: True)

📋 추출 옵션을 선택하세요:
1. 전체 페이지 추출 (CSV + TXT)
2. 처음 20페이지만 테스트 (CSV + TXT)
3. CSV만 저장
4. TXT만 저장

선택 (1-4): 1
📖 전체 텍스트 추출 시작: 231107_과실비율인정기준_온라인용.pdf
📄 총 600페이지 처리 예정
   📄 페이지 1/600 처리 중...
      🔍 EasyOCR: 30자 (5.5초)
      📄 pdfplumber: 49자
      ✅ 완료 - pdfplumber: 49자
   📄 페이지 2/600 처리 중...
      🔍 EasyOCR: 390자 (7.2초)
      📄 pdfplumber: 2441자
      ✅ 완료 - pdfplumber: 2441자
   📄 페이지 3/600 처리 중...
      🔍 EasyOCR: 624자 (7.9초)
      📄 pdfplumber: 2621자
      ✅ 완료 - pdfplumber: 2621자
   📄 페이지 4/600 처리 중...
      🔍 EasyOCR: 816자 (8.8초)
      📄 pdfplumber: 2826자
      ✅ 완료 - pdfplumber: 2826자
   📄 페이지 5/600 처리 중...
      🔍 EasyOCR: 572자 (7.8초)
      📄 pdfplumber: 2629자
      ✅ 완료 - pdfplumber: 2629자
   📄 페이지 6/600 처리 중...
      🔍 EasyOCR: 671자 (8.3초)
      📄 pdfplumber: 2867자
      ✅ 완료 - pdfplumber: 2867자
   📄 페이지 7/600 처리 중...
      🔍 EasyOCR: 21자 (5.4초)
      📄 pdfplumber: 21자
      ✅ 완료 - pdfplumber: 21자
   📄 페이지 8/600 처

      🔍 EasyOCR: 978자 (10.4초)
      📄 pdfplumber: 1056자
      ✅ 완료 - pdfplumber: 1056자
   📄 페이지 73/600 처리 중...
      🔍 EasyOCR: 695자 (8.7초)
      📄 pdfplumber: 739자
      ✅ 완료 - pdfplumber: 739자
   📄 페이지 74/600 처리 중...
      🔍 EasyOCR: 642자 (8.9초)
      📄 pdfplumber: 706자
      ✅ 완료 - pdfplumber: 706자
   📄 페이지 75/600 처리 중...
      🔍 EasyOCR: 1014자 (9.9초)
      📄 pdfplumber: 1052자
      ✅ 완료 - pdfplumber: 1052자
   📄 페이지 76/600 처리 중...
      🔍 EasyOCR: 1065자 (10.3초)
      📄 pdfplumber: 1130자
      ✅ 완료 - pdfplumber: 1130자
   📄 페이지 77/600 처리 중...
      🔍 EasyOCR: 854자 (9.7초)
      📄 pdfplumber: 924자
      ✅ 완료 - pdfplumber: 924자
   📄 페이지 78/600 처리 중...
      🔍 EasyOCR: 699자 (9.4초)
      📄 pdfplumber: 742자
      ✅ 완료 - pdfplumber: 742자
   📄 페이지 79/600 처리 중...
      🔍 EasyOCR: 1082자 (11.0초)
      📄 pdfplumber: 1120자
      ✅ 완료 - pdfplumber: 1120자
   📄 페이지 80/600 처리 중...
      🔍 EasyOCR: 1060자 (10.4초)
      📄 pdfplumber: 1130자
      ✅ 완료 - pdfplumber: 1130자
   📄 페이지 81/600 처리 중...
      🔍 Ea

      🔍 EasyOCR: 1051자 (10.1초)
      📄 pdfplumber: 1125자
      ✅ 완료 - pdfplumber: 1125자
   📄 페이지 146/600 처리 중...
      🔍 EasyOCR: 852자 (9.5초)
      📄 pdfplumber: 961자
      ✅ 완료 - pdfplumber: 961자
   📄 페이지 147/600 처리 중...
      🔍 EasyOCR: 466자 (7.8초)
      📄 pdfplumber: 524자
      ✅ 완료 - pdfplumber: 524자
   📄 페이지 148/600 처리 중...
      🔍 EasyOCR: 436자 (7.8초)
      📄 pdfplumber: 479자
      ✅ 완료 - pdfplumber: 479자
   📄 페이지 149/600 처리 중...
      🔍 EasyOCR: 910자 (9.6초)
      📄 pdfplumber: 980자
      ✅ 완료 - pdfplumber: 980자
   📄 페이지 150/600 처리 중...
      🔍 EasyOCR: 948자 (10.1초)
      📄 pdfplumber: 1029자
      ✅ 완료 - pdfplumber: 1029자
   📄 페이지 151/600 처리 중...
      🔍 EasyOCR: 673자 (8.5초)
      📄 pdfplumber: 736자
      ✅ 완료 - pdfplumber: 736자
   📄 페이지 152/600 처리 중...
      🔍 EasyOCR: 732자 (9.0초)
      📄 pdfplumber: 773자
      ✅ 완료 - pdfplumber: 773자
   📄 페이지 153/600 처리 중...
      🔍 EasyOCR: 929자 (9.5초)
      📄 pdfplumber: 988자
      ✅ 완료 - pdfplumber: 988자
   📄 페이지 154/600 처리 중...
      🔍 Easy

      🔍 EasyOCR: 477자 (7.9초)
      📄 pdfplumber: 513자
      ✅ 완료 - pdfplumber: 513자
   📄 페이지 219/600 처리 중...
      🔍 EasyOCR: 660자 (9.0초)
      📄 pdfplumber: 719자
      ✅ 완료 - pdfplumber: 719자
   📄 페이지 220/600 처리 중...
      🔍 EasyOCR: 1047자 (10.5초)
      📄 pdfplumber: 1122자
      ✅ 완료 - pdfplumber: 1122자
   📄 페이지 221/600 처리 중...
      🔍 EasyOCR: 1473자 (12.5초)
      📄 pdfplumber: 1538자
      ✅ 완료 - pdfplumber: 1538자
   📄 페이지 222/600 처리 중...
      🔍 EasyOCR: 619자 (8.4초)
      📄 pdfplumber: 669자
      ✅ 완료 - pdfplumber: 669자
   📄 페이지 223/600 처리 중...
      🔍 EasyOCR: 617자 (8.9초)
      📄 pdfplumber: 660자
      ✅ 완료 - pdfplumber: 660자
   📄 페이지 224/600 처리 중...
      🔍 EasyOCR: 952자 (10.1초)
      📄 pdfplumber: 1017자
      ✅ 완료 - pdfplumber: 1017자
   📄 페이지 225/600 처리 중...
      🔍 EasyOCR: 1085자 (11.1초)
      📄 pdfplumber: 1168자
      ✅ 완료 - pdfplumber: 1168자
   📄 페이지 226/600 처리 중...
      🔍 EasyOCR: 580자 (8.3초)
      📄 pdfplumber: 629자
      ✅ 완료 - pdfplumber: 629자
   📄 페이지 227/600 처리 중...
    

      🔍 EasyOCR: 598자 (8.7초)
      📄 pdfplumber: 665자
      ✅ 완료 - pdfplumber: 665자
   📄 페이지 292/600 처리 중...
      🔍 EasyOCR: 900자 (9.7초)
      📄 pdfplumber: 977자
      ✅ 완료 - pdfplumber: 977자
   📄 페이지 293/600 처리 중...
      🔍 EasyOCR: 783자 (9.1초)
      📄 pdfplumber: 823자
      ✅ 완료 - pdfplumber: 823자
   📄 페이지 294/600 처리 중...
      🔍 EasyOCR: 765자 (9.5초)
      📄 pdfplumber: 879자
      ✅ 완료 - pdfplumber: 879자
   📄 페이지 295/600 처리 중...
      🔍 EasyOCR: 945자 (9.9초)
      📄 pdfplumber: 1010자
      ✅ 완료 - pdfplumber: 1010자
   📄 페이지 296/600 처리 중...
      🔍 EasyOCR: 1058자 (10.6초)
      📄 pdfplumber: 1126자
      ✅ 완료 - pdfplumber: 1126자
   📄 페이지 297/600 처리 중...
      🔍 EasyOCR: 357자 (7.1초)
      📄 pdfplumber: 393자
      ✅ 완료 - pdfplumber: 393자
   📄 페이지 298/600 처리 중...
      🔍 EasyOCR: 713자 (9.4초)
      📄 pdfplumber: 805자
      ✅ 완료 - pdfplumber: 805자
   📄 페이지 299/600 처리 중...
      🔍 EasyOCR: 873자 (9.7초)
      📄 pdfplumber: 937자
      ✅ 완료 - pdfplumber: 937자
   📄 페이지 300/600 처리 중...
      🔍 EasyO

      🔍 EasyOCR: 1030자 (11.0초)
      📄 pdfplumber: 1107자
      ✅ 완료 - pdfplumber: 1107자
   📄 페이지 364/600 처리 중...
      🔍 EasyOCR: 1072자 (11.1초)
      📄 pdfplumber: 1134자
      ✅ 완료 - pdfplumber: 1134자
   📄 페이지 365/600 처리 중...
      🔍 EasyOCR: 604자 (8.5초)
      📄 pdfplumber: 639자
      ✅ 완료 - pdfplumber: 639자
   📄 페이지 366/600 처리 중...
      🔍 EasyOCR: 677자 (9.2초)
      📄 pdfplumber: 742자
      ✅ 완료 - pdfplumber: 742자
   📄 페이지 367/600 처리 중...
      🔍 EasyOCR: 1085자 (10.7초)
      📄 pdfplumber: 1161자
      ✅ 완료 - pdfplumber: 1161자
   📄 페이지 368/600 처리 중...
      🔍 EasyOCR: 1097자 (11.0초)
      📄 pdfplumber: 1163자
      ✅ 완료 - pdfplumber: 1163자
   📄 페이지 369/600 처리 중...
      🔍 EasyOCR: 1217자 (11.7초)
      📄 pdfplumber: 1282자
      ✅ 완료 - pdfplumber: 1282자
   📄 페이지 370/600 처리 중...
      🔍 EasyOCR: 734자 (9.4초)
      📄 pdfplumber: 773자
      ✅ 완료 - pdfplumber: 773자
   📄 페이지 371/600 처리 중...
      🔍 EasyOCR: 552자 (8.5초)
      📄 pdfplumber: 606자
      ✅ 완료 - pdfplumber: 606자
   📄 페이지 372/600 처리 중...

      🔍 EasyOCR: 607자 (8.4초)
      📄 pdfplumber: 663자
      ✅ 완료 - pdfplumber: 663자
   📄 페이지 437/600 처리 중...
      🔍 EasyOCR: 1024자 (10.1초)
      📄 pdfplumber: 1115자
      ✅ 완료 - pdfplumber: 1115자
   📄 페이지 438/600 처리 중...
      🔍 EasyOCR: 1125자 (10.8초)
      📄 pdfplumber: 1179자
      ✅ 완료 - pdfplumber: 1179자
   📄 페이지 439/600 처리 중...
      🔍 EasyOCR: 702자 (9.1초)
      📄 pdfplumber: 768자
      ✅ 완료 - pdfplumber: 768자
   📄 페이지 440/600 처리 중...
      🔍 EasyOCR: 972자 (10.0초)
      📄 pdfplumber: 1035자
      ✅ 완료 - pdfplumber: 1035자
   📄 페이지 441/600 처리 중...
      🔍 EasyOCR: 860자 (9.8초)
      📄 pdfplumber: 912자
      ✅ 완료 - pdfplumber: 912자
   📄 페이지 442/600 처리 중...
      🔍 EasyOCR: 558자 (8.3초)
      📄 pdfplumber: 635자
      ✅ 완료 - pdfplumber: 635자
   📄 페이지 443/600 처리 중...
      🔍 EasyOCR: 976자 (10.0초)
      📄 pdfplumber: 1050자
      ✅ 완료 - pdfplumber: 1050자
   📄 페이지 444/600 처리 중...
      🔍 EasyOCR: 773자 (9.3초)
      📄 pdfplumber: 804자
      ✅ 완료 - pdfplumber: 804자
   📄 페이지 445/600 처리 중...
     

      🔍 EasyOCR: 607자 (8.2초)
      📄 pdfplumber: 663자
      ✅ 완료 - pdfplumber: 663자
   📄 페이지 510/600 처리 중...
      🔍 EasyOCR: 555자 (8.7초)
      📄 pdfplumber: 591자
      ✅ 완료 - pdfplumber: 591자
   📄 페이지 511/600 처리 중...
      🔍 EasyOCR: 1036자 (10.2초)
      📄 pdfplumber: 1087자
      ✅ 완료 - pdfplumber: 1087자
   📄 페이지 512/600 처리 중...
      🔍 EasyOCR: 701자 (9.1초)
      📄 pdfplumber: 750자
      ✅ 완료 - pdfplumber: 750자
   📄 페이지 513/600 처리 중...
      🔍 EasyOCR: 615자 (9.2초)
      📄 pdfplumber: 693자
      ✅ 완료 - pdfplumber: 693자
   📄 페이지 514/600 처리 중...
      🔍 EasyOCR: 1051자 (10.5초)
      📄 pdfplumber: 1110자
      ✅ 완료 - pdfplumber: 1110자
   📄 페이지 515/600 처리 중...
      🔍 EasyOCR: 1050자 (10.5초)
      📄 pdfplumber: 1123자
      ✅ 완료 - pdfplumber: 1123자
   📄 페이지 516/600 처리 중...
      🔍 EasyOCR: 555자 (8.9초)
      📄 pdfplumber: 609자
      ✅ 완료 - pdfplumber: 609자
   📄 페이지 517/600 처리 중...
      🔍 EasyOCR: 1062자 (10.5초)
      📄 pdfplumber: 1122자
      ✅ 완료 - pdfplumber: 1122자
   📄 페이지 518/600 처리 중...
   

      🔍 EasyOCR: 929자 (9.7초)
      📄 pdfplumber: 980자
      ✅ 완료 - pdfplumber: 980자
   📄 페이지 583/600 처리 중...
      🔍 EasyOCR: 555자 (8.1초)
      📄 pdfplumber: 609자
      ✅ 완료 - pdfplumber: 609자
   📄 페이지 584/600 처리 중...
      🔍 EasyOCR: 581자 (8.9초)
      📄 pdfplumber: 571자
      ✅ 완료 - EasyOCR: 581자
   📄 페이지 585/600 처리 중...
      🔍 EasyOCR: 1009자 (10.2초)
      📄 pdfplumber: 1067자
      ✅ 완료 - pdfplumber: 1067자
   📄 페이지 586/600 처리 중...
      🔍 EasyOCR: 1075자 (10.5초)
      📄 pdfplumber: 1124자
      ✅ 완료 - pdfplumber: 1124자
   📄 페이지 587/600 처리 중...
      🔍 EasyOCR: 735자 (8.8초)
      📄 pdfplumber: 792자
      ✅ 완료 - pdfplumber: 792자
   📄 페이지 588/600 처리 중...
      🔍 EasyOCR: 412자 (7.9초)
      📄 pdfplumber: 426자
      ✅ 완료 - pdfplumber: 426자
   📄 페이지 589/600 처리 중...
      🔍 EasyOCR: 581자 (8.7초)
      📄 pdfplumber: 592자
      ✅ 완료 - pdfplumber: 592자
   📄 페이지 590/600 처리 중...
      🔍 EasyOCR: 806자 (9.6초)
      📄 pdfplumber: 824자
      ✅ 완료 - EasyOCR: 806자
   📄 페이지 591/600 처리 중...
      🔍 EasyOCR: 

# 4.마크다운

In [3]:
import fitz  # PyMuPDF
import pandas as pd
import io
import os
import re

# OCR 라이브러리 선택적 import
OCR_AVAILABLE = False
try:
    import easyocr
    OCR_TYPE = "easyocr"
    OCR_AVAILABLE = True
except ImportError:
    try:
        import pytesseract
        from PIL import Image
        OCR_TYPE = "tesseract"
        OCR_AVAILABLE = True
    except ImportError:
        OCR_TYPE = None
        print("OCR 라이브러리가 없습니다. 이미지 텍스트 추출은 건너뜁니다.")

def extract_text_from_pdf(pdf_path):
    """PDF에서 텍스트, 이미지, 표를 추출하여 마크다운으로 변환"""
    doc = fitz.open(pdf_path)
    markdown_content = []
    
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        
        # 페이지 제목 추가
        markdown_content.append(f"# 페이지 {page_num + 1}\n")
        
        # 텍스트 추출
        text = page.get_text()
        if text.strip():
            # 텍스트 정리 및 구조화
            cleaned_text = clean_text(text)
            markdown_content.append(cleaned_text + "\n")
        
        # 이미지 추출 및 OCR
        if OCR_AVAILABLE:
            image_list = page.get_images(full=True)
            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    
                    if pix.n - pix.alpha < 4:  # GRAY 또는 RGB
                        img_data = pix.tobytes("png")
                        
                        if OCR_TYPE == "easyocr":
                            # EasyOCR 사용
                            reader = easyocr.Reader(['ko', 'en'])
                            results = reader.readtext(img_data)
                            ocr_text = " ".join([result[1] for result in results])
                        else:
                            # Tesseract 사용
                            img_pil = Image.open(io.BytesIO(img_data))
                            ocr_text = pytesseract.image_to_string(img_pil, lang='kor+eng')
                        
                        if ocr_text.strip():
                            markdown_content.append(f"## 이미지 {img_index + 1} (OCR 텍스트)\n")
                            markdown_content.append(f"```\n{ocr_text.strip()}\n```\n")
                    
                    pix = None
                except Exception as e:
                    print(f"이미지 처리 오류 (페이지 {page_num + 1}, 이미지 {img_index + 1}): {e}")
        else:
            # OCR 없이 이미지 정보만 표시
            image_list = page.get_images(full=True)
            if image_list:
                markdown_content.append(f"## 이미지 {len(image_list)}개 발견 (OCR 미지원)\n")
        
        # 표 추출
        tables = extract_tables_from_page(page)
        for table_index, table in enumerate(tables):
            markdown_content.append(f"## 표 {table_index + 1}\n")
            markdown_content.append(table + "\n")
    
    doc.close()
    return "\n".join(markdown_content)

def clean_text(text):
    """텍스트 정리 및 마크다운 구조화"""
    lines = text.split('\n')
    cleaned_lines = []
    
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # 제목 패턴 감지 (번호나 특수문자로 시작하는 줄)
        if re.match(r'^[\d\.\)]+\s+', line) or re.match(r'^[가-힣]+\s*[\.\)]\s+', line):
            cleaned_lines.append(f"### {line}")
        # 강조 텍스트 (대문자나 특수문자가 많은 경우)
        elif len(re.findall(r'[A-Z가-힣]', line)) > len(line) * 0.7:
            cleaned_lines.append(f"**{line}**")
        else:
            cleaned_lines.append(line)
    
    return "\n".join(cleaned_lines)

def extract_tables_from_page(page):
    """페이지에서 표 추출"""
    tables = []
    
    try:
        # 텍스트 블록을 이용한 표 감지
        blocks = page.get_text("dict")["blocks"]
        
        # 표 형태의 텍스트 블록 찾기
        for block in blocks:
            if "lines" in block:
                lines = []
                for line in block["lines"]:
                    spans = []
                    for span in line["spans"]:
                        spans.append(span["text"])
                    line_text = " ".join(spans).strip()
                    if line_text:
                        lines.append(line_text)
                
                # 표 패턴 감지 (탭이나 공백으로 구분된 여러 열)
                if len(lines) > 1:
                    table_data = []
                    for line in lines:
                        # 공백이나 탭으로 구분된 열 분리
                        cols = re.split(r'\s{2,}|\t', line)
                        if len(cols) > 1:
                            table_data.append(cols)
                    
                    if len(table_data) > 1:
                        markdown_table = convert_to_markdown_table(table_data)
                        if markdown_table:
                            tables.append(markdown_table)
    
    except Exception as e:
        print(f"표 추출 오류: {e}")
    
    return tables

def convert_to_markdown_table(table_data):
    """표 데이터를 마크다운 테이블로 변환"""
    if not table_data or len(table_data) < 2:
        return None
    
    # 최대 열 수 계산
    max_cols = max(len(row) for row in table_data)
    
    # 각 행을 동일한 열 수로 맞춤
    normalized_data = []
    for row in table_data:
        while len(row) < max_cols:
            row.append("")
        normalized_data.append(row[:max_cols])
    
    # 마크다운 테이블 생성
    markdown_lines = []
    
    # 헤더
    header = "| " + " | ".join(normalized_data[0]) + " |"
    markdown_lines.append(header)
    
    # 구분선
    separator = "| " + " | ".join(["---"] * max_cols) + " |"
    markdown_lines.append(separator)
    
    # 데이터 행
    for row in normalized_data[1:]:
        data_row = "| " + " | ".join(row) + " |"
        markdown_lines.append(data_row)
    
    return "\n".join(markdown_lines)

def main():
    # PDF 파일 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        print("PDF 변환 시작...")
        markdown_content = extract_text_from_pdf(pdf_path)
        
        # 결과 저장
        output_path = pdf_path.replace('.pdf', '.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        
        print(f"변환 완료: {output_path}")
        
    except Exception as e:
        print(f"변환 중 오류 발생: {e}")

if __name__ == "__main__":
    main()

ModuleNotFoundError: No module named 'fitz'

In [2]:
!pip install PyMuPDF pandas

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     --------------------------------------- 18.7/18.7 MB 11.3 MB/s eta 0:00:00


In [4]:
# 1단계: 필수 패키지 설치
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("필수 패키지 설치 중...")
try:
    install_package("PyMuPDF")
    install_package("pandas")
    print("설치 완료!")
except Exception as e:
    print(f"설치 오류: {e}")

# 2단계: 패키지 import 및 PDF 변환 함수
try:
    import fitz  # PyMuPDF
    import pandas as pd
    import io
    import os
    import re
    print("패키지 import 성공!")
    
    def extract_text_from_pdf_simple(pdf_path):
        """간단한 PDF 텍스트 추출 (OCR 없음)"""
        if not os.path.exists(pdf_path):
            return f"파일을 찾을 수 없습니다: {pdf_path}"
        
        doc = fitz.open(pdf_path)
        markdown_content = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            # 페이지 제목
            markdown_content.append(f"# 페이지 {page_num + 1}\n")
            
            # 텍스트 추출
            text = page.get_text()
            if text.strip():
                # 기본 정리
                lines = text.split('\n')
                cleaned_lines = []
                
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue
                    
                    # 제목 패턴 감지
                    if re.match(r'^[\d\.\)]+\s+', line):
                        cleaned_lines.append(f"### {line}")
                    else:
                        cleaned_lines.append(line)
                
                markdown_content.append("\n".join(cleaned_lines) + "\n")
            
            # 표 추출 시도
            blocks = page.get_text("dict")["blocks"]
            for block_idx, block in enumerate(blocks):
                if "lines" in block:
                    lines = []
                    for line in block["lines"]:
                        spans = []
                        for span in line["spans"]:
                            spans.append(span["text"])
                        line_text = " ".join(spans).strip()
                        if line_text:
                            lines.append(line_text)
                    
                    # 표 패턴 감지
                    if len(lines) > 1:
                        table_data = []
                        for line in lines:
                            cols = re.split(r'\s{3,}|\t', line)
                            if len(cols) > 1:
                                table_data.append(cols)
                        
                        if len(table_data) > 1:
                            markdown_content.append(f"## 표 {block_idx + 1}\n")
                            # 마크다운 테이블 생성
                            max_cols = max(len(row) for row in table_data)
                            for row in table_data:
                                while len(row) < max_cols:
                                    row.append("")
                            
                            # 헤더
                            header = "| " + " | ".join(table_data[0]) + " |"
                            separator = "| " + " | ".join(["---"] * max_cols) + " |"
                            markdown_content.append(header)
                            markdown_content.append(separator)
                            
                            # 데이터 행
                            for row in table_data[1:]:
                                data_row = "| " + " | ".join(row) + " |"
                                markdown_content.append(data_row)
                            markdown_content.append("")
        
        doc.close()
        return "\n".join(markdown_content)
    
    # 3단계: PDF 변환 실행
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("PDF 변환 시작...")
    result = extract_text_from_pdf_simple(pdf_path)
    
    if "파일을 찾을 수 없습니다" in result:
        print(result)
        print("파일 경로를 확인해주세요.")
    else:
        # 결과 저장
        output_path = pdf_path.replace('.pdf', '.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result)
        
        print(f"변환 완료: {output_path}")
        print(f"변환된 내용 길이: {len(result)} 문자")
        
        # 처음 500자 미리보기
        print("\n=== 변환 결과 미리보기 ===")
        print(result[:500])
        print("...")

except ImportError as e:
    print(f"import 오류: {e}")
    print("커널을 다시 시작한 후 다시 실행해보세요.")
except Exception as e:
    print(f"실행 오류: {e}")

필수 패키지 설치 중...
설치 오류: Command '['C:\\Users\\kmj11\\anaconda3\\envs\\gpudm\\python.exe', '-m', 'pip', 'install', 'pandas']' returned non-zero exit status 1.
패키지 import 성공!
PDF 변환 시작...
변환 완료: C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.md
변환된 내용 길이: 561331 문자

=== 변환 결과 미리보기 ===
# 페이지 1

2023.6.
자동차사고
과실비율
인정기준

# 페이지 2

자동차사고 과실비율 인정기준
001
발간사.....................................................................................................................................006
제1편 개정경과..............................................................................................................009
제2편 총 설........................................................................................................................011
### 1. 과실비율 인정기준의 필요성.......................
...


In [5]:
# 변환된 마크다운 파일 내용 확인
import os

def analyze_converted_file(md_path):
    """변환된 마크다운 파일 분석"""
    
    if not os.path.exists(md_path):
        print(f"파일이 없습니다: {md_path}")
        return
    
    with open(md_path, 'r', encoding='utf-8') as f:
        content = f.read()
    
    lines = content.split('\n')
    
    # 기본 통계
    print("=== 변환 파일 분석 ===")
    print(f"총 문자 수: {len(content):,}")
    print(f"총 줄 수: {len(lines):,}")
    
    # 페이지 수 계산
    pages = [line for line in lines if line.startswith('# 페이지')]
    print(f"총 페이지 수: {len(pages)}")
    
    # 제목/구조 분석
    headers = [line for line in lines if line.startswith('#')]
    print(f"제목 개수: {len(headers)}")
    
    # 표 개수
    tables = [line for line in lines if line.startswith('## 표')]
    print(f"표 개수: {len(tables)}")
    
    # 처음 20줄 미리보기
    print("\n=== 처음 20줄 미리보기 ===")
    for i, line in enumerate(lines[:20]):
        if line.strip():
            print(f"{i+1:2d}: {line}")
    
    # 목차 추출
    print("\n=== 주요 제목 구조 ===")
    for line in lines[:100]:  # 처음 100줄에서 제목 찾기
        if line.startswith('### ') and ('.' in line or ')' in line):
            print(line)
    
    return content

# 변환된 파일 분석
md_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.md"
content = analyze_converted_file(md_path)

# 특정 페이지 내용 확인 (예: 첫 번째 페이지)
def show_page_content(content, page_num, max_lines=30):
    """특정 페이지 내용 표시"""
    lines = content.split('\n')
    page_start = -1
    page_end = -1
    
    for i, line in enumerate(lines):
        if line == f"# 페이지 {page_num}":
            page_start = i
        elif line.startswith(f"# 페이지 {page_num + 1}"):
            page_end = i
            break
    
    if page_start == -1:
        print(f"페이지 {page_num}을 찾을 수 없습니다.")
        return
    
    if page_end == -1:
        page_end = len(lines)
    
    print(f"\n=== 페이지 {page_num} 내용 (최대 {max_lines}줄) ===")
    page_lines = lines[page_start:page_end]
    
    for i, line in enumerate(page_lines[:max_lines]):
        if line.strip():
            print(f"{i+1:2d}: {line}")
    
    if len(page_lines) > max_lines:
        print(f"... (총 {len(page_lines)}줄 중 {max_lines}줄만 표시)")

# 첫 번째 페이지 내용 확인
if content:
    show_page_content(content, 1)

=== 변환 파일 분석 ===
총 문자 수: 561,331
총 줄 수: 26,109
총 페이지 수: 600
제목 개수: 1541
표 개수: 1

=== 처음 20줄 미리보기 ===
 1: # 페이지 1
 3: 2023.6.
 4: 자동차사고
 5: 과실비율
 6: 인정기준
 8: # 페이지 2
10: 자동차사고 과실비율 인정기준
11: 001
12: 발간사.....................................................................................................................................006
13: 제1편 개정경과..............................................................................................................009
14: 제2편 총 설........................................................................................................................011
15: ### 1. 과실비율 인정기준의 필요성........................................................................................012
16: ### 2. 과실과 과실상계................................................................................................................014
17: (1) 과실의 의의..................................................................................................................014
18: (2) 피해자 과실상계의 

# 추가보완(마크다운 + 표정제)

In [10]:
# PDF와 동일한 간단한 표 형태로 정제
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("필수 패키지 설치 중...")
try:
    install_package("PyMuPDF")
    install_package("pandas")
    print("설치 완료!")
except Exception as e:
    print(f"설치 오류: {e}")

try:
    import fitz  # PyMuPDF
    import pandas as pd
    import io
    import os
    import re
    print("패키지 import 성공!")
    
    def create_simple_markdown_table(lines, min_cols=2):
        """간단한 마크다운 표 생성 - PDF와 동일하게"""
        
        if len(lines) < 2:
            return None
        
        table_data = []
        
        # 각 줄을 컬럼으로 분리
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # 여러 공백이나 탭으로 구분된 컬럼들 분리
            cols = re.split(r'\s{2,}|\t', line)
            
            # 최소 컬럼 수 확인
            if len(cols) >= min_cols:
                table_data.append(cols)
        
        if len(table_data) < 2:
            return None
        
        # 최대 컬럼 수 계산
        max_cols = max(len(row) for row in table_data)
        
        # 모든 행을 같은 컬럼 수로 맞춤
        for row in table_data:
            while len(row) < max_cols:
                row.append("")
        
        # 마크다운 테이블 생성
        result = []
        
        # 헤더 (첫 번째 행)
        header = "| " + " | ".join(table_data[0]) + " |"
        result.append(header)
        
        # 구분선
        separator = "| " + " | ".join(["---"] * max_cols) + " |"
        result.append(separator)
        
        # 데이터 행들
        for row in table_data[1:]:
            data_row = "| " + " | ".join(row) + " |"
            result.append(data_row)
        
        return "\n".join(result)
    
    def is_table_content(lines):
        """표 내용인지 판별"""
        
        if len(lines) < 2:
            return False
        
        # 여러 컬럼으로 구분된 줄이 2개 이상 있는지 확인
        multi_col_count = 0
        for line in lines:
            cols = re.split(r'\s{3,}|\t', line.strip())
            if len(cols) >= 2:
                multi_col_count += 1
        
        return multi_col_count >= 2
    
    def extract_text_from_pdf_simple(pdf_path):
        """PDF 텍스트 추출 + 간단한 표 정제"""
        
        if not os.path.exists(pdf_path):
            return f"파일을 찾을 수 없습니다: {pdf_path}"
        
        doc = fitz.open(pdf_path)
        markdown_content = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            # 페이지 제목
            markdown_content.append(f"# 페이지 {page_num + 1}\n")
            
            # 기본 텍스트 추출
            text = page.get_text()
            if text.strip():
                lines = text.split('\n')
                cleaned_lines = []
                
                for line in lines:
                    line = line.strip()
                    if not line:
                        continue
                    
                    # 제목 패턴 감지
                    if re.match(r'^[\d\.\)]+\s+', line):
                        cleaned_lines.append(f"### {line}")
                    else:
                        cleaned_lines.append(line)
                
                markdown_content.append("\n".join(cleaned_lines) + "\n")
            
            # 블록별 표 추출
            blocks = page.get_text("dict")["blocks"]
            
            for block_idx, block in enumerate(blocks):
                if "lines" in block:
                    block_lines = []
                    
                    for line in block["lines"]:
                        spans = []
                        for span in line["spans"]:
                            spans.append(span["text"])
                        line_text = " ".join(spans).strip()
                        if line_text:
                            block_lines.append(line_text)
                    
                    # 표인지 확인
                    if is_table_content(block_lines):
                        table_markdown = create_simple_markdown_table(block_lines)
                        if table_markdown:
                            markdown_content.append(f"## 표 {block_idx + 1}")
                            markdown_content.append("")
                            markdown_content.append(table_markdown)
                            markdown_content.append("")
        
        doc.close()
        return "\n".join(markdown_content)
    
    # PDF 변환 실행
    pdf_files = [
        r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf",
    ]
    
    for pdf_path in pdf_files:
        print(f"\n{'='*60}")
        print(f"PDF 변환 시작: {os.path.basename(pdf_path)}")
        print(f"{'='*60}")
        
        result = extract_text_from_pdf_simple(pdf_path)
        
        if "파일을 찾을 수 없습니다" in result:
            print(result)
            continue
        
        # 결과 저장
        output_path = pdf_path.replace('.pdf', '_간단표정제.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result)
        
        print(f"변환 완료: {output_path}")
        print(f"변환된 내용 길이: {len(result):,} 문자")
        
        # 미리보기
        print("\n=== 변환 결과 미리보기 ===")
        lines = result.split('\n')
        for i, line in enumerate(lines[:50]):
            if line.strip():
                print(f"{i+1:2d}: {line}")
            elif line == "":
                print(f"{i+1:2d}: ")
        print("...")

except ImportError as e:
    print(f"import 오류: {e}")
except Exception as e:
    print(f"실행 오류: {e}")

print("\n설명:")
print("- 기존 텍스트는 그대로 유지")
print("- 표는 간단한 마크다운 테이블 형태로 변환")
print("- | 컬럼1 | 컬럼2 | 컬럼3 | 형태")
print("- |-------|-------|-------|")
print("- PDF 구조와 최대한 동일하게 유지")

필수 패키지 설치 중...
설치 오류: Command '['C:\\Users\\kmj11\\anaconda3\\envs\\gpudm\\python.exe', '-m', 'pip', 'install', 'pandas']' returned non-zero exit status 1.
패키지 import 성공!

PDF 변환 시작: 231107_과실비율인정기준_온라인용.pdf
변환 완료: C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용_간단표정제.md
변환된 내용 길이: 561,331 문자

=== 변환 결과 미리보기 ===
 1: # 페이지 1
 2: 
 3: 2023.6.
 4: 자동차사고
 5: 과실비율
 6: 인정기준
 7: 
 8: # 페이지 2
 9: 
10: 자동차사고 과실비율 인정기준
11: 001
12: 발간사.....................................................................................................................................006
13: 제1편 개정경과..............................................................................................................009
14: 제2편 총 설........................................................................................................................011
15: ### 1. 과실비율 인정기준의 필요성........................................................................................012
16: ### 2. 과실과 과실상계...............

In [12]:
# TXT 형태로 가장 깔끔한 표 추출 비교
import subprocess
import sys

def install_packages():
    packages = ["PyMuPDF", "pdfplumber"]
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"{package} 설치 완료")
        except:
            print(f"{package} 설치 실패")

install_packages()

try:
    import fitz
    import os
    import re
    
    try:
        import pdfplumber
        PDFPLUMBER_AVAILABLE = True
    except:
        PDFPLUMBER_AVAILABLE = False
    
    def method1_raw_text(pdf_path):
        """방법 1: 순수 원본 텍스트 (가장 원본에 가까움)"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(3, len(doc))):
            page = doc.load_page(page_num)
            text = page.get_text()
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            all_text.append(text)
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method2_structured_text(pdf_path):
        """방법 2: 구조화된 텍스트 (블록별 정리)"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(3, len(doc))):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            
            for block_idx, block in enumerate(blocks):
                if "lines" in block:
                    block_text = []
                    
                    for line in block["lines"]:
                        line_parts = []
                        for span in line["spans"]:
                            if span["text"].strip():
                                line_parts.append(span["text"].strip())
                        
                        if line_parts:
                            block_text.append(" ".join(line_parts))
                    
                    if block_text:
                        all_text.append(f"\n--- 블록 {block_idx + 1} ---")
                        all_text.append("\n".join(block_text))
            
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method3_clean_lines(pdf_path):
        """방법 3: 줄바꿈 정리된 깔끔한 텍스트"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(3, len(doc))):
            page = doc.load_page(page_num)
            text = page.get_text()
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            
            lines = text.split('\n')
            clean_lines = []
            
            for line in lines:
                line = line.strip()
                if line:  # 빈 줄 제거
                    clean_lines.append(line)
            
            # 연속된 중복 줄 제거
            final_lines = []
            prev_line = ""
            
            for line in clean_lines:
                if line != prev_line:  # 중복 줄 제거
                    final_lines.append(line)
                prev_line = line
            
            all_text.append("\n".join(final_lines))
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method4_table_focused(pdf_path):
        """방법 4: 표 영역만 집중 추출"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(3, len(doc))):
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            
            all_text.append(f"=== 페이지 {page_num + 1} - 표 영역만 ===")
            
            for block_idx, block in enumerate(blocks):
                if "lines" in block and len(block["lines"]) > 1:
                    
                    # 블록 텍스트 추출
                    block_lines = []
                    for line in block["lines"]:
                        line_text = ""
                        for span in line["spans"]:
                            line_text += span["text"]
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    # 표처럼 보이는 블록 판별 (여러 컬럼이 있는지)
                    multi_column_count = 0
                    for line in block_lines:
                        if len(re.split(r'\s{2,}', line)) >= 2:
                            multi_column_count += 1
                    
                    # 표로 판단되면 출력
                    if multi_column_count >= 2:
                        all_text.append(f"\n--- 표 후보 블록 {block_idx + 1} ---")
                        for line in block_lines:
                            all_text.append(line)
                        all_text.append("")
            
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method5_pdfplumber_text(pdf_path):
        """방법 5: pdfplumber로 텍스트 추출"""
        
        if not PDFPLUMBER_AVAILABLE:
            return "pdfplumber 사용 불가"
        
        all_text = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages[:3]):
                all_text.append(f"=== 페이지 {page_num + 1} (pdfplumber) ===")
                
                # 전체 텍스트
                page_text = page.extract_text()
                if page_text:
                    all_text.append(page_text)
                
                all_text.append("\n" + "="*50 + "\n")
        
        return "\n".join(all_text)
    
    def compare_all_methods(pdf_path):
        """모든 방법 비교"""
        
        if not os.path.exists(pdf_path):
            print(f"파일 없음: {pdf_path}")
            return
        
        print(f"TXT 추출 방법 비교: {os.path.basename(pdf_path)}")
        print("=" * 60)
        
        methods = [
            ("1_원본텍스트", method1_raw_text),
            ("2_구조화텍스트", method2_structured_text),
            ("3_깔끔한텍스트", method3_clean_lines),
            ("4_표영역만", method4_table_focused),
            ("5_pdfplumber", method5_pdfplumber_text)
        ]
        
        for method_name, method_func in methods:
            print(f"\n{method_name} 추출 중...")
            
            try:
                result = method_func(pdf_path)
                
                if result and len(result) > 100:
                    # TXT 파일로 저장
                    output_path = pdf_path.replace('.pdf', f'_{method_name}.txt')
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(result)
                    
                    print(f"저장 완료: {output_path}")
                    print(f"파일 크기: {len(result):,} 문자")
                    
                    # 처음 20줄 미리보기
                    lines = result.split('\n')
                    print("미리보기:")
                    for i, line in enumerate(lines[:20]):
                        if line.strip():
                            print(f"  {i+1:2d}: {line[:80]}")  # 80자까지만
                    print("...")
                    
                else:
                    print(f"결과 없음: {result}")
                    
            except Exception as e:
                print(f"오류: {e}")
            
            print("-" * 40)
    
    # 비교 실행
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    compare_all_methods(pdf_path)
    
    print("\n\n결과 분석:")
    print("1. 각 방법의 TXT 파일을 열어서 비교해보세요")
    print("2. 표가 가장 깔끔하게 나온 방법을 확인하세요")
    print("3. 선택된 방법으로 두 번째 PDF도 처리하세요")
    
    print("\n예상 결과:")
    print("- 원본텍스트: PDF 그대로, 레이아웃 유지")
    print("- 구조화텍스트: 블록별 정리, 읽기 쉬움")
    print("- 깔끔한텍스트: 중복 제거, 간결함")
    print("- 표영역만: 표로 보이는 부분만 추출")
    print("- pdfplumber: 다른 엔진으로 추출")

except Exception as e:
    print(f"전체 오류: {e}")

PyMuPDF 설치 완료
pdfplumber 설치 완료
TXT 추출 방법 비교: 231107_과실비율인정기준_온라인용.pdf

1_원본텍스트 추출 중...
저장 완료: C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용_1_원본텍스트.txt
파일 크기: 5,293 문자
미리보기:
   1: === 페이지 1 ===
   2: 2023.6.
   3: 자동차사고
   4: 과실비율 
   5: 인정기준
   8: ==================================================
  10: === 페이지 2 ===
  11: 자동차사고 과실비율 인정기준
  12: 001
  13: 발간사.............................................................................
  14: 제1편 개정경과........................................................................
  15: 제2편 총 설.........................................................................
  16: 1. 과실비율 인정기준의 필요성...............................................................
  17: 2. 과실과 과실상계.....................................................................
  18: (1) 과실의 의의......................................................................
  19: (2) 피해자 과실상계의 의의................................................................
  20: (3) 피해자 

In [13]:
# TXT 형태로 가장 깔끔한 표 추출 비교
import subprocess
import sys

def install_packages():
    packages = ["PyMuPDF", "pdfplumber"]
    for package in packages:
        try:
            subprocess.check_call([sys.executable, "-m", "pip", "install", package])
            print(f"{package} 설치 완료")
        except:
            print(f"{package} 설치 실패")

install_packages()

try:
    import fitz
    import os
    import re
    
    try:
        import pdfplumber
        PDFPLUMBER_AVAILABLE = True
    except:
        PDFPLUMBER_AVAILABLE = False
    
    def method1_raw_text(pdf_path):
        """방법 1: 순수 원본 텍스트 (가장 원본에 가까움)"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(100, len(doc))):  # 100페이지까지
            page = doc.load_page(page_num)
            text = page.get_text()
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            all_text.append(text)
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method2_structured_text(pdf_path):
        """방법 2: 구조화된 텍스트 (블록별 정리)"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(100, len(doc))):  # 100페이지까지
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            
            for block_idx, block in enumerate(blocks):
                if "lines" in block:
                    block_text = []
                    
                    for line in block["lines"]:
                        line_parts = []
                        for span in line["spans"]:
                            if span["text"].strip():
                                line_parts.append(span["text"].strip())
                        
                        if line_parts:
                            block_text.append(" ".join(line_parts))
                    
                    if block_text:
                        all_text.append(f"\n--- 블록 {block_idx + 1} ---")
                        all_text.append("\n".join(block_text))
            
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method3_clean_lines(pdf_path):
        """방법 3: 줄바꿈 정리된 깔끔한 텍스트"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(100, len(doc))):  # 100페이지까지
            page = doc.load_page(page_num)
            text = page.get_text()
            
            all_text.append(f"=== 페이지 {page_num + 1} ===")
            
            lines = text.split('\n')
            clean_lines = []
            
            for line in lines:
                line = line.strip()
                if line:  # 빈 줄 제거
                    clean_lines.append(line)
            
            # 연속된 중복 줄 제거
            final_lines = []
            prev_line = ""
            
            for line in clean_lines:
                if line != prev_line:  # 중복 줄 제거
                    final_lines.append(line)
                prev_line = line
            
            all_text.append("\n".join(final_lines))
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method4_table_focused(pdf_path):
        """방법 4: 표 영역만 집중 추출"""
        
        doc = fitz.open(pdf_path)
        all_text = []
        
        for page_num in range(min(100, len(doc))):  # 100페이지까지
            page = doc.load_page(page_num)
            blocks = page.get_text("dict")["blocks"]
            
            all_text.append(f"=== 페이지 {page_num + 1} - 표 영역만 ===")
            
            for block_idx, block in enumerate(blocks):
                if "lines" in block and len(block["lines"]) > 1:
                    
                    # 블록 텍스트 추출
                    block_lines = []
                    for line in block["lines"]:
                        line_text = ""
                        for span in line["spans"]:
                            line_text += span["text"]
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    # 표처럼 보이는 블록 판별 (여러 컬럼이 있는지)
                    multi_column_count = 0
                    for line in block_lines:
                        if len(re.split(r'\s{2,}', line)) >= 2:
                            multi_column_count += 1
                    
                    # 표로 판단되면 출력
                    if multi_column_count >= 2:
                        all_text.append(f"\n--- 표 후보 블록 {block_idx + 1} ---")
                        for line in block_lines:
                            all_text.append(line)
                        all_text.append("")
            
            all_text.append("\n" + "="*50 + "\n")
        
        doc.close()
        return "\n".join(all_text)
    
    def method5_pdfplumber_text(pdf_path):
        """방법 5: pdfplumber로 텍스트 추출"""
        
        if not PDFPLUMBER_AVAILABLE:
            return "pdfplumber 사용 불가"
        
        all_text = []
        
        with pdfplumber.open(pdf_path) as pdf:
            for page_num, page in enumerate(pdf.pages[:100]):  # 100페이지까지
                all_text.append(f"=== 페이지 {page_num + 1} (pdfplumber) ===")
                
                # 전체 텍스트
                page_text = page.extract_text()
                if page_text:
                    all_text.append(page_text)
                
                all_text.append("\n" + "="*50 + "\n")
        
        return "\n".join(all_text)
    
    def compare_all_methods(pdf_path):
        """모든 방법 비교"""
        
        if not os.path.exists(pdf_path):
            print(f"파일 없음: {pdf_path}")
            return
        
        print(f"TXT 추출 방법 비교: {os.path.basename(pdf_path)}")
        print("=" * 60)
        
        methods = [
            ("1_원본텍스트", method1_raw_text),
            ("2_구조화텍스트", method2_structured_text),
            ("3_깔끔한텍스트", method3_clean_lines),
            ("4_표영역만", method4_table_focused),
            ("5_pdfplumber", method5_pdfplumber_text)
        ]
        
        for method_name, method_func in methods:
            print(f"\n{method_name} 추출 중...")
            
            try:
                result = method_func(pdf_path)
                
                if result and len(result) > 100:
                    # TXT 파일로 저장
                    output_path = pdf_path.replace('.pdf', f'_{method_name}.txt')
                    with open(output_path, 'w', encoding='utf-8') as f:
                        f.write(result)
                    
                    print(f"저장 완료: {output_path}")
                    print(f"파일 크기: {len(result):,} 문자")
                    
                    # 처음 20줄 미리보기
                    lines = result.split('\n')
                    print("미리보기:")
                    for i, line in enumerate(lines[:20]):
                        if line.strip():
                            print(f"  {i+1:2d}: {line[:80]}")  # 80자까지만
                    print("...")
                    
                else:
                    print(f"결과 없음: {result}")
                    
            except Exception as e:
                print(f"오류: {e}")
            
            print("-" * 40)
    
    # 비교 실행
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    compare_all_methods(pdf_path)
    
    print("\n\n결과 분석:")
    print("1. 각 방법의 TXT 파일을 열어서 비교해보세요")
    print("2. 표가 가장 깔끔하게 나온 방법을 확인하세요")
    print("3. 선택된 방법으로 두 번째 PDF도 처리하세요")
    
    print("\n예상 결과:")
    print("- 원본텍스트: PDF 그대로, 레이아웃 유지")
    print("- 구조화텍스트: 블록별 정리, 읽기 쉬움")
    print("- 깔끔한텍스트: 중복 제거, 간결함")
    print("- 표영역만: 표로 보이는 부분만 추출")
    print("- pdfplumber: 다른 엔진으로 추출")

except Exception as e:
    print(f"전체 오류: {e}")

PyMuPDF 설치 완료
pdfplumber 설치 완료
TXT 추출 방법 비교: 231107_과실비율인정기준_온라인용.pdf

1_원본텍스트 추출 중...
저장 완료: C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용_1_원본텍스트.txt
파일 크기: 107,096 문자
미리보기:
   1: === 페이지 1 ===
   2: 2023.6.
   3: 자동차사고
   4: 과실비율 
   5: 인정기준
   8: ==================================================
  10: === 페이지 2 ===
  11: 자동차사고 과실비율 인정기준
  12: 001
  13: 발간사.............................................................................
  14: 제1편 개정경과........................................................................
  15: 제2편 총 설.........................................................................
  16: 1. 과실비율 인정기준의 필요성...............................................................
  17: 2. 과실과 과실상계.....................................................................
  18: (1) 과실의 의의......................................................................
  19: (2) 피해자 과실상계의 의의................................................................
  20: (3) 피해

In [14]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

print("필수 패키지 설치 중...")
try:
    install_package("PyMuPDF")
    print("설치 완료!")
except Exception as e:
    print(f"설치 오류: {e}")

try:
    import fitz
    import re
    import os
    
    def create_markdown_table(lines):
        """여러 줄을 마크다운 테이블로 변환"""
        
        if len(lines) < 2:
            return None
        
        # 각 줄을 공백으로 분리하여 컬럼 만들기
        table_rows = []
        for line in lines:
            line = line.strip()
            if not line:
                continue
            
            # 2개 이상의 공백 또는 탭으로 컬럼 분리
            columns = re.split(r'\s{2,}|\t+', line)
            if len(columns) >= 2:  # 최소 2개 컬럼 필요
                table_rows.append(columns)
        
        if len(table_rows) < 2:
            return None
        
        # 모든 행의 컬럼 수를 맞춤
        max_cols = max(len(row) for row in table_rows)
        for row in table_rows:
            while len(row) < max_cols:
                row.append("")
        
        # 마크다운 테이블 생성
        result = []
        
        # 첫 번째 행을 헤더로
        header = "| " + " | ".join(table_rows[0]) + " |"
        result.append(header)
        
        # 구분선
        separator = "| " + " | ".join(["-" * max(3, len(col)) for col in table_rows[0]]) + " |"
        result.append(separator)
        
        # 나머지 행들
        for row in table_rows[1:]:
            data_row = "| " + " | ".join(row) + " |"
            result.append(data_row)
        
        return "\n".join(result)
    
    def is_likely_table(lines):
        """줄들이 표일 가능성이 있는지 판단"""
        
        if len(lines) < 2:
            return False
        
        multi_column_count = 0
        for line in lines:
            # 2개 이상의 공백으로 분리되는 컬럼이 2개 이상인지
            columns = re.split(r'\s{2,}|\t+', line.strip())
            if len(columns) >= 2:
                multi_column_count += 1
        
        # 전체 줄의 절반 이상이 다중 컬럼이면 표로 판단
        return multi_column_count >= max(2, len(lines) // 2)
    
    def extract_pdf_to_markdown_with_tables(pdf_path):
        """PDF를 표가 포함된 마크다운으로 변환"""
        
        if not os.path.exists(pdf_path):
            return f"파일을 찾을 수 없습니다: {pdf_path}"
        
        doc = fitz.open(pdf_path)
        markdown_content = []
        
        print(f"PDF 변환 중: {os.path.basename(pdf_path)}")
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            
            print(f"  페이지 {page_num + 1}/{len(doc)} 처리 중...")
            
            # 페이지 제목
            markdown_content.append(f"# 페이지 {page_num + 1}")
            markdown_content.append("")
            
            # 일반 텍스트 추출
            page_text = page.get_text()
            text_lines = page_text.split('\n')
            
            # 텍스트를 정리하고 제목 구조 만들기
            processed_lines = []
            for line in text_lines:
                line = line.strip()
                if not line:
                    continue
                
                # 제목 패턴 감지
                if re.match(r'^\d+\.\s+', line):  # "1. 제목"
                    processed_lines.append(f"## {line}")
                elif re.match(r'^\(\d+\)\s+', line):  # "(1) 제목"
                    processed_lines.append(f"### {line}")
                elif re.match(r'^\d+\)\s+', line):  # "1) 제목"
                    processed_lines.append(f"#### {line}")
                else:
                    processed_lines.append(line)
            
            # 일반 텍스트 추가
            if processed_lines:
                markdown_content.extend(processed_lines)
                markdown_content.append("")
            
            # 블록별로 표 추출 시도
            blocks = page.get_text("dict")["blocks"]
            table_count = 0
            
            for block in blocks:
                if "lines" in block:
                    block_lines = []
                    
                    # 블록에서 텍스트 라인들 추출
                    for line in block["lines"]:
                        line_text = ""
                        for span in line["spans"]:
                            line_text += span["text"]
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    # 이 블록이 표인지 확인
                    if is_likely_table(block_lines):
                        table_markdown = create_markdown_table(block_lines)
                        if table_markdown:
                            table_count += 1
                            markdown_content.append(f"### 표 {table_count}")
                            markdown_content.append("")
                            markdown_content.append(table_markdown)
                            markdown_content.append("")
            
            # 페이지 구분선
            markdown_content.append("---")
            markdown_content.append("")
        
        doc.close()
        
        final_content = "\n".join(markdown_content)
        print(f"변환 완료! 총 {len(final_content):,} 문자")
        
        return final_content
    
    # PDF 파일들 처리
    pdf_files = [
        r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf",
    ]
    
    for pdf_path in pdf_files:
        print(f"\n{'='*60}")
        print(f"처리 시작: {os.path.basename(pdf_path)}")
        print(f"{'='*60}")
        
        result = extract_pdf_to_markdown_with_tables(pdf_path)
        
        if "파일을 찾을 수 없습니다" in result:
            print(result)
            continue
        
        # 마크다운 파일로 저장
        output_path = pdf_path.replace('.pdf', '_표포함.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(result)
        
        print(f"저장 완료: {output_path}")
        
        # 표가 얼마나 생성되었는지 확인
        table_count = result.count("### 표")
        print(f"생성된 표 개수: {table_count}개")
        
        # 미리보기 (표 부분 위주로)
        print("\n=== 표 관련 내용 미리보기 ===")
        lines = result.split('\n')
        in_table = False
        preview_count = 0
        
        for i, line in enumerate(lines):
            if "### 표" in line:
                print(f"\n{line}")
                in_table = True
                preview_count = 0
            elif in_table and preview_count < 10:
                print(line)
                preview_count += 1
                if line == "---" or preview_count >= 10:
                    in_table = False
                    print("...")
                    break

except ImportError as e:
    print(f"import 오류: {e}")
except Exception as e:
    print(f"실행 오류: {e}")

print("\n완료!")
print("생성된 .md 파일을 열어서 표가 제대로 변환되었는지 확인하세요.")

필수 패키지 설치 중...
설치 완료!

처리 시작: 231107_과실비율인정기준_온라인용.pdf
PDF 변환 중: 231107_과실비율인정기준_온라인용.pdf
  페이지 1/600 처리 중...
  페이지 2/600 처리 중...
  페이지 3/600 처리 중...
  페이지 4/600 처리 중...
  페이지 5/600 처리 중...
  페이지 6/600 처리 중...
  페이지 7/600 처리 중...
  페이지 8/600 처리 중...
  페이지 9/600 처리 중...
  페이지 10/600 처리 중...
  페이지 11/600 처리 중...
  페이지 12/600 처리 중...
  페이지 13/600 처리 중...
  페이지 14/600 처리 중...
  페이지 15/600 처리 중...
  페이지 16/600 처리 중...
  페이지 17/600 처리 중...
  페이지 18/600 처리 중...
  페이지 19/600 처리 중...
  페이지 20/600 처리 중...
  페이지 21/600 처리 중...
  페이지 22/600 처리 중...
  페이지 23/600 처리 중...
  페이지 24/600 처리 중...
  페이지 25/600 처리 중...
  페이지 26/600 처리 중...
  페이지 27/600 처리 중...
  페이지 28/600 처리 중...
  페이지 29/600 처리 중...
  페이지 30/600 처리 중...
  페이지 31/600 처리 중...
  페이지 32/600 처리 중...
  페이지 33/600 처리 중...
  페이지 34/600 처리 중...
  페이지 35/600 처리 중...
  페이지 36/600 처리 중...
  페이지 37/600 처리 중...
  페이지 38/600 처리 중...
  페이지 39/600 처리 중...
  페이지 40/600 처리 중...
  페이지 41/600 처리 중...
  페이지 42/600 처리 중...
  페이지 43/600 처리 중...
  페이지 44/600 처리 

  페이지 399/600 처리 중...
  페이지 400/600 처리 중...
  페이지 401/600 처리 중...
  페이지 402/600 처리 중...
  페이지 403/600 처리 중...
  페이지 404/600 처리 중...
  페이지 405/600 처리 중...
  페이지 406/600 처리 중...
  페이지 407/600 처리 중...
  페이지 408/600 처리 중...
  페이지 409/600 처리 중...
  페이지 410/600 처리 중...
  페이지 411/600 처리 중...
  페이지 412/600 처리 중...
  페이지 413/600 처리 중...
  페이지 414/600 처리 중...
  페이지 415/600 처리 중...
  페이지 416/600 처리 중...
  페이지 417/600 처리 중...
  페이지 418/600 처리 중...
  페이지 419/600 처리 중...
  페이지 420/600 처리 중...
  페이지 421/600 처리 중...
  페이지 422/600 처리 중...
  페이지 423/600 처리 중...
  페이지 424/600 처리 중...
  페이지 425/600 처리 중...
  페이지 426/600 처리 중...
  페이지 427/600 처리 중...
  페이지 428/600 처리 중...
  페이지 429/600 처리 중...
  페이지 430/600 처리 중...
  페이지 431/600 처리 중...
  페이지 432/600 처리 중...
  페이지 433/600 처리 중...
  페이지 434/600 처리 중...
  페이지 435/600 처리 중...
  페이지 436/600 처리 중...
  페이지 437/600 처리 중...
  페이지 438/600 처리 중...
  페이지 439/600 처리 중...
  페이지 440/600 처리 중...
  페이지 441/600 처리 중...
  페이지 442/600 처리 중...
  페이지 443/600 처리 중...
  페이지 444/

In [15]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    install_package("PyMuPDF")
except:
    pass

try:
    import fitz
    import re
    import os
    
    def debug_pdf_structure(pdf_path, start_page=35, end_page=45):
        """PDF 구조 디버깅 - 표가 있을 만한 페이지들 분석"""
        
        if not os.path.exists(pdf_path):
            print(f"파일 없음: {pdf_path}")
            return
        
        doc = fitz.open(pdf_path)
        
        print(f"PDF 구조 분석: 페이지 {start_page}-{end_page}")
        print("="*60)
        
        for page_num in range(start_page-1, min(end_page, len(doc))):
            page = doc.load_page(page_num)
            
            print(f"\n=== 페이지 {page_num + 1} ===")
            
            # 방법 1: 일반 텍스트
            text = page.get_text()
            print("일반 텍스트 (처음 10줄):")
            for i, line in enumerate(text.split('\n')[:10]):
                if line.strip():
                    print(f"  {i+1}: {line}")
            
            # 방법 2: 블록별 분석
            blocks = page.get_text("dict")["blocks"]
            print(f"\n블록 수: {len(blocks)}")
            
            for block_idx, block in enumerate(blocks[:3]):  # 처음 3개 블록만
                if "lines" in block:
                    print(f"\n블록 {block_idx + 1}:")
                    block_lines = []
                    for line in block["lines"]:
                        line_text = ""
                        for span in line["spans"]:
                            line_text += span["text"]
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    for i, line in enumerate(block_lines[:5]):  # 블록당 5줄만
                        print(f"    {i+1}: {line}")
            
            # 방법 3: 특정 키워드 찾기
            keywords = ['보1', '차1', '과실비율', '조정예시', '①', '②', '③']
            found_keywords = []
            for keyword in keywords:
                if keyword in text:
                    found_keywords.append(keyword)
            
            if found_keywords:
                print(f"\n발견된 키워드: {found_keywords}")
                
                # 키워드 주변 텍스트 보기
                for keyword in found_keywords[:2]:  # 처음 2개만
                    lines = text.split('\n')
                    for i, line in enumerate(lines):
                        if keyword in line:
                            print(f"\n'{keyword}' 주변 텍스트:")
                            start = max(0, i-3)
                            end = min(len(lines), i+5)
                            for j in range(start, end):
                                marker = ">>> " if j == i else "    "
                                print(f"{marker}{j+1:3d}: {lines[j]}")
                            break
            
            print("-" * 40)
        
        doc.close()
    
    def extract_with_pattern_matching(pdf_path):
        """패턴 매칭으로 표 영역 찾기"""
        
        doc = fitz.open(pdf_path)
        found_tables = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text()
            lines = text.split('\n')
            
            # 사례 패턴 찾기 (보1, 차1 등)
            for i, line in enumerate(lines):
                if re.search(r'보\d+|차\d+', line) and '사고' in line:
                    print(f"\n페이지 {page_num + 1}에서 사례 발견: {line}")
                    
                    # 해당 줄 주변 15줄 추출
                    start = max(0, i-5)
                    end = min(len(lines), i+15)
                    
                    table_area = []
                    for j in range(start, end):
                        if lines[j].strip():
                            table_area.append(lines[j].strip())
                    
                    print("추출된 영역:")
                    for k, area_line in enumerate(table_area):
                        print(f"  {k+1:2d}: {area_line}")
                    
                    found_tables.append({
                        'page': page_num + 1,
                        'content': table_area,
                        'title': line.strip()
                    })
        
        doc.close()
        return found_tables
    
    def manual_table_creation(found_tables):
        """찾은 표 영역을 수동으로 마크다운 테이블로 변환"""
        
        markdown_content = []
        markdown_content.append("# 과실비율 표 (수동 정제)")
        markdown_content.append("")
        
        for table in found_tables:
            markdown_content.append(f"## 페이지 {table['page']}: {table['title']}")
            markdown_content.append("")
            
            # 패턴별로 다르게 처리
            content = table['content']
            
            # 참여자 정보 찾기
            participants = []
            basic_ratio = ""
            adjustments = []
            
            for line in content:
                # (보), (차) 패턴
                if re.match(r'^\([보차]\)', line):
                    participants.append(line)
                # 기본 과실비율
                elif '기본' in line and '과실' in line:
                    basic_ratio = line
                # 조정 요소 (①, ②, +, - 포함)
                elif re.search(r'[①②③④⑤⑥⑦⑧⑨⑩]|[+\-]\d+', line):
                    adjustments.append(line)
            
            # 마크다운으로 구성
            if participants:
                markdown_content.append("### 상황")
                for p in participants:
                    markdown_content.append(f"- {p}")
                markdown_content.append("")
            
            if basic_ratio:
                markdown_content.append("### 기본 과실비율")
                markdown_content.append(f"- {basic_ratio}")
                markdown_content.append("")
            
            if adjustments:
                markdown_content.append("### 조정 요소")
                markdown_content.append("")
                markdown_content.append("| 구분 | 내용 | 조정값 |")
                markdown_content.append("|------|------|--------|")
                
                for adj in adjustments:
                    # 조정값 분리
                    if re.search(r'[+\-]\d+', adj):
                        match = re.search(r'(.+?)([+\-]\d+)', adj)
                        if match:
                            desc = match.group(1).strip()
                            value = match.group(2)
                            # 번호 분리
                            number_match = re.match(r'([①②③④⑤⑥⑦⑧⑨⑩])(.*)', desc)
                            if number_match:
                                number = number_match.group(1)
                                desc = number_match.group(2).strip()
                                markdown_content.append(f"| {number} | {desc} | {value} |")
                            else:
                                markdown_content.append(f"| | {desc} | {value} |")
                        else:
                            markdown_content.append(f"| | {adj} | |")
                    else:
                        # 비적용 등
                        if '비적용' in adj:
                            desc = adj.replace('비적용', '').strip()
                            markdown_content.append(f"| | {desc} | 비적용 |")
                        else:
                            markdown_content.append(f"| | {adj} | |")
                
                markdown_content.append("")
            
            # 원본 텍스트도 참고용으로 추가
            markdown_content.append("### 원본 텍스트 (참고)")
            markdown_content.append("```")
            for line in content:
                markdown_content.append(line)
            markdown_content.append("```")
            markdown_content.append("")
            markdown_content.append("---")
            markdown_content.append("")
        
        return "\n".join(markdown_content)
    
    # 실행
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("1단계: PDF 구조 디버깅")
    debug_pdf_structure(pdf_path, 35, 45)
    
    print("\n\n2단계: 패턴 매칭으로 표 찾기")
    found_tables = extract_with_pattern_matching(pdf_path)
    
    print(f"\n찾은 표 개수: {len(found_tables)}")
    
    if found_tables:
        print("\n3단계: 수동 테이블 생성")
        manual_markdown = manual_table_creation(found_tables)
        
        # 결과 저장
        output_path = pdf_path.replace('.pdf', '_수동표정제.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(manual_markdown)
        
        print(f"수동 정제 결과 저장: {output_path}")
        
        # 미리보기
        print("\n=== 수동 정제 결과 미리보기 ===")
        lines = manual_markdown.split('\n')
        for i, line in enumerate(lines[:30]):
            print(f"{i+1:2d}: {line}")
        print("...")
    else:
        print("표를 찾지 못했습니다.")
        print("다른 페이지 범위를 시도해보거나 키워드를 조정해보세요.")

except Exception as e:
    print(f"오류: {e}")

print("\n해결 방안:")
print("1. 디버깅 결과를 보고 표가 어떤 형태로 되어있는지 확인")
print("2. 키워드나 페이지 범위를 조정해서 재시도")
print("3. 필요시 중요한 표만 수동으로 입력")
print("4. 또는 다른 PDF 추출 도구 사용 고려")

1단계: PDF 구조 디버깅
PDF 구조 분석: 페이지 35-45

=== 페이지 35 ===
일반 텍스트 (처음 10줄):
  1: 제1장. 자동차와 보행자의 사고
  2: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
  3: 034
  4: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의 
  5: 조명으로 인하여 자동차의 운전자가 전조등에 의하지 않더라도 보행자의 발견이 용이한 
  6: 장소에서의 사고는 가산하지 않는다.
  7: (2) 횡단보도 부근
  8: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지 
  9: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
  10: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.

블록 수: 29

블록 1:
    1: 제1장. 자동차와 보행자의 사고

블록 2:
    1: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
    2: 034

블록 3:
    1: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의

발견된 키워드: ['보1', '과실비율', '③']

'보1' 주변 텍스트:
      6: 장소에서의 사고는 가산하지 않는다.
      7: (2) 횡단보도 부근
      8: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지 
>>>   9: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
     10: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.
     11: (3) 간선도로
     12: - ‘간선도로’라 함은 차도폭이 20m 이상이거나 또는 왕복 6차로 이상의 도로 또는 제한시속 
     13: 80km 이상인 도로로서 교통량이 많은 도로를 말한다.




페이지 35에서 사례 발견: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
추출된 영역:
   1: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의
   2: 조명으로 인하여 자동차의 운전자가 전조등에 의하지 않더라도 보행자의 발견이 용이한
   3: 장소에서의 사고는 가산하지 않는다.
   4: (2) 횡단보도 부근
   5: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지
   6: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그
   7: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.
   8: (3) 간선도로
   9: - ‘간선도로’라 함은 차도폭이 20m 이상이거나 또는 왕복 6차로 이상의 도로 또는 제한시속
  10: 80km 이상인 도로로서 교통량이 많은 도로를 말한다.
  11: - 간선도로인 경우 차량의 통행이 많고 차량이 고속주행을 하는 반면 보행자의 도로횡단 등을
  12: 도와주는 시설물이 설치된 경우가 많으므로 보행자의 과실을 가산할 수 있다.
  13: (4) 정지·후퇴·ㄹ자 보행
  14: - 보행자가 횡단 중 갑자기 멈추어 서는 경우(정지), 다시 돌아서서 출발점으로 돌아가거나
  15: 뒷걸음질하는 경우(후퇴), 차도를 ㄹ자로 걸어가거나 또는 어슬렁거리는 경우에 보행자의
  16: 과실을 가산할 수 있다.
  17: (5) 횡단규제표지
  18: - 횡단금지표시 등의 안전표지 또는 가드레일, 펜스, 차단봉 등에 의하여 차도횡단이 금지된
  19: 장소를 횡단하는 경우에는 보행자의 과실을 가산할 수 있다.
  20: (6) 교차로 대각선 횡단

페이지 71에서 사례 발견: 는 차량과의 사고는 보17, 보18 기준을 적용한다. 다만 차량의 우회전임을 알리는 신호기
추출된 영역:
   1: 되기 어려운 경우에는 보행자의 과실을 가산하지 않는


페이지 316에서 사례 발견: 1) 2개 차량이 나란히 통행 가능한 차로폭에서의 사고 [차19] 
추출된 영역:
   1: 제2장. 자동차와 자동차(이륜차 포함)의 사고
   2: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
   3: 315
   4: (6) 교차로 부근 동시 우회전 내지 좌회전 사고
   5: 1) 2개 차량이 나란히 통행 가능한 차로폭에서의 사고 [차19]
   6: 차19-1
   7: 후행 직진 대 선행 좌(우)회전
   8: (A) 후행 직진
   9: (B) 좌(우)회전
  10: 기본 과실비율
  11: A20 B80
  12: 과
  13: 실
  14: 비
  15: 율
  16: 조
  17: 정
  18: 예
  19: 시

페이지 318에서 사례 발견: 2) 동시 우회전 사고 [차20] 
추출된 영역:
   1: 제2장. 자동차와 자동차(이륜차 포함)의 사고
   2: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
   3: 317
   4: 2) 동시 우회전 사고 [차20]
   5: 차20-1
   6: 우측 우회전 대 좌측 우회전
   7: (A) 우회전(오른쪽차)
   8: (B) 우회전(왼쪽차)
   9: 기본 과실비율
  10: A40 B60
  11: 과
  12: 실
  13: 비
  14: 율
  15: 조
  16: 정
  17: 예
  18: 시

페이지 319에서 사례 발견: 발생한 사고는 차20-2를 적용한다.
추출된 영역:
   1: ⊙제3편 제2장 3. 수정요소의 해설 부분을 참조한다.
   2: 활용시 참고 사항
   3: ⊙B차량이 앞에 나아가 먼저(선행하여) 우회전 대기 중인데, 그 우측 빈 공간이 ‘매우’ 좁은데도
   4: A차량이 무리하게 나중에 진입하였다가 B차량이 우회전 출발하면서 그 회전반경으로
   5: 발생한 사고는 차20-2를 적용한다.
   6: 관련 법규
   7: ⊙도로교통법 제14조(차로의 설치 등)
   8: ② 차마의


찾은 표 개수: 79

3단계: 수동 테이블 생성
수동 정제 결과 저장: C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용_수동표정제.md

=== 수동 정제 결과 미리보기 ===
 1: # 과실비율 표 (수동 정제)
 2: 
 3: ## 페이지 3: (6) 기타 사고유형 [보29~보36].......................................................................................106
 4: 
 5: ### 원본 텍스트 (참고)
 6: ```
 7: (4) 횡단시설 부근(신호등 없음) [보20~보21]................................................................084
 8: (5) 횡단보도 없음..............................................................................................................089
 9: 1) 도로 유형별 [보22~보23]........................................................................................089
10: 2) 보도와 차도(구분 있음) [보24~보26]......................................................................096
11: 3) 보도와 차도(구분 없음) [보27~보28]......................................................................100
12: (6) 기타 사고유형 [보29~보36].......................................................................................106
1

In [16]:
import subprocess
import sys

def install_package(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

try:
    install_package("PyMuPDF")
except:
    pass

try:
    import fitz
    import re
    import os
    
    def debug_pdf_structure(pdf_path, start_page=35, end_page=45):
        """PDF 구조 디버깅 - 표가 있을 만한 페이지들 분석"""
        
        if not os.path.exists(pdf_path):
            print(f"파일 없음: {pdf_path}")
            return
        
        doc = fitz.open(pdf_path)
        
        print(f"PDF 구조 분석: 페이지 {start_page}-{end_page}")
        print("="*60)
        
        for page_num in range(start_page-1, min(end_page, len(doc))):
            page = doc.load_page(page_num)
            
            print(f"\n=== 페이지 {page_num + 1} ===")
            
            # 방법 1: 일반 텍스트
            text = page.get_text()
            print("일반 텍스트 (처음 10줄):")
            for i, line in enumerate(text.split('\n')[:10]):
                if line.strip():
                    print(f"  {i+1}: {line}")
            
            # 방법 2: 블록별 분석
            blocks = page.get_text("dict")["blocks"]
            print(f"\n블록 수: {len(blocks)}")
            
            for block_idx, block in enumerate(blocks[:3]):  # 처음 3개 블록만
                if "lines" in block:
                    print(f"\n블록 {block_idx + 1}:")
                    block_lines = []
                    for line in block["lines"]:
                        line_text = ""
                        for span in line["spans"]:
                            line_text += span["text"]
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    for i, line in enumerate(block_lines[:5]):  # 블록당 5줄만
                        print(f"    {i+1}: {line}")
            
            # 방법 3: 특정 키워드 찾기
            keywords = ['보1', '차1', '과실비율', '조정예시', '①', '②', '③']
            found_keywords = []
            for keyword in keywords:
                if keyword in text:
                    found_keywords.append(keyword)
            
            if found_keywords:
                print(f"\n발견된 키워드: {found_keywords}")
                
                # 키워드 주변 텍스트 보기
                for keyword in found_keywords[:2]:  # 처음 2개만
                    lines = text.split('\n')
                    for i, line in enumerate(lines):
                        if keyword in line:
                            print(f"\n'{keyword}' 주변 텍스트:")
                            start = max(0, i-3)
                            end = min(len(lines), i+5)
                            for j in range(start, end):
                                marker = ">>> " if j == i else "    "
                                print(f"{marker}{j+1:3d}: {lines[j]}")
                            break
            
            print("-" * 40)
        
        doc.close()
    
    def extract_with_pattern_matching(pdf_path):
        """패턴 매칭으로 표 영역 찾기"""
        
        doc = fitz.open(pdf_path)
        found_tables = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            text = page.get_text()
            lines = text.split('\n')
            
            # 사례 패턴 찾기 (보1, 차1 등)
            for i, line in enumerate(lines):
                if re.search(r'보\d+|차\d+', line) and '사고' in line:
                    print(f"\n페이지 {page_num + 1}에서 사례 발견: {line}")
                    
                    # 해당 줄 주변 15줄 추출
                    start = max(0, i-5)
                    end = min(len(lines), i+15)
                    
                    table_area = []
                    for j in range(start, end):
                        if lines[j].strip():
                            table_area.append(lines[j].strip())
                    
                    print("추출된 영역:")
                    for k, area_line in enumerate(table_area):
                        print(f"  {k+1:2d}: {area_line}")
                    
                    found_tables.append({
                        'page': page_num + 1,
                        'content': table_area,
                        'title': line.strip()
                    })
        
        doc.close()
        return found_tables
    
    def manual_table_creation(found_tables):
        """찾은 표 영역을 수동으로 마크다운 테이블로 변환"""
        
        markdown_content = []
        markdown_content.append("# 과실비율 표 (수동 정제)")
        markdown_content.append("")
        
        for table in found_tables:
            markdown_content.append(f"## 페이지 {table['page']}: {table['title']}")
            markdown_content.append("")
            
            # 패턴별로 다르게 처리
            content = table['content']
            
            # 참여자 정보 찾기
            participants = []
            basic_ratio = ""
            adjustments = []
            
            for line in content:
                # (보), (차) 패턴
                if re.match(r'^\([보차]\)', line):
                    participants.append(line)
                # 기본 과실비율
                elif '기본' in line and '과실' in line:
                    basic_ratio = line
                # 조정 요소 (①, ②, +, - 포함)
                elif re.search(r'[①②③④⑤⑥⑦⑧⑨⑩]|[+\-]\d+', line):
                    adjustments.append(line)
            
            # 마크다운으로 구성
            if participants:
                markdown_content.append("### 상황")
                for p in participants:
                    markdown_content.append(f"- {p}")
                markdown_content.append("")
            
            if basic_ratio:
                markdown_content.append("### 기본 과실비율")
                markdown_content.append(f"- {basic_ratio}")
                markdown_content.append("")
            
            if adjustments:
                markdown_content.append("### 조정 요소")
                markdown_content.append("")
                markdown_content.append("| 구분 | 내용 | 조정값 |")
                markdown_content.append("|------|------|--------|")
                
                for adj in adjustments:
                    # 조정값 분리
                    if re.search(r'[+\-]\d+', adj):
                        match = re.search(r'(.+?)([+\-]\d+)', adj)
                        if match:
                            desc = match.group(1).strip()
                            value = match.group(2)
                            # 번호 분리
                            number_match = re.match(r'([①②③④⑤⑥⑦⑧⑨⑩])(.*)', desc)
                            if number_match:
                                number = number_match.group(1)
                                desc = number_match.group(2).strip()
                                markdown_content.append(f"| {number} | {desc} | {value} |")
                            else:
                                markdown_content.append(f"| | {desc} | {value} |")
                        else:
                            markdown_content.append(f"| | {adj} | |")
                    else:
                        # 비적용 등
                        if '비적용' in adj:
                            desc = adj.replace('비적용', '').strip()
                            markdown_content.append(f"| | {desc} | 비적용 |")
                        else:
                            markdown_content.append(f"| | {adj} | |")
                
                markdown_content.append("")
            
            # 원본 텍스트도 참고용으로 추가
            markdown_content.append("### 원본 텍스트 (참고)")
            markdown_content.append("```")
            for line in content:
                markdown_content.append(line)
            markdown_content.append("```")
            markdown_content.append("")
            markdown_content.append("---")
            markdown_content.append("")
        
        return "\n".join(markdown_content)
    
    # 실행
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    print("1단계: PDF 구조 디버깅")
    debug_pdf_structure(pdf_path, 35, 45)
    
    print("\n\n2단계: 패턴 매칭으로 표 찾기")
    found_tables = extract_with_pattern_matching(pdf_path)
    
    print(f"\n찾은 표 개수: {len(found_tables)}")
    
    if found_tables:
        print("\n3단계: 수동 테이블 생성")
        manual_markdown = manual_table_creation(found_tables)
        
        # 결과 저장
        output_path = pdf_path.replace('.pdf', '_수동표정제.md')
        with open(output_path, 'w', encoding='utf-8') as f:
            f.write(manual_markdown)
        
        print(f"수동 정제 결과 저장: {output_path}")
        
        # 미리보기
        print("\n=== 수동 정제 결과 미리보기 ===")
        lines = manual_markdown.split('\n')
        for i, line in enumerate(lines[:30]):
            print(f"{i+1:2d}: {line}")
        print("...")
    else:
        print("표를 찾지 못했습니다.")
        print("다른 페이지 범위를 시도해보거나 키워드를 조정해보세요.")

except Exception as e:
    print(f"오류: {e}")

print("\n해결 방안:")
print("1. 디버깅 결과를 보고 표가 어떤 형태로 되어있는지 확인")
print("2. 키워드나 페이지 범위를 조정해서 재시도")
print("3. 필요시 중요한 표만 수동으로 입력")
print("4. 또는 다른 PDF 추출 도구 사용 고려")

1단계: PDF 구조 디버깅
PDF 구조 분석: 페이지 35-45

=== 페이지 35 ===
일반 텍스트 (처음 10줄):
  1: 제1장. 자동차와 보행자의 사고
  2: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
  3: 034
  4: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의 
  5: 조명으로 인하여 자동차의 운전자가 전조등에 의하지 않더라도 보행자의 발견이 용이한 
  6: 장소에서의 사고는 가산하지 않는다.
  7: (2) 횡단보도 부근
  8: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지 
  9: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
  10: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.

블록 수: 29

블록 1:
    1: 제1장. 자동차와 보행자의 사고

블록 2:
    1: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
    2: 034

블록 3:
    1: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의

발견된 키워드: ['보1', '과실비율', '③']

'보1' 주변 텍스트:
      6: 장소에서의 사고는 가산하지 않는다.
      7: (2) 횡단보도 부근
      8: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지 
>>>   9: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
     10: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.
     11: (3) 간선도로
     12: - ‘간선도로’라 함은 차도폭이 20m 이상이거나 또는 왕복 6차로 이상의 도로 또는 제한시속 
     13: 80km 이상인 도로로서 교통량이 많은 도로를 말한다.




페이지 35에서 사례 발견: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그 
추출된 영역:
   1: 또는 차 뒤에서 도로를 횡단하는 경우에도 보행자의 과실을 가산하지 않는다. 가로등 등의
   2: 조명으로 인하여 자동차의 운전자가 전조등에 의하지 않더라도 보행자의 발견이 용이한
   3: 장소에서의 사고는 가산하지 않는다.
   4: (2) 횡단보도 부근
   5: - ‘횡단보도 부근’이라는 개념은 횡단보도로부터 10m 내외의 지점을 말하고, 10~30m까지
   6: 지점의 사고는 보14~보20 기준에 적용된 과실에 각 10%의 보행자 과실을 가산하며, 그
   7: 거리를 넘는 지점에서의 사고는 무단횡단의 예를 적용한다.
   8: (3) 간선도로
   9: - ‘간선도로’라 함은 차도폭이 20m 이상이거나 또는 왕복 6차로 이상의 도로 또는 제한시속
  10: 80km 이상인 도로로서 교통량이 많은 도로를 말한다.
  11: - 간선도로인 경우 차량의 통행이 많고 차량이 고속주행을 하는 반면 보행자의 도로횡단 등을
  12: 도와주는 시설물이 설치된 경우가 많으므로 보행자의 과실을 가산할 수 있다.
  13: (4) 정지·후퇴·ㄹ자 보행
  14: - 보행자가 횡단 중 갑자기 멈추어 서는 경우(정지), 다시 돌아서서 출발점으로 돌아가거나
  15: 뒷걸음질하는 경우(후퇴), 차도를 ㄹ자로 걸어가거나 또는 어슬렁거리는 경우에 보행자의
  16: 과실을 가산할 수 있다.
  17: (5) 횡단규제표지
  18: - 횡단금지표시 등의 안전표지 또는 가드레일, 펜스, 차단봉 등에 의하여 차도횡단이 금지된
  19: 장소를 횡단하는 경우에는 보행자의 과실을 가산할 수 있다.
  20: (6) 교차로 대각선 횡단

페이지 71에서 사례 발견: 는 차량과의 사고는 보17, 보18 기준을 적용한다. 다만 차량의 우회전임을 알리는 신호기
추출된 영역:
   1: 되기 어려운 경우에는 보행자의 과실을 가산하지 않는


페이지 335에서 사례 발견: 침범한 경우에는 중앙선 침범사고인 차31-1 기준을 적용하고 본 기준을 적용하지 아니한다.
추출된 영역:
   1: 334
   2: 활용시 참고 사항
   3: ⊙도로가 아닌 장소 부분에 마을입구, 주유소, 음식점 등이 있어 중앙선을 넘어 도로가 아닌
   4: 장소로 나가는 경우에 본 기준을 적용하고, 양 차량이 교행하다가 그 이외의 사유로 중앙선을
   5: 침범한 경우에는 중앙선 침범사고인 차31-1 기준을 적용하고 본 기준을 적용하지 아니한다.
   6: ⊙중앙선의 의미는 추월을 위해 안전에 주의하여 침범할 수 있지만(실선은 원칙적으로 금지
   7: 됨) 즉시 본래 차로로 복귀해야 한다는 것이므로 도로에 설치된 중앙선이 실선이든 점선이든
   8: 상관없이 적용된다.
   9: 관련 법규
  10: ⊙도로교통법 제13조(차마의 통행)
  11: ③ 차마의 운전자는 도로(보도와 차도가 구분된 도로에서는 차도를 말한다)의 중앙(중앙선이
  12: 설치되어 있는 경우에는 그 중앙선을 말한다. 이하 같다) 우측 부분을 통행하여야 한다.
  13: ⊙도로교통법 제18조(횡단 등의 금지)
  14: ① 차마의 운전자는 보행자나 다른 차마의 정상적인 통행을 방해할 우려가 있는 경우에는
  15: 차마를 운전하여 도로를 횡단하거나 유턴 또는 후진하여서는 아니 된다.

페이지 340에서 사례 발견: (2) 중앙선 없거나 중앙선침범 미적용 도로에서 교행 사고 [차32] 
추출된 영역:
   1: 제2장. 자동차와 자동차(이륜차 포함)의 사고
   2: 자동차사고 과실비율 인정기준 │ 제3편 사고유형별 과실비율 적용기준
   3: 339
   4: (2) 중앙선 없거나 중앙선침범 미적용 도로에서 교행 사고 [차32]
   5: 차32-1
   6: 직진 대 맞은편 직진(교행사고)
   7: (A) 직진
   8: (B) 맞은편 직진
   9: 기본 과실비율
  10: A50 B50
  11: 과
  12: 실
  13: 비
  14: 율


찾은 표 개수: 79

3단계: 수동 테이블 생성
수동 정제 결과 저장: C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용_수동표정제.md

=== 수동 정제 결과 미리보기 ===
 1: # 과실비율 표 (수동 정제)
 2: 
 3: ## 페이지 3: (6) 기타 사고유형 [보29~보36].......................................................................................106
 4: 
 5: ### 원본 텍스트 (참고)
 6: ```
 7: (4) 횡단시설 부근(신호등 없음) [보20~보21]................................................................084
 8: (5) 횡단보도 없음..............................................................................................................089
 9: 1) 도로 유형별 [보22~보23]........................................................................................089
10: 2) 보도와 차도(구분 있음) [보24~보26]......................................................................096
11: 3) 보도와 차도(구분 없음) [보27~보28]......................................................................100
12: (6) 기타 사고유형 [보29~보36].......................................................................................106
1

# 5. 표 추출 텍스트화

In [1]:
import cv2
import numpy as np
import fitz  # PyMuPDF
import pytesseract
import pandas as pd
import os
import re
from pathlib import Path

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

class FaultRatioPDFProcessor:
    def __init__(self):
        self.extracted_images = []
        self.structured_results = []
        self.stats = {
            'total_images': 0,
            'successful_extractions': 0,
            'table_structures_found': 0
        }
    
    def process_complete_pdf(self, pdf_path):
        """PDF 전체 처리: 이미지 추출 → 구조화된 텍스트 변환"""
        
        print("🚀 과실비율 PDF 완전 처리 시작")
        print("=" * 60)
        print(f"📄 파일: {os.path.basename(pdf_path)}")
        
        if not os.path.exists(pdf_path):
            print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
            return
        
        # 1단계: PDF에서 이미지 추출
        print(f"\n🔍 1단계: PDF에서 이미지 추출")
        image_paths = self._extract_images_from_pdf(pdf_path)
        
        if not image_paths:
            print("❌ 추출된 이미지가 없습니다.")
            return
        
        # 2단계: 각 이미지를 구조화된 텍스트로 변환
        print(f"\n🏗️ 2단계: 이미지 → 구조화된 텍스트 변환")
        self._process_all_images(image_paths)
        
        # 3단계: 통합 결과 생성
        print(f"\n📝 3단계: 통합 마크다운 파일 생성")
        self._create_final_document(pdf_path)
        
        # 4단계: 통계 출력
        self._print_final_statistics()
    
    def _extract_images_from_pdf(self, pdf_path):
        """PDF에서 이미지들을 추출"""
        output_dir = "extracted_images"
        Path(output_dir).mkdir(exist_ok=True)
        
        doc = fitz.open(pdf_path)
        image_paths = []
        
        print(f"📊 총 페이지 수: {len(doc)}")
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            if image_list:
                print(f"📄 페이지 {page_num+1}: {len(image_list)}개 이미지 발견")
            
            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    
                    if pix.n - pix.alpha < 4:  # GRAY or RGB
                        img_path = f"{output_dir}/page_{page_num+1}_img_{img_index+1}.png"
                        pix.save(img_path)
                        image_paths.append(img_path)
                        self.stats['total_images'] += 1
                    
                    pix = None
                except Exception as e:
                    print(f"  ⚠️ 이미지 추출 실패 - 페이지 {page_num+1}, 이미지 {img_index+1}: {str(e)}")
        
        doc.close()
        print(f"✅ 총 {len(image_paths)}개 이미지 추출 완료")
        return image_paths
    
    def _process_all_images(self, image_paths):
        """모든 이미지를 구조화된 텍스트로 변환"""
        
        for i, img_path in enumerate(image_paths, 1):
            filename = os.path.basename(img_path)
            progress = f"{i}/{len(image_paths)} ({i/len(image_paths)*100:.1f}%)"
            
            print(f"처리 중: {progress} - {filename}")
            
            # 이미지 분석 및 텍스트 추출
            result = self._analyze_single_image(img_path)
            
            if result:
                self.structured_results.append(result)
                self.stats['successful_extractions'] += 1
                
                if result.get('has_table_structure'):
                    self.stats['table_structures_found'] += 1
                    print(f"  ✅ 표 구조 감지됨")
                else:
                    print(f"  ✅ 텍스트 추출 완료")
            else:
                print(f"  ❌ 처리 실패")
    
    def _analyze_single_image(self, image_path):
        """단일 이미지 분석 및 구조화"""
        try:
            img = cv2.imread(image_path)
            if img is None:
                return None
            
            height, width = img.shape[:2]
            
            # 이미지가 표 형태인지 판단
            is_table_like = self._detect_table_structure(img)
            
            if is_table_like:
                # 표 형태로 처리
                structured_text = self._extract_table_structure(img)
                has_table = True
            else:
                # 일반 텍스트로 처리
                structured_text = self._extract_general_text(img)
                has_table = False
            
            return {
                'image_file': os.path.basename(image_path),
                'page_info': self._extract_page_info(image_path),
                'image_size': f"{width}x{height}",
                'has_table_structure': has_table,
                'structured_content': structured_text,
                'extraction_method': 'table' if has_table else 'general'
            }
            
        except Exception as e:
            print(f"    ❌ 분석 오류: {e}")
            return None
    
    def _detect_table_structure(self, img):
        """이미지가 표 구조인지 감지"""
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # 1. 라인 감지 (표의 격자선)
            edges = cv2.Canny(gray, 50, 150)
            
            # 수평선 감지
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (40, 1))
            horizontal_lines = cv2.morphologyEx(edges, cv2.MORPH_OPEN, horizontal_kernel)
            
            # 수직선 감지  
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 40))
            vertical_lines = cv2.morphologyEx(edges, cv2.MORPH_OPEN, vertical_kernel)
            
            # 라인이 충분히 있으면 표로 판단
            h_lines = cv2.countNonZero(horizontal_lines)
            v_lines = cv2.countNonZero(vertical_lines)
            
            # 2. 텍스트 기반 판단
            quick_text = pytesseract.image_to_string(gray, config='--psm 6 -l kor+eng')
            has_table_keywords = any(word in quick_text for word in ['과실', '비율', '%', '가산', '감산'])
            has_numbers = bool(re.search(r'\d+', quick_text))
            
            # 표 구조 점수 계산
            table_score = 0
            if h_lines > 100: table_score += 2
            if v_lines > 100: table_score += 2
            if has_table_keywords: table_score += 3
            if has_numbers: table_score += 1
            
            return table_score >= 4
            
        except:
            return False
    
    def _extract_table_structure(self, img):
        """표 구조를 유지하며 텍스트 추출"""
        try:
            # 이미지 전처리
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # 크기 확대 (OCR 정확도 향상)
            scale_factor = 3
            height, width = gray.shape
            resized = cv2.resize(gray, (width * scale_factor, height * scale_factor), 
                               interpolation=cv2.INTER_CUBIC)
            
            # 샤프닝
            kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            sharpened = cv2.filter2D(resized, -1, kernel)
            
            # 노이즈 제거
            denoised = cv2.medianBlur(sharpened, 3)
            
            # OCR with layout preservation
            config = r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1'
            text = pytesseract.image_to_string(denoised, config=config)
            
            if not text.strip():
                return "텍스트를 추출할 수 없습니다."
            
            # 표 형태로 구조화
            return self._format_as_markdown_table(text)
            
        except Exception as e:
            return f"표 추출 오류: {e}"
    
    def _extract_general_text(self, img):
        """일반 텍스트 추출"""
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # 기본 OCR
            config = r'--oem 3 --psm 6 -l kor+eng'
            text = pytesseract.image_to_string(gray, config=config)
            
            return text.strip() if text.strip() else "텍스트가 감지되지 않았습니다."
            
        except Exception as e:
            return f"텍스트 추출 오류: {e}"
    
    def _format_as_markdown_table(self, text):
        """텍스트를 마크다운 표로 포맷팅"""
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        if not lines:
            return "추출된 텍스트가 없습니다."
        
        # 과실비율 관련 라인만 필터링
        table_lines = []
        for line in lines:
            if (any(char.isdigit() for char in line) or 
                any(keyword in line for keyword in ['과실', '비율', '%', '가산', '감산', '보행자', '차량', '신호', '위반'])):
                table_lines.append(line)
        
        if not table_lines:
            # 모든 텍스트를 코드 블록으로 반환
            return f"```\n{chr(10).join(lines)}\n```"
        
        # 마크다운 표 생성
        table_data = []
        
        for line in table_lines:
            # 여러 공백을 구분자로 사용
            cells = re.split(r'\s{2,}', line)
            
            # 단일 공백으로도 시도
            if len(cells) == 1:
                cells = line.split()
            
            # 셀 정리
            clean_cells = [cell.strip() for cell in cells if cell.strip()]
            
            if len(clean_cells) >= 1:  # 최소 1열
                table_data.append(clean_cells)
        
        if not table_data:
            return f"```\n{chr(10).join(lines)}\n```"
        
        # 표 생성
        markdown_lines = []
        
        # 가장 많은 열 수 찾기
        max_cols = max(len(row) for row in table_data) if table_data else 2
        
        # 기본 헤더 생성
        headers = ['구분', '내용'] + [f'항목{i}' for i in range(3, max_cols + 1)]
        headers = headers[:max_cols]
        
        markdown_lines.append("| " + " | ".join(headers) + " |")
        markdown_lines.append("|" + "|".join([" --- " for _ in headers]) + "|")
        
        # 데이터 행들 추가
        for row in table_data:
            # 열 수 맞추기
            padded_row = row + [''] * (max_cols - len(row))
            padded_row = padded_row[:max_cols]
            
            markdown_lines.append("| " + " | ".join(padded_row) + " |")
        
        return "\n".join(markdown_lines)
    
    def _extract_page_info(self, image_path):
        """이미지 파일명에서 페이지 정보 추출"""
        filename = os.path.basename(image_path)
        match = re.search(r'page_(\d+)_img_(\d+)', filename)
        if match:
            return f"페이지 {match.group(1)}, 이미지 {match.group(2)}"
        return filename
    
    def _create_final_document(self, pdf_path):
        """최종 통합 마크다운 문서 생성"""
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_file = f"{base_name}_구조화된텍스트.md"
        
        content = f"""# {base_name} - 구조화된 텍스트 추출 결과

> 추출 일시: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
> 총 이미지: {self.stats['total_images']}개
> 성공 추출: {self.stats['successful_extractions']}개
> 표 구조 감지: {self.stats['table_structures_found']}개

---

"""
        
        # 각 결과별로 섹션 생성
        for i, result in enumerate(self.structured_results, 1):
            content += f"""## {i}. {result['page_info']}

**파일**: `{result['image_file']}`  
**크기**: {result['image_size']}  
**추출 방식**: {result['extraction_method']}  
**표 구조**: {'✅ 감지됨' if result['has_table_structure'] else '❌ 없음'}

### 추출된 내용

{result['structured_content']}

---

"""
        
        # 파일 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"✅ 최종 문서 생성: {output_file}")
        return output_file
    
    def _print_final_statistics(self):
        """최종 통계 출력"""
        print(f"\n📊 처리 완료 - 최종 통계")
        print(f"=" * 40)
        print(f"총 이미지 수: {self.stats['total_images']}")
        print(f"성공 추출: {self.stats['successful_extractions']}")
        print(f"실패: {self.stats['total_images'] - self.stats['successful_extractions']}")
        print(f"표 구조 감지: {self.stats['table_structures_found']}")
        
        if self.stats['total_images'] > 0:
            success_rate = (self.stats['successful_extractions'] / self.stats['total_images']) * 100
            print(f"성공률: {success_rate:.1f}%")

def main():
    """메인 실행 함수"""
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    processor = FaultRatioPDFProcessor()
    processor.process_complete_pdf(pdf_path)

if __name__ == "__main__":
    main()

🚀 과실비율 PDF 완전 처리 시작
📄 파일: 231107_과실비율인정기준_온라인용.pdf

🔍 1단계: PDF에서 이미지 추출
📊 총 페이지 수: 600
📄 페이지 39: 1개 이미지 발견
📄 페이지 43: 1개 이미지 발견
📄 페이지 47: 2개 이미지 발견
📄 페이지 50: 2개 이미지 발견
📄 페이지 54: 1개 이미지 발견
📄 페이지 57: 2개 이미지 발견
📄 페이지 61: 2개 이미지 발견
📄 페이지 64: 1개 이미지 발견
📄 페이지 67: 1개 이미지 발견
📄 페이지 70: 1개 이미지 발견
📄 페이지 74: 2개 이미지 발견
📄 페이지 78: 2개 이미지 발견
📄 페이지 82: 1개 이미지 발견
📄 페이지 85: 2개 이미지 발견
📄 페이지 90: 1개 이미지 발견
📄 페이지 94: 1개 이미지 발견
📄 페이지 97: 2개 이미지 발견
📄 페이지 98: 1개 이미지 발견
📄 페이지 101: 1개 이미지 발견
📄 페이지 104: 2개 이미지 발견
📄 페이지 107: 1개 이미지 발견
📄 페이지 109: 1개 이미지 발견
📄 페이지 111: 1개 이미지 발견
📄 페이지 114: 2개 이미지 발견
📄 페이지 117: 1개 이미지 발견
📄 페이지 119: 1개 이미지 발견
📄 페이지 121: 2개 이미지 발견
📄 페이지 129: 1개 이미지 발견
📄 페이지 148: 1개 이미지 발견
📄 페이지 152: 1개 이미지 발견
📄 페이지 155: 1개 이미지 발견
📄 페이지 158: 1개 이미지 발견
📄 페이지 160: 1개 이미지 발견
📄 페이지 164: 1개 이미지 발견
📄 페이지 167: 1개 이미지 발견
📄 페이지 170: 1개 이미지 발견
📄 페이지 172: 1개 이미지 발견
📄 페이지 175: 1개 이미지 발견
📄 페이지 178: 1개 이미지 발견
📄 페이지 180: 2개 이미지 발견
📄 페이지 183: 1개 이미지 발견
📄 페이지 186: 1개 이미지 발견
📄 페이지 189: 1개 이미지 발견
📄 페이지 192: 1개 이미지 발견
📄 페이지 1

  ✅ 표 구조 감지됨
처리 중: 79/218 (36.2%) - page_256_img_1.png
  ✅ 표 구조 감지됨
처리 중: 80/218 (36.7%) - page_259_img_1.png
  ✅ 표 구조 감지됨
처리 중: 81/218 (37.2%) - page_262_img_1.png
  ✅ 표 구조 감지됨
처리 중: 82/218 (37.6%) - page_265_img_1.png
  ✅ 표 구조 감지됨
처리 중: 83/218 (38.1%) - page_268_img_1.png
  ✅ 표 구조 감지됨
처리 중: 84/218 (38.5%) - page_271_img_1.png
  ✅ 표 구조 감지됨
처리 중: 85/218 (39.0%) - page_275_img_1.png
  ✅ 표 구조 감지됨
처리 중: 86/218 (39.4%) - page_278_img_1.png
  ✅ 표 구조 감지됨
처리 중: 87/218 (39.9%) - page_281_img_1.png
  ✅ 표 구조 감지됨
처리 중: 88/218 (40.4%) - page_285_img_1.png
  ✅ 표 구조 감지됨
처리 중: 89/218 (40.8%) - page_288_img_1.png
  ✅ 표 구조 감지됨
처리 중: 90/218 (41.3%) - page_291_img_1.png
  ✅ 표 구조 감지됨
처리 중: 91/218 (41.7%) - page_294_img_1.png
  ✅ 표 구조 감지됨
처리 중: 92/218 (42.2%) - page_294_img_2.png
  ✅ 표 구조 감지됨
처리 중: 93/218 (42.7%) - page_298_img_1.png
  ✅ 표 구조 감지됨
처리 중: 94/218 (43.1%) - page_298_img_2.png
  ✅ 표 구조 감지됨
처리 중: 95/218 (43.6%) - page_302_img_1.png
  ✅ 표 구조 감지됨
처리 중: 96/218 (44.0%) - page_305_img_1.png
  ✅ 표 구조 감

In [2]:
import cv2
import numpy as np
import fitz  # PyMuPDF
import pytesseract
import pandas as pd
import os
import re
from pathlib import Path

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

class SmartFaultRatioPDFProcessor:
    def __init__(self):
        self.meaningful_results = []
        self.filtered_out = []
        self.stats = {
            'total_images': 0,
            'meaningful_extractions': 0,
            'filtered_out_count': 0,
            'actual_tables': 0
        }
    
    def process_pdf_smart_filter(self, pdf_path):
        """PDF 처리 - 의미있는 표만 추출"""
        
        print("🧠 스마트 과실비율 PDF 처리 시작")
        print("=" * 60)
        print(f"📄 파일: {os.path.basename(pdf_path)}")
        
        if not os.path.exists(pdf_path):
            print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
            return
        
        # 1단계: PDF에서 이미지 추출
        print(f"\n🔍 1단계: PDF에서 이미지 추출")
        image_paths = self._extract_images_from_pdf(pdf_path)
        
        if not image_paths:
            print("❌ 추출된 이미지가 없습니다.")
            return
        
        # 2단계: 이미지 사전 필터링 및 처리
        print(f"\n🎯 2단계: 의미있는 이미지만 선별 처리")
        self._process_with_smart_filtering(image_paths)
        
        # 3단계: 최종 결과 생성
        print(f"\n📝 3단계: 의미있는 결과만 통합")
        self._create_filtered_document(pdf_path)
        
        # 4단계: 통계 출력
        self._print_smart_statistics()
    
    def _extract_images_from_pdf(self, pdf_path):
        """PDF에서 이미지들을 추출 (기존과 동일)"""
        output_dir = "extracted_images"
        Path(output_dir).mkdir(exist_ok=True)
        
        doc = fitz.open(pdf_path)
        image_paths = []
        
        for page_num in range(len(doc)):
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            for img_index, img in enumerate(image_list):
                try:
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    
                    if pix.n - pix.alpha < 4:
                        img_path = f"{output_dir}/page_{page_num+1}_img_{img_index+1}.png"
                        pix.save(img_path)
                        image_paths.append(img_path)
                        self.stats['total_images'] += 1
                    
                    pix = None
                except Exception as e:
                    continue
        
        doc.close()
        print(f"✅ 총 {len(image_paths)}개 이미지 추출 완료")
        return image_paths
    
    def _process_with_smart_filtering(self, image_paths):
        """스마트 필터링으로 의미있는 이미지만 처리"""
        
        for i, img_path in enumerate(image_paths, 1):
            filename = os.path.basename(img_path)
            progress = f"{i}/{len(image_paths)} ({i/len(image_paths)*100:.1f}%)"
            
            print(f"검사 중: {progress} - {filename}")
            
            # 사전 필터링: 의미있는 이미지인지 판단
            is_meaningful = self._is_meaningful_image(img_path)
            
            if not is_meaningful:
                self.filtered_out.append({
                    'image_file': filename,
                    'reason': '신호등/아이콘/작은그래픽'
                })
                self.stats['filtered_out_count'] += 1
                print(f"  🚫 필터링됨: 의미없는 이미지")
                continue
            
            # 의미있는 이미지 처리
            result = self._analyze_meaningful_image(img_path)
            
            if result and result.get('has_useful_content'):
                self.meaningful_results.append(result)
                self.stats['meaningful_extractions'] += 1
                
                if result.get('is_actual_table'):
                    self.stats['actual_tables'] += 1
                    print(f"  ✅ 과실비율 표 발견!")
                else:
                    print(f"  ✅ 의미있는 텍스트 추출")
            else:
                print(f"  ⚠️ 의미있는 내용 없음")
    
    def _is_meaningful_image(self, image_path):
        """이미지가 의미있는지 사전 판단"""
        try:
            img = cv2.imread(image_path)
            if img is None:
                return False
            
            height, width = img.shape[:2]
            
            # 1. 크기 필터링 (너무 작은 이미지 제외)
            if width < 200 or height < 200:
                return False
            
            # 2. 종횡비 필터링 (정사각형에 가까운 작은 아이콘들 제외)
            aspect_ratio = max(width, height) / min(width, height)
            if aspect_ratio < 1.5 and max(width, height) < 400:  # 작은 정사각형
                return False
            
            # 3. 빠른 OCR로 텍스트 양 확인
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            quick_text = pytesseract.image_to_string(gray, config='--psm 6 -l kor+eng')
            
            # 텍스트가 너무 적거나 의미없는 경우
            clean_text = re.sub(r'[^\w가-힣]', '', quick_text)
            if len(clean_text) < 5:  # 의미있는 글자 5자 미만
                return False
            
            # 4. 과실비율 관련 키워드 확인
            fault_keywords = ['과실', '비율', '보행자', '차량', '신호', '교차로', '추돌', '위반', '가산', '감산']
            has_keywords = any(keyword in quick_text for keyword in fault_keywords)
            
            # 5. 숫자나 퍼센트 포함 여부
            has_numbers = bool(re.search(r'\d+', quick_text))
            has_percent = '%' in quick_text
            
            # 의미있는 이미지 판단 기준
            meaningful_score = 0
            if has_keywords: meaningful_score += 3
            if has_numbers: meaningful_score += 1
            if has_percent: meaningful_score += 2
            if len(clean_text) > 20: meaningful_score += 1
            if aspect_ratio > 2: meaningful_score += 1  # 가로로 긴 표 형태
            
            return meaningful_score >= 3
            
        except Exception as e:
            return False
    
    def _analyze_meaningful_image(self, image_path):
        """의미있는 이미지 상세 분석"""
        try:
            img = cv2.imread(image_path)
            height, width = img.shape[:2]
            
            # 고품질 OCR 수행
            extracted_content = self._high_quality_ocr(img)
            
            if not extracted_content or len(extracted_content.strip()) < 10:
                return None
            
            # 과실비율 표인지 판단
            is_table = self._is_fault_ratio_table(extracted_content)
            
            return {
                'image_file': os.path.basename(image_path),
                'page_info': self._extract_page_info(image_path),
                'image_size': f"{width}x{height}",
                'has_useful_content': True,
                'is_actual_table': is_table,
                'structured_content': self._format_meaningful_content(extracted_content, is_table),
                'raw_text': extracted_content
            }
            
        except Exception as e:
            return None
    
    def _high_quality_ocr(self, img):
        """고품질 OCR 수행"""
        try:
            # 다중 전처리 및 OCR 시도
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            # 방법 1: 크기 확대 + 샤프닝
            scale_factor = 3
            height, width = gray.shape
            resized = cv2.resize(gray, (width * scale_factor, height * scale_factor), 
                               interpolation=cv2.INTER_CUBIC)
            
            kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
            sharpened = cv2.filter2D(resized, -1, kernel)
            
            # 여러 OCR 설정으로 시도
            configs = [
                r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1',
                r'--oem 3 --psm 4 -l kor+eng',
                r'--oem 3 --psm 12 -l kor+eng'
            ]
            
            best_result = ""
            
            for config in configs:
                try:
                    result = pytesseract.image_to_string(sharpened, config=config)
                    if len(result.strip()) > len(best_result.strip()):
                        best_result = result
                except:
                    continue
            
            return best_result.strip()
            
        except Exception as e:
            return ""
    
    def _is_fault_ratio_table(self, text):
        """텍스트가 과실비율 표인지 판단"""
        if not text:
            return False
        
        # 과실비율 표의 특징
        table_indicators = [
            r'\d+%',  # 퍼센트
            r'과실.*비율',  # 과실비율
            r'가산.*요소',  # 가산요소
            r'감산.*요소',  # 감산요소
            r'보행자.*\d+',  # 보행자 + 숫자
            r'차량.*\d+',   # 차량 + 숫자
        ]
        
        score = 0
        for pattern in table_indicators:
            if re.search(pattern, text):
                score += 1
        
        # 여러 줄에 걸친 구조적 텍스트
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        if len(lines) >= 3:
            score += 1
        
        return score >= 2
    
    def _format_meaningful_content(self, text, is_table):
        """의미있는 내용을 적절히 포맷팅"""
        if not text.strip():
            return "내용을 추출할 수 없습니다."
        
        if is_table:
            # 표 형태로 포맷팅
            return self._create_proper_table(text)
        else:
            # 일반 텍스트를 정리해서 반환
            lines = [line.strip() for line in text.split('\n') if line.strip()]
            return '\n'.join(lines)
    
    def _create_proper_table(self, text):
        """실제 표 형태로 변환"""
        lines = [line.strip() for line in text.split('\n') if line.strip()]
        
        # 과실비율 데이터 추출
        table_rows = []
        
        for line in lines:
            # 숫자나 과실비율 키워드가 포함된 라인만
            if (re.search(r'\d+', line) or 
                any(keyword in line for keyword in ['과실', '비율', '가산', '감산', '보행자', '차량'])):
                
                # 적절한 구분으로 셀 나누기
                cells = self._smart_cell_split(line)
                if cells and len(cells) >= 2:
                    table_rows.append(cells)
        
        if not table_rows:
            return f"```\n{chr(10).join(lines)}\n```"
        
        # 마크다운 표 생성
        if len(table_rows) == 1:
            # 단일 행인 경우 헤더 추가
            headers = ['구분', '내용', '비율'][:len(table_rows[0])]
            markdown_lines = [
                "| " + " | ".join(headers) + " |",
                "|" + "|".join([" --- " for _ in headers]) + "|",
                "| " + " | ".join(table_rows[0]) + " |"
            ]
        else:
            # 다중 행인 경우
            max_cols = max(len(row) for row in table_rows)
            headers = ['구분', '내용', '비율'] + [f'항목{i}' for i in range(4, max_cols + 1)]
            headers = headers[:max_cols]
            
            markdown_lines = [
                "| " + " | ".join(headers) + " |",
                "|" + "|".join([" --- " for _ in headers]) + "|"
            ]
            
            for row in table_rows:
                padded_row = row + [''] * (max_cols - len(row))
                padded_row = padded_row[:max_cols]
                markdown_lines.append("| " + " | ".join(padded_row) + " |")
        
        return "\n".join(markdown_lines)
    
    def _smart_cell_split(self, line):
        """라인을 스마트하게 셀로 분할"""
        # 여러 공백을 구분자로 사용
        cells = re.split(r'\s{2,}', line)
        
        if len(cells) == 1:
            # 단일 공백으로 재시도
            cells = line.split()
        
        # 너무 짧은 셀들은 합치기
        merged_cells = []
        temp_cell = ""
        
        for cell in cells:
            cell = cell.strip()
            if not cell:
                continue
                
            if len(cell) <= 2 and temp_cell and not re.search(r'\d', cell):
                # 짧은 한글 단위는 앞 셀과 합치기
                temp_cell += " " + cell
            else:
                if temp_cell:
                    merged_cells.append(temp_cell)
                temp_cell = cell
        
        if temp_cell:
            merged_cells.append(temp_cell)
        
        return merged_cells
    
    def _extract_page_info(self, image_path):
        """페이지 정보 추출"""
        filename = os.path.basename(image_path)
        match = re.search(r'page_(\d+)_img_(\d+)', filename)
        if match:
            return f"페이지 {match.group(1)}, 이미지 {match.group(2)}"
        return filename
    
    def _create_filtered_document(self, pdf_path):
        """필터링된 의미있는 결과만으로 문서 생성"""
        base_name = os.path.splitext(os.path.basename(pdf_path))[0]
        output_file = f"{base_name}_의미있는표만추출.md"
        
        content = f"""# {base_name} - 과실비율 표 추출 결과 (필터링됨)

> 추출 일시: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}
> 전체 이미지: {self.stats['total_images']}개
> 필터링 제외: {self.stats['filtered_out_count']}개
> 의미있는 추출: {self.stats['meaningful_extractions']}개
> 실제 과실비율 표: {self.stats['actual_tables']}개

---

"""
        
        if not self.meaningful_results:
            content += "## ⚠️ 의미있는 표를 찾을 수 없습니다\n\n"
            content += "모든 이미지가 신호등 아이콘이나 작은 그래픽 요소였습니다.\n"
        else:
            # 실제 과실비율 표만 먼저 표시
            table_count = 0
            for result in self.meaningful_results:
                if result.get('is_actual_table'):
                    table_count += 1
                    content += f"""## 과실비율 표 {table_count}. {result['page_info']}

**파일**: `{result['image_file']}`  
**크기**: {result['image_size']}

{result['structured_content']}

---

"""
            
            # 기타 의미있는 텍스트
            other_count = 0
            for result in self.meaningful_results:
                if not result.get('is_actual_table'):
                    other_count += 1
                    content += f"""## 기타 텍스트 {other_count}. {result['page_info']}

**파일**: `{result['image_file']}`

```
{result['structured_content']}
```

---

"""
        
        # 필터링된 항목들 요약
        content += f"""## 📋 필터링 요약

총 {self.stats['filtered_out_count']}개 이미지가 다음 이유로 제외되었습니다:
- 신호등 아이콘
- 작은 그래픽 요소  
- 의미없는 기호나 도형
- 텍스트가 너무 적은 이미지

"""
        
        # 파일 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(content)
        
        print(f"✅ 필터링된 결과 저장: {output_file}")
        return output_file
    
    def _print_smart_statistics(self):
        """스마트 처리 통계 출력"""
        print(f"\n📊 스마트 필터링 완료 - 최종 통계")
        print(f"=" * 50)
        print(f"전체 이미지: {self.stats['total_images']}")
        print(f"🚫 필터링 제외: {self.stats['filtered_out_count']} (신호등/아이콘)")
        print(f"✅ 의미있는 추출: {self.stats['meaningful_extractions']}")
        print(f"🎯 실제 과실비율 표: {self.stats['actual_tables']}")
        
        if self.stats['total_images'] > 0:
            useful_rate = (self.stats['meaningful_extractions'] / self.stats['total_images']) * 100
            print(f"📈 유용한 이미지 비율: {useful_rate:.1f}%")

def main():
    """메인 실행 함수"""
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    processor = SmartFaultRatioPDFProcessor()
    processor.process_pdf_smart_filter(pdf_path)

if __name__ == "__main__":
    main()

🧠 스마트 과실비율 PDF 처리 시작
📄 파일: 231107_과실비율인정기준_온라인용.pdf

🔍 1단계: PDF에서 이미지 추출
✅ 총 218개 이미지 추출 완료

🎯 2단계: 의미있는 이미지만 선별 처리
검사 중: 1/218 (0.5%) - page_39_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 2/218 (0.9%) - page_43_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 3/218 (1.4%) - page_47_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 4/218 (1.8%) - page_47_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 5/218 (2.3%) - page_50_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 6/218 (2.8%) - page_50_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 7/218 (3.2%) - page_54_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 8/218 (3.7%) - page_57_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 9/218 (4.1%) - page_57_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 10/218 (4.6%) - page_61_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 11/218 (5.0%) - page_61_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 12/218 (5.5%) - page_64_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 13/218 (6.0%) - page_67_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 14/218 (6.4%) - page_70_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 15/218 (6.9%) - page_74_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 16

  🚫 필터링됨: 의미없는 이미지
검사 중: 179/218 (82.1%) - page_525_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 180/218 (82.6%) - page_525_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 181/218 (83.0%) - page_528_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 182/218 (83.5%) - page_528_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 183/218 (83.9%) - page_531_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 184/218 (84.4%) - page_531_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 185/218 (84.9%) - page_533_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 186/218 (85.3%) - page_533_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 187/218 (85.8%) - page_536_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 188/218 (86.2%) - page_536_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 189/218 (86.7%) - page_538_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 190/218 (87.2%) - page_538_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 191/218 (87.6%) - page_541_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 192/218 (88.1%) - page_541_img_2.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 193/218 (88.5%) - page_544_img_1.png
  🚫 필터링됨: 의미없는 이미지
검사 중: 194/218 (89.0%) - page_544_img_2.png
  🚫 필터링됨

In [3]:
import cv2
import numpy as np
import pytesseract
import os

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def test_layout_preservation_methods(image_path):
    """다양한 방법으로 레이아웃 보존 테스트"""
    
    print("🧪 레이아웃 보존 OCR 실험")
    print("=" * 60)
    print(f"📄 테스트 이미지: {os.path.basename(image_path)}")
    
    if not os.path.exists(image_path):
        print(f"❌ 이미지를 찾을 수 없습니다: {image_path}")
        return
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ 이미지를 읽을 수 없습니다")
        return
    
    print(f"📐 이미지 크기: {img.shape[1]}x{img.shape[0]}")
    
    # 여러 방법으로 테스트
    methods = [
        ("방법1: 기본 OCR", test_basic_layout),
        ("방법2: 위치 정보 포함", test_position_based),
        ("방법3: 블록 단위 처리", test_block_based),
        ("방법4: 레이아웃 보존 모드", test_layout_mode),
        ("방법5: 고해상도 처리", test_high_resolution)
    ]
    
    results = {}
    
    for method_name, method_func in methods:
        print(f"\n{'='*40}")
        print(f"🔍 {method_name}")
        print(f"{'='*40}")
        
        try:
            result = method_func(img.copy())
            results[method_name] = result
            
            # 결과 미리보기
            preview = result[:200].replace('\n', '\\n') if result else "추출 실패"
            print(f"📄 결과 미리보기: {preview}...")
            print(f"📊 추출된 글자 수: {len(result) if result else 0}")
            
        except Exception as e:
            print(f"❌ 오류: {e}")
            results[method_name] = f"오류: {e}"
    
    # 결과 저장
    save_comparison_results(image_path, results)

def test_basic_layout(img):
    """방법1: 기본 OCR"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    config = r'--oem 3 --psm 6 -l kor+eng'
    result = pytesseract.image_to_string(gray, config=config)
    
    print("📝 기본 OCR 결과:")
    lines = result.split('\n')
    for i, line in enumerate(lines[:5]):  # 처음 5줄만 출력
        print(f"  {i+1}: '{line}'")
    
    return result

def test_position_based(img):
    """방법2: 위치 정보를 포함한 OCR"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 위치 정보와 함께 텍스트 추출
    data = pytesseract.image_to_data(gray, config=r'--oem 3 --psm 6 -l kor+eng', output_type=pytesseract.Output.DICT)
    
    # 위치별로 텍스트 정렬
    text_blocks = []
    for i in range(len(data['text'])):
        if int(data['conf'][i]) > 30:  # 신뢰도 30 이상
            text = data['text'][i].strip()
            if text:
                x = data['left'][i]
                y = data['top'][i]
                text_blocks.append((y, x, text))
    
    # Y 좌표로 정렬 (위에서 아래로)
    text_blocks.sort(key=lambda item: (item[0], item[1]))
    
    # 같은 Y 좌표 근처의 텍스트들을 한 줄로 묶기
    lines = []
    current_line = []
    current_y = -1
    
    for y, x, text in text_blocks:
        if current_y == -1 or abs(y - current_y) < 20:  # 같은 줄로 간주
            current_line.append((x, text))
            current_y = y
        else:
            # 새로운 줄 시작
            if current_line:
                # X 좌표로 정렬해서 줄 완성
                current_line.sort()
                line_text = '  '.join([text for x, text in current_line])
                lines.append(line_text)
            current_line = [(x, text)]
            current_y = y
    
    # 마지막 줄 처리
    if current_line:
        current_line.sort()
        line_text = '  '.join([text for x, text in current_line])
        lines.append(line_text)
    
    result = '\n'.join(lines)
    
    print("📝 위치 기반 정렬 결과:")
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: '{line}'")
    
    return result

def test_block_based(img):
    """방법3: 블록 단위로 처리"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 이미지를 좌우로 분할 (다이어그램 vs 표)
    height, width = gray.shape
    
    # 오른쪽 절반 (표 부분)
    right_half = gray[:, width//2:]
    
    # 블록 모드로 OCR
    config = r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1'
    result = pytesseract.image_to_string(right_half, config=config)
    
    print("📝 블록 기반 처리 결과 (표 영역만):")
    lines = result.split('\n')
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: '{line}'")
    
    return result

def test_layout_mode(img):
    """방법4: 레이아웃 보존 모드"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 레이아웃 보존에 특화된 설정
    config = r'--oem 3 --psm 4 -l kor+eng -c preserve_interword_spaces=1 -c textord_tabfind_find_tables=1'
    result = pytesseract.image_to_string(gray, config=config)
    
    print("📝 레이아웃 보존 모드 결과:")
    lines = result.split('\n')
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: '{line}'")
    
    return result

def test_high_resolution(img):
    """방법5: 고해상도 처리"""
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 해상도 4배 증가
    scale_factor = 4
    height, width = gray.shape
    resized = cv2.resize(gray, (width * scale_factor, height * scale_factor), 
                        interpolation=cv2.INTER_CUBIC)
    
    # 샤프닝
    kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
    sharpened = cv2.filter2D(resized, -1, kernel)
    
    # 노이즈 제거
    denoised = cv2.medianBlur(sharpened, 5)
    
    config = r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1'
    result = pytesseract.image_to_string(denoised, config=config)
    
    print("📝 고해상도 처리 결과:")
    lines = result.split('\n')
    for i, line in enumerate(lines[:5]):
        print(f"  {i+1}: '{line}'")
    
    return result

def save_comparison_results(image_path, results):
    """비교 결과를 파일로 저장"""
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"OCR비교결과_{base_name}.md"
    
    content = f"""# OCR 레이아웃 보존 실험 결과

> 테스트 이미지: {os.path.basename(image_path)}
> 실험 일시: {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}

---

"""
    
    for method, result in results.items():
        content += f"""## {method}

```
{result[:500] if result else '추출 실패'}
```

**글자 수**: {len(result) if result else 0}

---

"""
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"\n💾 비교 결과 저장: {output_file}")

def test_with_sample_image():
    """샘플 이미지로 테스트"""
    
    # 추출된 이미지가 있는지 확인
    image_dir = "extracted_images"
    if os.path.exists(image_dir):
        image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
        if image_files:
            # 큰 이미지 찾기 (작은 아이콘들 제외)
            for img_file in image_files:
                img_path = os.path.join(image_dir, img_file)
                img = cv2.imread(img_path)
                if img is not None:
                    height, width = img.shape[:2]
                    if width > 400 and height > 300:  # 적당한 크기의 이미지
                        print(f"🎯 테스트 대상: {img_file} ({width}x{height})")
                        test_layout_preservation_methods(img_path)
                        return
            
            # 큰 이미지가 없으면 첫 번째 이미지로 테스트
            first_image = os.path.join(image_dir, image_files[0])
            print(f"🎯 첫 번째 이미지로 테스트: {image_files[0]}")
            test_layout_preservation_methods(first_image)
        else:
            print("❌ extracted_images 폴더에 이미지가 없습니다.")
    else:
        print("❌ extracted_images 폴더가 없습니다.")
        print("먼저 PDF에서 이미지를 추출해주세요.")

if __name__ == "__main__":
    import pandas as pd
    test_with_sample_image()

🎯 첫 번째 이미지로 테스트: page_101_img_1.png
🧪 레이아웃 보존 OCR 실험
📄 테스트 이미지: page_101_img_1.png
📐 이미지 크기: 312x312

🔍 방법1: 기본 OCR
📝 기본 OCR 결과:
  1: ''
📄 결과 미리보기: 추출 실패...
📊 추출된 글자 수: 0

🔍 방법2: 위치 정보 포함
📝 위치 기반 정렬 결과:
📄 결과 미리보기: 추출 실패...
📊 추출된 글자 수: 0

🔍 방법3: 블록 단위 처리
📝 블록 기반 처리 결과 (표 영역만):
  1: ''
📄 결과 미리보기: 추출 실패...
📊 추출된 글자 수: 0

🔍 방법4: 레이아웃 보존 모드
📝 레이아웃 보존 모드 결과:
  1: ''
📄 결과 미리보기: 추출 실패...
📊 추출된 글자 수: 0

🔍 방법5: 고해상도 처리
📝 고해상도 처리 결과:
  1: ''
📄 결과 미리보기: 추출 실패...
📊 추출된 글자 수: 0

💾 비교 결과 저장: OCR비교결과_page_101_img_1.md


In [6]:
import os
import fitz  # PyMuPDF
import cv2

def extract_images_from_page_39_onwards():
    """39페이지부터 이미지 추출"""
    
    print("🎯 39페이지부터 이미지 추출 시작")
    print("=" * 50)
    
    # PDF 경로
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # 파일 존재 확인
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    # 출력 폴더 생성
    output_dir = "extracted_images"
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📄 전체 페이지 수: {total_pages}")
        print(f"🔍 39페이지부터 확인 시작...")
        
        extracted_count = 0
        start_page = 38  # 39페이지 (0-based index)
        
        # 먼저 39-50페이지 정도만 확인해보기
        end_page = min(start_page + 12, total_pages)  # 39~50페이지
        
        for page_num in range(start_page, end_page):
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            if image_list:
                print(f"📄 페이지 {page_num+1}: {len(image_list)}개 이미지 발견!")
                
                for img_index, img in enumerate(image_list):
                    try:
                        xref = img[0]
                        pix = fitz.Pixmap(doc, xref)
                        
                        if pix.n - pix.alpha < 4:  # GRAY or RGB
                            img_path = f"{output_dir}/page_{page_num+1}_img_{img_index+1}.png"
                            pix.save(img_path)
                            
                            # 이미지 크기 확인
                            test_img = cv2.imread(img_path)
                            if test_img is not None:
                                h, w = test_img.shape[:2]
                                print(f"   ✅ {os.path.basename(img_path)} ({w}x{h})")
                                extracted_count += 1
                            
                        pix = None
                        
                    except Exception as e:
                        print(f"   ❌ 이미지 {img_index+1} 추출 실패: {e}")
            else:
                print(f"📄 페이지 {page_num+1}: 이미지 없음")
        
        doc.close()
        
        print(f"\n✅ 39-{end_page}페이지에서 {extracted_count}개 이미지 추출 완료")
        
        if extracted_count > 0:
            print(f"📁 저장 위치: {os.path.abspath(output_dir)}")
            
            # 이제 OCR 테스트 해보기
            print(f"\n🧪 추출된 이미지로 OCR 테스트...")
            test_ocr_on_extracted_images()
        else:
            print(f"❌ 이미지를 찾을 수 없습니다. 페이지 범위를 확인해주세요.")
            
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")

def extract_all_images_from_39():
    """39페이지부터 끝까지 모든 이미지 추출"""
    
    print("🚀 39페이지부터 전체 이미지 추출")
    print("=" * 50)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    output_dir = "extracted_images"
    os.makedirs(output_dir, exist_ok=True)
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📄 전체 페이지 수: {total_pages}")
        print(f"🔍 39페이지({39})부터 마지막 페이지({total_pages})까지 처리...")
        
        extracted_count = 0
        start_page = 38  # 39페이지 (0-based)
        
        for page_num in range(start_page, total_pages):
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            if image_list:
                print(f"📄 페이지 {page_num+1}: {len(image_list)}개 이미지")
                
                for img_index, img in enumerate(image_list):
                    try:
                        xref = img[0]
                        pix = fitz.Pixmap(doc, xref)
                        
                        if pix.n - pix.alpha < 4:
                            img_path = f"{output_dir}/page_{page_num+1}_img_{img_index+1}.png"
                            pix.save(img_path)
                            extracted_count += 1
                            
                            # 10개마다 진행상황 출력
                            if extracted_count % 10 == 0:
                                print(f"   진행: {extracted_count}개 추출됨...")
                        
                        pix = None
                        
                    except Exception as e:
                        continue
            
            # 50페이지마다 진행상황 출력
            if (page_num + 1) % 50 == 0:
                print(f"📊 페이지 {page_num+1}까지 처리... 총 {extracted_count}개 추출")
        
        doc.close()
        
        print(f"\n🎉 추출 완료!")
        print(f"📊 총 {extracted_count}개 이미지 추출")
        print(f"📁 저장 위치: {os.path.abspath(output_dir)}")
        
        return extracted_count
        
    except Exception as e:
        print(f"❌ 오류: {e}")
        return 0

def test_ocr_on_extracted_images():
    """추출된 이미지들로 OCR 테스트"""
    
    import pytesseract
    
    # Tesseract 경로 설정
    pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
    
    image_dir = "extracted_images"
    if not os.path.exists(image_dir):
        print(f"❌ {image_dir} 폴더가 없습니다.")
        return
    
    image_files = [f for f in os.listdir(image_dir) if f.endswith('.png')]
    
    if not image_files:
        print(f"❌ 추출된 이미지가 없습니다.")
        return
    
    print(f"\n🧪 OCR 테스트 시작 (처음 3개 이미지)")
    print("=" * 40)
    
    # 처음 3개 이미지로 테스트
    test_count = min(3, len(image_files))
    
    for i in range(test_count):
        img_file = image_files[i]
        img_path = os.path.join(image_dir, img_file)
        
        print(f"\n🔍 테스트 {i+1}: {img_file}")
        
        try:
            img = cv2.imread(img_path)
            if img is None:
                print(f"   ❌ 이미지 읽기 실패")
                continue
            
            h, w = img.shape[:2]
            print(f"   📐 크기: {w}x{h}")
            
            # 기본 OCR
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            
            config = r'--oem 3 --psm 6 -l kor+eng'
            text = pytesseract.image_to_string(gray, config=config)
            
            if text.strip():
                print(f"   ✅ OCR 성공!")
                # 처음 100자만 출력
                preview = text.strip()[:100].replace('\n', ' ')
                print(f"   📄 미리보기: {preview}...")
                
                # 과실비율 관련 키워드 확인
                keywords = ['과실', '비율', '보행자', '차량', '%']
                found_keywords = [kw for kw in keywords if kw in text]
                if found_keywords:
                    print(f"   🎯 관련 키워드: {', '.join(found_keywords)}")
            else:
                print(f"   ❌ 텍스트 추출 실패")
                
        except Exception as e:
            print(f"   ❌ OCR 오류: {e}")

def main():
    """메인 실행 함수"""
    print("과실비율 PDF 이미지 추출 (39페이지부터)")
    print("선택하세요:")
    print("1. 39-50페이지만 테스트 추출")
    print("2. 39페이지부터 전체 추출")
    
    choice = input("\n선택 (1 또는 2): ").strip()
    
    if choice == "1":
        extract_images_from_page_39_onwards()
    elif choice == "2":
        confirm = input("전체 추출하면 시간이 오래 걸릴 수 있습니다. 계속하시겠습니까? (y/n): ")
        if confirm.lower() == 'y':
            extract_all_images_from_39()
        else:
            print("취소되었습니다.")
    else:
        print("잘못된 선택입니다.")

if __name__ == "__main__":
    main()

과실비율 PDF 이미지 추출 (39페이지부터)
선택하세요:
1. 39-50페이지만 테스트 추출
2. 39페이지부터 전체 추출

선택 (1 또는 2): 2
전체 추출하면 시간이 오래 걸릴 수 있습니다. 계속하시겠습니까? (y/n): y
🚀 39페이지부터 전체 이미지 추출
📄 전체 페이지 수: 600
🔍 39페이지(39)부터 마지막 페이지(600)까지 처리...
📄 페이지 39: 1개 이미지
📄 페이지 43: 1개 이미지
📄 페이지 47: 2개 이미지
📄 페이지 50: 2개 이미지
📊 페이지 50까지 처리... 총 6개 추출
📄 페이지 54: 1개 이미지
📄 페이지 57: 2개 이미지
📄 페이지 61: 2개 이미지
   진행: 10개 추출됨...
📄 페이지 64: 1개 이미지
📄 페이지 67: 1개 이미지
📄 페이지 70: 1개 이미지
📄 페이지 74: 2개 이미지
📄 페이지 78: 2개 이미지
📄 페이지 82: 1개 이미지
📄 페이지 85: 2개 이미지
   진행: 20개 추출됨...
📄 페이지 90: 1개 이미지
📄 페이지 94: 1개 이미지
📄 페이지 97: 2개 이미지
📄 페이지 98: 1개 이미지
📊 페이지 100까지 처리... 총 26개 추출
📄 페이지 101: 1개 이미지
📄 페이지 104: 2개 이미지
📄 페이지 107: 1개 이미지
   진행: 30개 추출됨...
📄 페이지 109: 1개 이미지
📄 페이지 111: 1개 이미지
📄 페이지 114: 2개 이미지
📄 페이지 117: 1개 이미지
📄 페이지 119: 1개 이미지
📄 페이지 121: 2개 이미지
📄 페이지 129: 1개 이미지
📄 페이지 148: 1개 이미지
   진행: 40개 추출됨...
📊 페이지 150까지 처리... 총 40개 추출
📄 페이지 152: 1개 이미지
📄 페이지 155: 1개 이미지
📄 페이지 158: 1개 이미지
📄 페이지 160: 1개 이미지
📄 페이지 164: 1개 이미지
📄 페이지 167: 1개 이미지
📄 페이지 170: 1개 이미지
📄 페이지 172: 1개 이미지

In [7]:
import fitz  # PyMuPDF
import pytesseract
import cv2
import numpy as np
import os
import re

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def extract_table_text_from_pdf_images():
    """PDF 39페이지부터 이미지 안의 표 텍스트 추출"""
    
    print("📊 PDF 이미지 속 표 텍스트 추출 (39페이지부터)")
    print("=" * 60)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📄 전체 페이지 수: {total_pages}")
        print(f"🎯 39페이지부터 처리 시작...")
        
        # 결과 저장용
        all_extracted_tables = []
        
        # 처음에는 39-45페이지만 테스트
        start_page = 38  # 39페이지 (0-based)
        end_page = min(start_page + 7, total_pages)  # 39-45페이지
        
        for page_num in range(start_page, end_page):
            print(f"\n📄 페이지 {page_num+1} 처리 중...")
            
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            if not image_list:
                print(f"   ℹ️ 이미지 없음")
                continue
            
            print(f"   🖼️ {len(image_list)}개 이미지 발견")
            
            for img_index, img in enumerate(image_list):
                try:
                    # 이미지를 메모리에서 직접 처리
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    
                    if pix.n - pix.alpha < 4:  # GRAY or RGB
                        # PIL Image로 변환
                        img_data = pix.tobytes("png")
                        
                        # numpy 배열로 변환
                        nparr = np.frombuffer(img_data, np.uint8)
                        cv_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        
                        if cv_image is not None:
                            h, w = cv_image.shape[:2]
                            print(f"      이미지 {img_index+1}: {w}x{h}")
                            
                            # 표 텍스트 추출
                            table_text = extract_table_text_from_image(cv_image, page_num+1, img_index+1)
                            
                            if table_text and table_text.strip():
                                all_extracted_tables.append({
                                    'page': page_num+1,
                                    'image': img_index+1,
                                    'content': table_text
                                })
                                print(f"         ✅ 텍스트 추출 성공!")
                            else:
                                print(f"         ❌ 텍스트 추출 실패")
                    
                    pix = None
                    
                except Exception as e:
                    print(f"      ❌ 이미지 {img_index+1} 처리 오류: {e}")
        
        doc.close()
        
        # 결과 저장
        if all_extracted_tables:
            save_extracted_table_text(all_extracted_tables)
            print(f"\n🎉 총 {len(all_extracted_tables)}개 표에서 텍스트 추출 완료!")
        else:
            print(f"\n❌ 추출된 텍스트가 없습니다.")
            
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")

def extract_table_text_from_image(cv_image, page_num, img_num):
    """이미지에서 표 텍스트 추출 (레이아웃 보존)"""
    
    try:
        # 이미지 전처리
        gray = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY)
        
        # 크기가 너무 작으면 확대
        h, w = gray.shape
        if w < 400 or h < 300:
            scale_factor = 2
            gray = cv2.resize(gray, (w * scale_factor, h * scale_factor), 
                            interpolation=cv2.INTER_CUBIC)
        
        # 여러 OCR 방법 시도
        methods = [
            # 방법 1: 기본 레이아웃 보존
            {
                'name': '기본 레이아웃',
                'config': r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1'
            },
            # 방법 2: 테이블 모드
            {
                'name': '테이블 모드', 
                'config': r'--oem 3 --psm 4 -l kor+eng -c preserve_interword_spaces=1'
            },
            # 방법 3: 위치 정보 포함
            {
                'name': '위치 기반',
                'config': r'--oem 3 --psm 6 -l kor+eng',
                'use_position': True
            }
        ]
        
        best_result = ""
        best_method = ""
        
        for method in methods:
            try:
                if method.get('use_position'):
                    # 위치 기반 추출
                    result = extract_with_position_info(gray, method['config'])
                else:
                    # 일반 OCR
                    result = pytesseract.image_to_string(gray, config=method['config'])
                
                # 결과 품질 평가
                if result and len(result.strip()) > len(best_result.strip()):
                    # 과실비율 관련 키워드가 있으면 더 높은 점수
                    keywords = ['과실', '비율', '보행자', '차량', '%', '가산', '감산']
                    if any(keyword in result for keyword in keywords):
                        best_result = result
                        best_method = method['name']
                    elif not best_result:  # 키워드가 없어도 텍스트가 있으면 저장
                        best_result = result
                        best_method = method['name']
                        
            except Exception as e:
                continue
        
        if best_result.strip():
            print(f"         📝 {best_method}으로 추출")
            # 결과를 표 형태로 정리
            return format_as_structured_text(best_result, page_num, img_num)
        else:
            return None
            
    except Exception as e:
        return None

def extract_with_position_info(gray, config):
    """위치 정보를 활용한 텍스트 추출"""
    
    try:
        # 위치 데이터 추출
        data = pytesseract.image_to_data(gray, config=config, output_type=pytesseract.Output.DICT)
        
        # 신뢰도가 높은 텍스트만 선별
        text_blocks = []
        for i in range(len(data['text'])):
            if int(data['conf'][i]) > 30:  # 신뢰도 30 이상
                text = data['text'][i].strip()
                if text:
                    x = data['left'][i]
                    y = data['top'][i]
                    w = data['width'][i]
                    h = data['height'][i]
                    text_blocks.append((y, x, text, w, h))
        
        if not text_blocks:
            return ""
        
        # Y 좌표로 정렬 (위에서 아래로)
        text_blocks.sort()
        
        # 같은 줄의 텍스트들을 그룹핑
        lines = []
        current_line = []
        current_y = -1
        
        for y, x, text, w, h in text_blocks:
            if current_y == -1 or abs(y - current_y) < 15:  # 같은 줄
                current_line.append((x, text))
                current_y = y
            else:
                # 새로운 줄
                if current_line:
                    current_line.sort()  # X 좌표로 정렬
                    line_text = "  ".join([text for x, text in current_line])
                    lines.append(line_text)
                current_line = [(x, text)]
                current_y = y
        
        # 마지막 줄 처리
        if current_line:
            current_line.sort()
            line_text = "  ".join([text for x, text in current_line])
            lines.append(line_text)
        
        return "\n".join(lines)
        
    except Exception as e:
        return ""

def format_as_structured_text(raw_text, page_num, img_num):
    """추출된 텍스트를 구조화된 형태로 포맷팅"""
    
    if not raw_text.strip():
        return ""
    
    lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
    
    # 과실비율 표 형태로 정리
    result = f"## 페이지 {page_num}, 이미지 {img_num}\n\n"
    
    # 과실비율 관련 라인들 찾기
    table_lines = []
    other_lines = []
    
    for line in lines:
        if (any(keyword in line for keyword in ['과실', '비율', '가산', '감산', '보행자', '차량']) or
            re.search(r'\d+%?', line)):
            table_lines.append(line)
        else:
            other_lines.append(line)
    
    if table_lines:
        result += "**과실비율 표:**\n\n"
        
        # 표 형태로 변환 시도
        markdown_table = convert_to_simple_table(table_lines)
        if markdown_table:
            result += markdown_table + "\n\n"
        else:
            # 표 변환 실패시 리스트 형태로
            for line in table_lines:
                result += f"- {line}\n"
            result += "\n"
    
    if other_lines:
        result += "**기타 텍스트:**\n\n"
        for line in other_lines:
            result += f"- {line}\n"
        result += "\n"
    
    result += "**원본 텍스트:**\n```\n" + raw_text + "\n```\n\n"
    result += "---\n\n"
    
    return result

def convert_to_simple_table(lines):
    """라인들을 간단한 마크다운 표로 변환"""
    
    try:
        table_data = []
        
        for line in lines:
            # 공백 기준으로 분할
            parts = re.split(r'\s{2,}', line)  # 2개 이상 공백으로 분할
            
            if len(parts) == 1:
                parts = line.split()  # 단일 공백으로 재시도
            
            clean_parts = [part.strip() for part in parts if part.strip()]
            
            if len(clean_parts) >= 2:
                table_data.append(clean_parts)
        
        if not table_data:
            return None
        
        # 마크다운 표 생성
        max_cols = max(len(row) for row in table_data)
        headers = ['구분', '내용'] + [f'열{i}' for i in range(3, max_cols + 1)]
        headers = headers[:max_cols]
        
        markdown_lines = []
        markdown_lines.append("| " + " | ".join(headers) + " |")
        markdown_lines.append("|" + "|".join([" --- " for _ in headers]) + "|")
        
        for row in table_data:
            padded_row = row + [''] * (max_cols - len(row))
            padded_row = padded_row[:max_cols]
            markdown_lines.append("| " + " | ".join(padded_row) + " |")
        
        return "\n".join(markdown_lines)
        
    except Exception as e:
        return None

def save_extracted_table_text(extracted_tables):
    """추출된 표 텍스트를 파일로 저장"""
    
    output_file = "과실비율표_텍스트추출결과.md"
    
    content = f"""# 과실비율 PDF 표 텍스트 추출 결과

> 추출 일시: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
> 추출된 표 수: {len(extracted_tables)}개

---

"""
    
    for i, table in enumerate(extracted_tables, 1):
        content += f"# 표 {i}\n\n"
        content += table['content']
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 결과 파일 저장: {output_file}")

if __name__ == "__main__":
    extract_table_text_from_pdf_images()

📊 PDF 이미지 속 표 텍스트 추출 (39페이지부터)
📄 전체 페이지 수: 600
🎯 39페이지부터 처리 시작...

📄 페이지 39 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x310
         📝 기본 레이아웃으로 추출
         ✅ 텍스트 추출 성공!

📄 페이지 40 처리 중...
   ℹ️ 이미지 없음

📄 페이지 41 처리 중...
   ℹ️ 이미지 없음

📄 페이지 42 처리 중...
   ℹ️ 이미지 없음

📄 페이지 43 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
         📝 기본 레이아웃으로 추출
         ✅ 텍스트 추출 성공!

📄 페이지 44 처리 중...
   ℹ️ 이미지 없음

📄 페이지 45 처리 중...
   ℹ️ 이미지 없음
📄 결과 파일 저장: 과실비율표_텍스트추출결과.md

🎉 총 2개 표에서 텍스트 추출 완료!


In [9]:
import fitz  # PyMuPDF
import os
from datetime import datetime
import re

def extract_text_preserve_layout():
    """PDF 텍스트 추출 (레이아웃 보존, 이미지 영역 제외)"""
    
    print("📄 PDF 텍스트 추출 (레이아웃 보존)")
    print("=" * 60)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📊 전체 페이지 수: {total_pages}")
        print(f"🚀 텍스트 추출 시작 (이미지 영역 제외)...")
        
        all_pages_content = []
        stats = {
            'total_pages': total_pages,
            'pages_with_text': 0,
            'total_characters': 0,
            'images_found': 0,
            'images_excluded': 0
        }
        
        for page_num in range(total_pages):
            if page_num % 10 == 0:
                print(f"진행률: {page_num+1}/{total_pages} ({(page_num+1)/total_pages*100:.1f}%)")
            
            page = doc.load_page(page_num)
            
            # 이미지 영역 정보 수집
            image_areas = get_image_areas(page)
            stats['images_found'] += len(image_areas)
            
            # 텍스트 추출 (이미지 영역 제외)
            page_content = extract_text_exclude_images(page, image_areas, page_num + 1)
            
            if page_content and page_content.strip():
                all_pages_content.append(page_content)
                stats['pages_with_text'] += 1
                stats['total_characters'] += len(page_content)
            
            if image_areas:
                stats['images_excluded'] += len(image_areas)
        
        doc.close()
        
        # 결과 저장
        save_extracted_content(all_pages_content, stats)
        
        print(f"\n🎉 텍스트 추출 완료!")
        print(f"📊 처리 통계:")
        print(f"   - 전체 페이지: {stats['total_pages']}")
        print(f"   - 텍스트 있는 페이지: {stats['pages_with_text']}")
        print(f"   - 발견된 이미지: {stats['images_found']}")
        print(f"   - 제외된 이미지: {stats['images_excluded']}")
        print(f"   - 총 글자 수: {stats['total_characters']:,}")
        
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")

def get_image_areas(page):
    """페이지의 이미지 영역 좌표 수집"""
    
    image_areas = []
    
    try:
        image_list = page.get_images()
        
        for img_index, img in enumerate(image_list):
            try:
                # 이미지의 위치 정보 추출
                xref = img[0]
                
                # 이미지가 실제로 배치된 위치 찾기
                for item in page.get_drawings():
                    if hasattr(item, 'rect'):
                        # 이미지 영역으로 추정되는 사각형
                        rect = item['rect']
                        if rect.width > 50 and rect.height > 50:  # 충분히 큰 영역만
                            image_areas.append(rect)
                
                # 이미지 블록 직접 찾기
                blocks = page.get_text("dict")
                if blocks and "blocks" in blocks:
                    for block in blocks["blocks"]:
                        if block.get("type") == 1:  # 이미지 블록
                            bbox = block.get("bbox")
                            if bbox:
                                rect = fitz.Rect(bbox)
                                image_areas.append(rect)
                
            except Exception as e:
                continue
    
    except Exception as e:
        pass
    
    return image_areas

def extract_text_exclude_images(page, image_areas, page_num):
    """이미지 영역을 제외하고 텍스트 추출"""
    
    try:
        # 여러 방법으로 텍스트 추출 시도
        methods = [
            extract_with_blocks_layout,
            extract_with_basic_layout,
            extract_with_sorted_text
        ]
        
        best_content = ""
        best_score = 0
        
        for method in methods:
            try:
                content = method(page, image_areas, page_num)
                score = evaluate_text_layout_quality(content)
                
                if score > best_score:
                    best_score = score
                    best_content = content
                    
            except Exception as e:
                continue
        
        return best_content
        
    except Exception as e:
        return ""

def extract_with_blocks_layout(page, image_areas, page_num):
    """블록 기반 레이아웃 보존 추출"""
    
    try:
        blocks = page.get_text("dict")
        if not blocks or "blocks" not in blocks:
            return ""
        
        text_blocks = []
        
        for block in blocks["blocks"]:
            if block.get("type") == 0:  # 텍스트 블록만
                block_bbox = fitz.Rect(block["bbox"])
                
                # 이미지 영역과 겹치는지 확인
                overlaps_image = False
                for img_area in image_areas:
                    if block_bbox.intersects(img_area):
                        overlap_area = block_bbox.intersect(img_area).get_area()
                        block_area = block_bbox.get_area()
                        
                        if overlap_area > block_area * 0.3:  # 30% 이상 겹치면 제외
                            overlaps_image = True
                            break
                
                if not overlaps_image and "lines" in block:
                    # 블록 내 텍스트 추출
                    block_lines = []
                    
                    for line in block["lines"]:
                        line_text = ""
                        line_spans = []
                        
                        for span in line["spans"]:
                            text = span.get("text", "").strip()
                            if text:
                                line_spans.append({
                                    'text': text,
                                    'x': span.get("bbox", [0, 0, 0, 0])[0],
                                    'font_size': span.get("size", 12)
                                })
                        
                        if line_spans:
                            # 같은 줄 내에서 X 좌표로 정렬
                            line_spans.sort(key=lambda s: s['x'])
                            
                            # 적절한 간격으로 텍스트 연결
                            line_parts = []
                            prev_x = 0
                            
                            for span in line_spans:
                                x_gap = span['x'] - prev_x
                                
                                if prev_x > 0 and x_gap > span['font_size'] * 2:
                                    # 큰 간격이면 탭이나 여러 공백 추가
                                    line_parts.append("  ")
                                
                                line_parts.append(span['text'])
                                prev_x = span['x'] + len(span['text']) * span['font_size'] * 0.6
                            
                            line_text = "".join(line_parts)
                        
                        if line_text.strip():
                            block_lines.append(line_text.strip())
                    
                    if block_lines:
                        y_pos = block["bbox"][1]  # Y 좌표
                        block_text = "\n".join(block_lines)
                        text_blocks.append((y_pos, block_text))
        
        # Y 좌표로 정렬 (위에서 아래로)
        text_blocks.sort(key=lambda x: x[0])
        
        # 페이지 헤더 추가
        page_content = f"\n\n--- 페이지 {page_num} ---\n\n"
        
        # 블록들 연결
        for y_pos, block_text in text_blocks:
            page_content += block_text + "\n\n"
        
        return page_content
        
    except Exception as e:
        return ""

def extract_with_basic_layout(page, image_areas, page_num):
    """기본 레이아웃 보존 추출"""
    
    try:
        # 정렬된 텍스트 추출
        text = page.get_text("text", sort=True)
        
        if not text or not text.strip():
            return ""
        
        # 페이지 헤더 추가
        formatted_text = f"\n\n--- 페이지 {page_num} ---\n\n"
        formatted_text += text
        
        return formatted_text
        
    except Exception as e:
        return ""

def extract_with_sorted_text(page, image_areas, page_num):
    """정렬된 텍스트 추출"""
    
    try:
        # 기본 텍스트 추출
        text = page.get_text()
        
        if not text or not text.strip():
            return ""
        
        # 줄 단위로 정리
        lines = text.split('\n')
        clean_lines = []
        
        for line in lines:
            line = line.strip()
            if line:
                clean_lines.append(line)
        
        if clean_lines:
            formatted_text = f"\n\n--- 페이지 {page_num} ---\n\n"
            formatted_text += "\n".join(clean_lines)
            return formatted_text
        
        return ""
        
    except Exception as e:
        return ""

def evaluate_text_layout_quality(content):
    """텍스트 레이아웃 품질 평가"""
    
    if not content or not content.strip():
        return 0
    
    score = 0
    
    # 텍스트 길이
    length = len(content.strip())
    score += min(length * 0.01, 50)
    
    # 줄바꿈 구조
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    score += len(lines) * 0.5
    
    # 한글 포함 여부
    korean_chars = sum(1 for c in content if '가' <= c <= '힣')
    score += korean_chars * 0.02
    
    # 과실비율 관련 키워드
    keywords = ['과실', '비율', '조', '항', '호', '기준', '적용']
    for keyword in keywords:
        if keyword in content:
            score += 5
    
    # 구조적 요소 (번호, 제목 등)
    if re.search(r'제\d+조', content):
        score += 10
    if re.search(r'\d+\.\s', content):
        score += 5
    
    return score

def save_extracted_content(all_pages_content, stats):
    """추출된 내용을 파일로 저장"""
    
    # 메인 파일 (전체 내용)
    main_output = "과실비율_전체텍스트.md"
    
    content = f"""# 과실비율 인정기준 전체 텍스트

> 추출 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
> 전체 페이지: {stats['total_pages']}
> 텍스트 페이지: {stats['pages_with_text']}
> 총 글자 수: {stats['total_characters']:,}
> 제외된 이미지: {stats['images_excluded']}개

---

"""
    
    # 모든 페이지 내용 추가
    for page_content in all_pages_content:
        content += page_content
    
    # 파일 저장
    with open(main_output, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 전체 텍스트 저장: {main_output}")
    
    # 구조화된 버전도 저장
    save_structured_version(all_pages_content, stats)

def save_structured_version(all_pages_content, stats):
    """구조화된 버전으로도 저장"""
    
    structured_output = "과실비율_구조화텍스트.md"
    
    content = f"""# 과실비율 인정기준 (구조화)

> 추출 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📊 문서 통계

- **전체 페이지**: {stats['total_pages']}
- **텍스트 페이지**: {stats['pages_with_text']}
- **총 글자 수**: {stats['total_characters']:,}
- **제외된 이미지**: {stats['images_excluded']}개

---

## 📄 본문 내용

"""
    
    # 과실비율 관련 섹션들을 구분해서 저장
    current_section = ""
    
    for page_content in all_pages_content:
        # 조문이나 큰 제목 감지
        if re.search(r'제\d+조|제\d+장|제\d+편', page_content):
            sections = re.findall(r'(제\d+조[^제]*|제\d+장[^제]*|제\d+편[^제]*)', page_content)
            for section in sections:
                content += f"\n## {section[:50]}...\n\n"
                content += section + "\n\n"
        else:
            content += page_content
    
    with open(structured_output, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 구조화 텍스트 저장: {structured_output}")

if __name__ == "__main__":
    extract_text_preserve_layout()

📄 PDF 텍스트 추출 (레이아웃 보존)
📊 전체 페이지 수: 600
🚀 텍스트 추출 시작 (이미지 영역 제외)...
진행률: 1/600 (0.2%)
진행률: 11/600 (1.8%)
진행률: 21/600 (3.5%)
진행률: 31/600 (5.2%)
진행률: 41/600 (6.8%)
진행률: 51/600 (8.5%)
진행률: 61/600 (10.2%)
진행률: 71/600 (11.8%)
진행률: 81/600 (13.5%)
진행률: 91/600 (15.2%)
진행률: 101/600 (16.8%)
진행률: 111/600 (18.5%)
진행률: 121/600 (20.2%)
진행률: 131/600 (21.8%)
진행률: 141/600 (23.5%)
진행률: 151/600 (25.2%)
진행률: 161/600 (26.8%)
진행률: 171/600 (28.5%)
진행률: 181/600 (30.2%)
진행률: 191/600 (31.8%)
진행률: 201/600 (33.5%)
진행률: 211/600 (35.2%)
진행률: 221/600 (36.8%)
진행률: 231/600 (38.5%)
진행률: 241/600 (40.2%)
진행률: 251/600 (41.8%)
진행률: 261/600 (43.5%)
진행률: 271/600 (45.2%)
진행률: 281/600 (46.8%)
진행률: 291/600 (48.5%)
진행률: 301/600 (50.2%)
진행률: 311/600 (51.8%)
진행률: 321/600 (53.5%)
진행률: 331/600 (55.2%)
진행률: 341/600 (56.8%)
진행률: 351/600 (58.5%)
진행률: 361/600 (60.2%)
진행률: 371/600 (61.8%)
진행률: 381/600 (63.5%)
진행률: 391/600 (65.2%)
진행률: 401/600 (66.8%)
진행률: 411/600 (68.5%)
진행률: 421/600 (70.2%)
진행률: 431/600 (71.8%)
진행률: 441/600 (73.5%)
진행률: 4

In [11]:
import fitz  # PyMuPDF
import os
from datetime import datetime
import re

def extract_text_preserve_layout():
    """PDF 텍스트 추출 (레이아웃 보존, 이미지 영역 제외)"""
    
    print("📄 PDF 텍스트 추출 (레이아웃 보존)")
    print("=" * 60)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📊 전체 페이지 수: {total_pages}")
        print(f"🚀 텍스트 추출 시작 (이미지 영역 제외)...")
        
        all_pages_content = []
        stats = {
            'total_pages': total_pages,
            'pages_with_text': 0,
            'total_characters': 0,
            'images_found': 0,
            'images_excluded': 0
        }
        
        for page_num in range(total_pages):
            if page_num % 10 == 0:
                print(f"진행률: {page_num+1}/{total_pages} ({(page_num+1)/total_pages*100:.1f}%)")
            
            page = doc.load_page(page_num)
            
            # 이미지 영역 정보 수집
            image_areas = get_image_areas(page)
            stats['images_found'] += len(image_areas)
            
            # 텍스트 추출 (이미지 영역 제외)
            page_content = extract_text_exclude_images(page, image_areas, page_num + 1)
            
            if page_content and page_content.strip():
                all_pages_content.append(page_content)
                stats['pages_with_text'] += 1
                stats['total_characters'] += len(page_content)
            
            if image_areas:
                stats['images_excluded'] += len(image_areas)
        
        doc.close()
        
        # 결과 저장
        save_extracted_content(all_pages_content, stats)
        
        print(f"\n🎉 텍스트 추출 완료!")
        print(f"📊 처리 통계:")
        print(f"   - 전체 페이지: {stats['total_pages']}")
        print(f"   - 텍스트 있는 페이지: {stats['pages_with_text']}")
        print(f"   - 발견된 이미지: {stats['images_found']}")
        print(f"   - 제외된 이미지: {stats['images_excluded']}")
        print(f"   - 총 글자 수: {stats['total_characters']:,}")
        
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")

def get_image_areas(page):
    """페이지의 이미지 영역 좌표 수집"""
    
    image_areas = []
    
    try:
        image_list = page.get_images()
        
        for img_index, img in enumerate(image_list):
            try:
                # 이미지의 위치 정보 추출
                xref = img[0]
                
                # 이미지가 실제로 배치된 위치 찾기
                for item in page.get_drawings():
                    if hasattr(item, 'rect'):
                        # 이미지 영역으로 추정되는 사각형
                        rect = item['rect']
                        if rect.width > 50 and rect.height > 50:  # 충분히 큰 영역만
                            image_areas.append(rect)
                
                # 이미지 블록 직접 찾기
                blocks = page.get_text("dict")
                if blocks and "blocks" in blocks:
                    for block in blocks["blocks"]:
                        if block.get("type") == 1:  # 이미지 블록
                            bbox = block.get("bbox")
                            if bbox:
                                rect = fitz.Rect(bbox)
                                image_areas.append(rect)
                
            except Exception as e:
                continue
    
    except Exception as e:
        pass
    
    return image_areas

def extract_text_exclude_images(page, image_areas, page_num):
    """이미지 영역을 제외하고 텍스트 추출"""
    
    try:
        # 여러 방법으로 텍스트 추출 시도
        methods = [
            extract_with_blocks_layout,
            extract_with_basic_layout,
            extract_with_sorted_text
        ]
        
        best_content = ""
        best_score = 0
        
        for method in methods:
            try:
                content = method(page, image_areas, page_num)
                score = evaluate_text_layout_quality(content)
                
                if score > best_score:
                    best_score = score
                    best_content = content
                    
            except Exception as e:
                continue
        
        return best_content
        
    except Exception as e:
        return ""

def extract_with_blocks_layout(page, image_areas, page_num):
    """블록 기반 레이아웃 보존 추출 (표 정렬 개선)"""
    
    try:
        blocks = page.get_text("dict")
        if not blocks or "blocks" not in blocks:
            return ""
        
        # 모든 텍스트 요소를 위치별로 수집
        all_text_elements = []
        
        for block in blocks["blocks"]:
            if block.get("type") == 0:  # 텍스트 블록만
                block_bbox = fitz.Rect(block["bbox"])
                
                # 이미지 영역과 겹치는지 확인
                overlaps_image = False
                for img_area in image_areas:
                    if block_bbox.intersects(img_area):
                        overlap_area = block_bbox.intersect(img_area).get_area()
                        block_area = block_bbox.get_area()
                        
                        if overlap_area > block_area * 0.3:  # 30% 이상 겹치면 제외
                            overlaps_image = True
                            break
                
                if not overlaps_image and "lines" in block:
                    for line in block["lines"]:
                        for span in line["spans"]:
                            text = span.get("text", "").strip()
                            if text:
                                bbox = span.get("bbox", [0, 0, 0, 0])
                                all_text_elements.append({
                                    'text': text,
                                    'x': bbox[0],
                                    'y': bbox[1],
                                    'width': bbox[2] - bbox[0],
                                    'height': bbox[3] - bbox[1],
                                    'font_size': span.get("size", 12)
                                })
        
        if not all_text_elements:
            return ""
        
        # Y 좌표로 그룹핑 (같은 줄 찾기)
        lines = group_elements_by_line(all_text_elements)
        
        # 각 줄을 표 형태로 정렬
        formatted_lines = []
        
        for line_elements in lines:
            formatted_line = format_line_as_table(line_elements)
            if formatted_line:
                formatted_lines.append(formatted_line)
        
        # 페이지 헤더 추가
        page_content = f"\n\n--- 페이지 {page_num} ---\n\n"
        
        # 표 형태 감지 및 포맷팅
        table_sections = detect_and_format_tables(formatted_lines)
        
        for section in table_sections:
            page_content += section + "\n\n"
        
        return page_content
        
    except Exception as e:
        return ""

def group_elements_by_line(elements, line_tolerance=5):
    """텍스트 요소들을 같은 줄별로 그룹핑"""
    
    if not elements:
        return []
    
    # Y 좌표로 정렬
    elements.sort(key=lambda e: e['y'])
    
    lines = []
    current_line = [elements[0]]
    current_y = elements[0]['y']
    
    for element in elements[1:]:
        if abs(element['y'] - current_y) <= line_tolerance:
            # 같은 줄로 간주
            current_line.append(element)
        else:
            # 새로운 줄 시작
            if current_line:
                # 현재 줄을 X 좌표로 정렬
                current_line.sort(key=lambda e: e['x'])
                lines.append(current_line)
            
            current_line = [element]
            current_y = element['y']
    
    # 마지막 줄 추가
    if current_line:
        current_line.sort(key=lambda e: e['x'])
        lines.append(current_line)
    
    return lines

def format_line_as_table(line_elements):
    """한 줄의 요소들을 표 형태로 정렬"""
    
    if not line_elements:
        return ""
    
    # X 좌표 기준으로 컬럼 구분
    columns = detect_columns(line_elements)
    
    if len(columns) <= 1:
        # 단일 컬럼인 경우
        return " ".join([elem['text'] for elem in line_elements])
    
    # 다중 컬럼인 경우 정렬
    formatted_parts = []
    
    for i, column in enumerate(columns):
        column_text = " ".join([elem['text'] for elem in column])
        
        if i == 0:
            # 첫 번째 컬럼은 그대로
            formatted_parts.append(column_text)
        else:
            # 나머지 컬럼들은 적절한 간격으로 정렬
            prev_column_end = max([elem['x'] + elem['width'] for elem in columns[i-1]])
            current_column_start = min([elem['x'] for elem in column])
            gap = current_column_start - prev_column_end
            
            # 간격에 따라 탭 또는 공백 추가
            if gap > 20:  # 큰 간격
                separator = "\t"
            elif gap > 10:  # 중간 간격
                separator = "    "  # 4개 공백
            else:  # 작은 간격
                separator = "  "   # 2개 공백
            
            formatted_parts.append(separator + column_text)
    
    return "".join(formatted_parts)

def detect_columns(line_elements):
    """한 줄 내의 요소들을 컬럼별로 구분"""
    
    if len(line_elements) <= 1:
        return [line_elements]
    
    # X 좌표 간격을 분석해서 컬럼 구분
    columns = []
    current_column = [line_elements[0]]
    
    for i in range(1, len(line_elements)):
        prev_elem = line_elements[i-1]
        curr_elem = line_elements[i]
        
        # 이전 요소와의 간격 계산
        gap = curr_elem['x'] - (prev_elem['x'] + prev_elem['width'])
        
        # 폰트 크기 기준으로 간격 판단
        avg_font_size = (prev_elem['font_size'] + curr_elem['font_size']) / 2
        
        if gap > avg_font_size * 1.5:  # 폰트 크기의 1.5배 이상 간격이면 새 컬럼
            columns.append(current_column)
            current_column = [curr_elem]
        else:
            current_column.append(curr_elem)
    
    columns.append(current_column)
    return columns

def detect_and_format_tables(formatted_lines):
    """표 섹션을 감지하고 마크다운 표로 변환"""
    
    sections = []
    current_section = []
    
    for line in formatted_lines:
        # 표 형태인지 판단 (탭이나 여러 공백 포함)
        if '\t' in line or '    ' in line:
            current_section.append(line)
        else:
            # 일반 텍스트
            if current_section:
                # 이전 표 섹션 마무리
                table_md = convert_section_to_markdown_table(current_section)
                if table_md:
                    sections.append(table_md)
                else:
                    sections.append('\n'.join(current_section))
                current_section = []
            
            sections.append(line)
    
    # 마지막 섹션 처리
    if current_section:
        table_md = convert_section_to_markdown_table(current_section)
        if table_md:
            sections.append(table_md)
        else:
            sections.append('\n'.join(current_section))
    
    return sections

def convert_section_to_markdown_table(section_lines):
    """표 섹션을 마크다운 표로 변환"""
    
    if not section_lines:
        return ""
    
    # 과실비율 관련 키워드가 있는지 확인
    text_content = '\n'.join(section_lines)
    is_fault_table = any(keyword in text_content for keyword in 
                        ['과실', '비율', '가산', '감산', '보행자', '차량', '%'])
    
    if not is_fault_table and len(section_lines) < 3:
        # 과실비율과 관련 없고 줄 수가 적으면 일반 텍스트로
        return '\n'.join(section_lines)
    
    # 표 데이터 추출
    table_data = []
    
    for line in section_lines:
        # 탭이나 여러 공백으로 분할
        parts = re.split(r'\t+|    +', line)
        clean_parts = [part.strip() for part in parts if part.strip()]
        
        if len(clean_parts) >= 2:
            table_data.append(clean_parts)
        elif len(clean_parts) == 1 and clean_parts[0]:
            # 단일 항목도 포함
            table_data.append([clean_parts[0], ""])
    
    if not table_data:
        return '\n'.join(section_lines)
    
    # 마크다운 표 생성
    max_cols = max(len(row) for row in table_data)
    
    # 컬럼 수에 따른 헤더 설정
    if max_cols == 2:
        headers = ["구분", "내용"]
    elif max_cols == 3:
        headers = ["구분", "내용", "비율"]
    elif max_cols == 4:
        headers = ["구분", "내용", "비율", "비고"]
    else:
        headers = ["구분"] + [f"항목{i}" for i in range(2, max_cols + 1)]
    
    # 마크다운 표 생성
    markdown_lines = []
    markdown_lines.append("| " + " | ".join(headers) + " |")
    markdown_lines.append("|" + "|".join([" --- " for _ in headers]) + "|")
    
    for row in table_data:
        # 열 수 맞추기
        padded_row = row + [""] * (max_cols - len(row))
        padded_row = padded_row[:max_cols]
        
        # 셀 내용 정리 (너무 긴 내용은 줄임)
        clean_row = []
        for cell in padded_row:
            if len(cell) > 50:
                cell = cell[:47] + "..."
            clean_row.append(cell)
        
        markdown_lines.append("| " + " | ".join(clean_row) + " |")
    
    return "\n".join(markdown_lines)

def extract_with_basic_layout(page, image_areas, page_num):
    """기본 레이아웃 보존 추출"""
    
    try:
        # 정렬된 텍스트 추출
        text = page.get_text("text", sort=True)
        
        if not text or not text.strip():
            return ""
        
        # 페이지 헤더 추가
        formatted_text = f"\n\n--- 페이지 {page_num} ---\n\n"
        formatted_text += text
        
        return formatted_text
        
    except Exception as e:
        return ""

def extract_with_sorted_text(page, image_areas, page_num):
    """정렬된 텍스트 추출"""
    
    try:
        # 기본 텍스트 추출
        text = page.get_text()
        
        if not text or not text.strip():
            return ""
        
        # 줄 단위로 정리
        lines = text.split('\n')
        clean_lines = []
        
        for line in lines:
            line = line.strip()
            if line:
                clean_lines.append(line)
        
        if clean_lines:
            formatted_text = f"\n\n--- 페이지 {page_num} ---\n\n"
            formatted_text += "\n".join(clean_lines)
            return formatted_text
        
        return ""
        
    except Exception as e:
        return ""

def evaluate_text_layout_quality(content):
    """텍스트 레이아웃 품질 평가"""
    
    if not content or not content.strip():
        return 0
    
    score = 0
    
    # 텍스트 길이
    length = len(content.strip())
    score += min(length * 0.01, 50)
    
    # 줄바꿈 구조
    lines = [line.strip() for line in content.split('\n') if line.strip()]
    score += len(lines) * 0.5
    
    # 한글 포함 여부
    korean_chars = sum(1 for c in content if '가' <= c <= '힣')
    score += korean_chars * 0.02
    
    # 과실비율 관련 키워드
    keywords = ['과실', '비율', '조', '항', '호', '기준', '적용']
    for keyword in keywords:
        if keyword in content:
            score += 5
    
    # 구조적 요소 (번호, 제목 등)
    if re.search(r'제\d+조', content):
        score += 10
    if re.search(r'\d+\.\s', content):
        score += 5
    
    return score

def save_extracted_content(all_pages_content, stats):
    """추출된 내용을 파일로 저장"""
    
    # 메인 파일 (전체 내용)
    main_output = "과실비율_전체텍스트.md"
    
    content = f"""# 과실비율 인정기준 전체 텍스트

> 추출 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
> 전체 페이지: {stats['total_pages']}
> 텍스트 페이지: {stats['pages_with_text']}
> 총 글자 수: {stats['total_characters']:,}
> 제외된 이미지: {stats['images_excluded']}개

---

"""
    
    # 모든 페이지 내용 추가
    for page_content in all_pages_content:
        content += page_content
    
    # 파일 저장
    with open(main_output, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 전체 텍스트 저장: {main_output}")
    
    # 구조화된 버전도 저장
    save_structured_version(all_pages_content, stats)

def save_structured_version(all_pages_content, stats):
    """구조화된 버전으로도 저장"""
    
    structured_output = "과실비율_구조화텍스트.md"
    
    content = f"""# 과실비율 인정기준 (구조화)

> 추출 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📊 문서 통계

- **전체 페이지**: {stats['total_pages']}
- **텍스트 페이지**: {stats['pages_with_text']}
- **총 글자 수**: {stats['total_characters']:,}
- **제외된 이미지**: {stats['images_excluded']}개

---

## 📄 본문 내용

"""
    
    # 과실비율 관련 섹션들을 구분해서 저장
    current_section = ""
    
    for page_content in all_pages_content:
        # 조문이나 큰 제목 감지
        if re.search(r'제\d+조|제\d+장|제\d+편', page_content):
            sections = re.findall(r'(제\d+조[^제]*|제\d+장[^제]*|제\d+편[^제]*)', page_content)
            for section in sections:
                content += f"\n## {section[:50]}...\n\n"
                content += section + "\n\n"
        else:
            content += page_content
    
    with open(structured_output, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 구조화 텍스트 저장: {structured_output}")

if __name__ == "__main__":
    extract_text_preserve_layout()

📄 PDF 텍스트 추출 (레이아웃 보존)
📊 전체 페이지 수: 600
🚀 텍스트 추출 시작 (이미지 영역 제외)...
진행률: 1/600 (0.2%)
진행률: 11/600 (1.8%)
진행률: 21/600 (3.5%)
진행률: 31/600 (5.2%)
진행률: 41/600 (6.8%)
진행률: 51/600 (8.5%)
진행률: 61/600 (10.2%)
진행률: 71/600 (11.8%)
진행률: 81/600 (13.5%)
진행률: 91/600 (15.2%)
진행률: 101/600 (16.8%)
진행률: 111/600 (18.5%)
진행률: 121/600 (20.2%)
진행률: 131/600 (21.8%)
진행률: 141/600 (23.5%)
진행률: 151/600 (25.2%)
진행률: 161/600 (26.8%)
진행률: 171/600 (28.5%)
진행률: 181/600 (30.2%)
진행률: 191/600 (31.8%)
진행률: 201/600 (33.5%)
진행률: 211/600 (35.2%)
진행률: 221/600 (36.8%)
진행률: 231/600 (38.5%)
진행률: 241/600 (40.2%)
진행률: 251/600 (41.8%)
진행률: 261/600 (43.5%)
진행률: 271/600 (45.2%)
진행률: 281/600 (46.8%)
진행률: 291/600 (48.5%)
진행률: 301/600 (50.2%)
진행률: 311/600 (51.8%)
진행률: 321/600 (53.5%)
진행률: 331/600 (55.2%)
진행률: 341/600 (56.8%)
진행률: 351/600 (58.5%)
진행률: 361/600 (60.2%)
진행률: 371/600 (61.8%)
진행률: 381/600 (63.5%)
진행률: 391/600 (65.2%)
진행률: 401/600 (66.8%)
진행률: 411/600 (68.5%)
진행률: 421/600 (70.2%)
진행률: 431/600 (71.8%)
진행률: 441/600 (73.5%)
진행률: 4

In [12]:
import cv2
import numpy as np
import pytesseract
import re
import os

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def process_fault_ratio_table_image(image_path):
    """과실비율 표 이미지 전용 처리"""
    
    print(f"🎯 과실비율 표 이미지 처리: {os.path.basename(image_path)}")
    print("=" * 50)
    
    if not os.path.exists(image_path):
        print(f"❌ 이미지를 찾을 수 없습니다: {image_path}")
        return
    
    img = cv2.imread(image_path)
    if img is None:
        print(f"❌ 이미지를 읽을 수 없습니다")
        return
    
    h, w = img.shape[:2]
    print(f"📐 이미지 크기: {w} x {h}")
    
    # 1. 오른쪽 표 영역만 분리 (다이어그램 제외)
    table_region = extract_table_region(img)
    
    # 2. 여러 전처리 방법으로 OCR 시도
    ocr_results = []
    
    # 방법 1: 기본 전처리
    result1 = process_with_basic_preprocessing(table_region)
    if result1:
        ocr_results.append(("기본 전처리", result1))
    
    # 방법 2: 색상 분리
    result2 = process_with_color_separation(table_region)
    if result2:
        ocr_results.append(("색상 분리", result2))
    
    # 방법 3: 고해상도 + 강화
    result3 = process_with_enhancement(table_region)
    if result3:
        ocr_results.append(("고해상도 강화", result3))
    
    # 방법 4: 영역별 분할 처리
    result4 = process_with_region_splitting(table_region)
    if result4:
        ocr_results.append(("영역별 분할", result4))
    
    # 3. 가장 좋은 결과 선택 및 후처리
    if ocr_results:
        best_result = select_best_ocr_result(ocr_results)
        formatted_result = format_as_fault_ratio_table(best_result)
        
        # 결과 저장
        save_result(image_path, formatted_result, ocr_results)
        
        print(f"\n🎉 처리 완료!")
        print(f"📄 결과 파일: {os.path.splitext(os.path.basename(image_path))[0]}_과실비율표.md")
    else:
        print(f"\n❌ 모든 OCR 방법 실패")

def extract_table_region(img):
    """이미지에서 표 영역만 추출 (오른쪽 부분)"""
    
    h, w = img.shape[:2]
    
    # 오른쪽 60% 영역 (표가 있는 부분)
    table_start_x = int(w * 0.4)
    table_region = img[:, table_start_x:]
    
    print(f"📊 표 영역 추출: {table_region.shape[1]} x {table_region.shape[0]}")
    
    return table_region

def process_with_basic_preprocessing(img):
    """기본 전처리로 OCR"""
    
    try:
        # 그레이스케일 변환
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 크기 확대 (3배)
        scale_factor = 3
        h, w = gray.shape
        enlarged = cv2.resize(gray, (w * scale_factor, h * scale_factor), 
                            interpolation=cv2.INTER_CUBIC)
        
        # 노이즈 제거
        denoised = cv2.medianBlur(enlarged, 3)
        
        # 샤프닝
        kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        sharpened = cv2.filter2D(denoised, -1, kernel)
        
        # OCR 실행
        config = r'--oem 3 --psm 6 -l kor+eng -c preserve_interword_spaces=1'
        result = pytesseract.image_to_string(sharpened, config=config)
        
        print(f"   기본 전처리: {len(result)}자 추출")
        return result
        
    except Exception as e:
        print(f"   기본 전처리 실패: {e}")
        return ""

def process_with_color_separation(img):
    """색상 분리로 텍스트 영역 강조"""
    
    try:
        # HSV 색공간으로 변환
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # 흰색/밝은 영역 마스크 (텍스트 배경)
        lower_white = np.array([0, 0, 200])
        upper_white = np.array([180, 30, 255])
        white_mask = cv2.inRange(hsv, lower_white, upper_white)
        
        # 마스크 적용
        result = cv2.bitwise_and(img, img, mask=white_mask)
        gray = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
        
        # 이진화
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        # 크기 확대
        scale_factor = 4
        h, w = binary.shape
        enlarged = cv2.resize(binary, (w * scale_factor, h * scale_factor), 
                            interpolation=cv2.INTER_CUBIC)
        
        # OCR
        config = r'--oem 3 --psm 6 -l kor+eng'
        result = pytesseract.image_to_string(enlarged, config=config)
        
        print(f"   색상 분리: {len(result)}자 추출")
        return result
        
    except Exception as e:
        print(f"   색상 분리 실패: {e}")
        return ""

def process_with_enhancement(img):
    """고해상도 + 강화 처리"""
    
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 매우 큰 크기로 확대 (5배)
        scale_factor = 5
        h, w = gray.shape
        enlarged = cv2.resize(gray, (w * scale_factor, h * scale_factor), 
                            interpolation=cv2.INTER_CUBIC)
        
        # 대비 향상
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
        enhanced = clahe.apply(enlarged)
        
        # 강한 샤프닝
        kernel = np.array([[-2,-2,-2], [-2,17,-2], [-2,-2,-2]])
        sharpened = cv2.filter2D(enhanced, -1, kernel)
        
        # 모폴로지 연산으로 글자 정리
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
        cleaned = cv2.morphologyEx(sharpened, cv2.MORPH_CLOSE, kernel)
        
        # OCR
        config = r'--oem 3 --psm 4 -l kor+eng -c preserve_interword_spaces=1'
        result = pytesseract.image_to_string(cleaned, config=config)
        
        print(f"   고해상도 강화: {len(result)}자 추출")
        return result
        
    except Exception as e:
        print(f"   고해상도 강화 실패: {e}")
        return ""

def process_with_region_splitting(img):
    """영역을 나누어서 개별 처리"""
    
    try:
        h, w = img.shape[:2]
        
        # 상단/하단으로 분할
        top_half = img[:h//2, :]
        bottom_half = img[h//2:, :]
        
        results = []
        
        for i, region in enumerate([top_half, bottom_half], 1):
            gray = cv2.cvtColor(region, cv2.COLOR_BGR2GRAY)
            
            # 확대 및 전처리
            scale_factor = 4
            rh, rw = gray.shape
            enlarged = cv2.resize(gray, (rw * scale_factor, rh * scale_factor), 
                                interpolation=cv2.INTER_CUBIC)
            
            # 적응적 이진화
            binary = cv2.adaptiveThreshold(enlarged, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                         cv2.THRESH_BINARY, 11, 2)
            
            # OCR
            config = r'--oem 3 --psm 6 -l kor+eng'
            text = pytesseract.image_to_string(binary, config=config)
            
            if text.strip():
                results.append(text.strip())
        
        combined_result = "\n".join(results)
        print(f"   영역별 분할: {len(combined_result)}자 추출")
        return combined_result
        
    except Exception as e:
        print(f"   영역별 분할 실패: {e}")
        return ""

def select_best_ocr_result(ocr_results):
    """가장 좋은 OCR 결과 선택"""
    
    best_result = ""
    best_score = 0
    best_method = ""
    
    for method, result in ocr_results:
        score = evaluate_fault_ratio_result(result)
        print(f"   {method}: 점수 {score:.1f}")
        
        if score > best_score:
            best_score = score
            best_result = result
            best_method = method
    
    print(f"✅ 최종 선택: {best_method} (점수: {best_score:.1f})")
    return best_result

def evaluate_fault_ratio_result(text):
    """과실비율 OCR 결과 품질 평가"""
    
    if not text or not text.strip():
        return 0
    
    score = 0
    
    # 1. 과실비율 관련 키워드
    keywords = ['과실', '비율', '보행자', '차량', '가산', '감산', '기본']
    for keyword in keywords:
        if keyword in text:
            score += 10
    
    # 2. 숫자와 퍼센트
    numbers = re.findall(r'\d+', text)
    score += len(numbers) * 2
    
    if '%' in text:
        score += 5
    
    # 3. 구조적 요소
    if re.search(r'[ㅣ|]', text):  # 구분자
        score += 5
    
    # 4. 한글 비율
    korean_chars = sum(1 for c in text if '가' <= c <= '힣')
    score += korean_chars * 0.1
    
    # 5. 텍스트 길이 (적당한 길이면 가점)
    length = len(text.strip())
    if 50 <= length <= 500:
        score += length * 0.05
    
    return score

def format_as_fault_ratio_table(raw_text):
    """추출된 텍스트를 과실비율 표로 포맷팅"""
    
    if not raw_text or not raw_text.strip():
        return "텍스트를 추출할 수 없습니다."
    
    lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
    
    # 제목 추출
    title = ""
    if lines:
        first_line = lines[0]
        if any(word in first_line for word in ['보행자', '차량', '사고', '횡단']):
            title = first_line
            lines = lines[1:]
    
    # 과실비율 데이터 추출
    formatted_lines = []
    
    for line in lines:
        # 구분자로 분리 시도
        if 'ㅣ' in line:
            parts = line.split('ㅣ')
        elif '|' in line:
            parts = line.split('|')
        else:
            # 숫자 앞에서 분리
            match = re.search(r'(.+?)(\d+%?)(.*)$', line)
            if match:
                parts = [match.group(1), match.group(2), match.group(3)]
            else:
                parts = [line]
        
        # 부분들 정리
        clean_parts = [part.strip() for part in parts if part.strip()]
        
        if clean_parts:
            formatted_lines.append(clean_parts)
    
    # 마크다운 표 생성
    result = ""
    
    if title:
        result += f"# {title}\n\n"
    
    if formatted_lines:
        # 테이블 헤더 결정
        max_cols = max(len(row) for row in formatted_lines) if formatted_lines else 2
        
        if max_cols <= 2:
            headers = ["구분", "내용"]
        elif max_cols == 3:
            headers = ["구분", "내용", "비율"]
        else:
            headers = ["구분", "내용", "비율", "비고"]
        
        # 마크다운 테이블
        result += "| " + " | ".join(headers) + " |\n"
        result += "|" + "|".join([" --- " for _ in headers]) + "|\n"
        
        for row in formatted_lines:
            # 열 수 맞추기
            padded_row = row + [""] * (len(headers) - len(row))
            padded_row = padded_row[:len(headers)]
            
            result += "| " + " | ".join(padded_row) + " |\n"
    
    result += f"\n**원본 텍스트:**\n```\n{raw_text}\n```\n"
    
    return result

def save_result(image_path, formatted_result, all_results):
    """결과를 파일로 저장"""
    
    base_name = os.path.splitext(os.path.basename(image_path))[0]
    output_file = f"{base_name}_과실비율표.md"
    
    content = f"""# 과실비율표 OCR 추출 결과

> 이미지: {os.path.basename(image_path)}
> 처리 일시: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 최종 결과

{formatted_result}

## 모든 OCR 시도 결과

"""
    
    for method, result in all_results:
        content += f"### {method}\n\n"
        content += f"```\n{result[:300]}...\n```\n\n"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 결과 저장: {output_file}")

# 테스트용 함수를 전체 처리 함수로 변경
def process_full_pdf_with_specialized_ocr():
    """PDF 전체를 과실비율 표 전용 OCR로 처리"""
    
    print("🚀 PDF 전체 과실비율 표 OCR 처리")
    print("=" * 60)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        import fitz  # PyMuPDF
        
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📄 전체 페이지: {total_pages}")
        print(f"🎯 39페이지부터 처리 시작...")
        
        all_extracted_tables = []
        stats = {
            'total_images': 0,
            'successful_extractions': 0,
            'failed_extractions': 0
        }
        
        # 39페이지부터 전체 처리
        start_page = 38  # 39페이지 (0-based)
        
        for page_num in range(start_page, total_pages):
            print(f"\n📄 페이지 {page_num+1} 처리 중...")
            
            page = doc.load_page(page_num)
            image_list = page.get_images()
            
            if not image_list:
                print(f"   ℹ️ 이미지 없음")
                continue
            
            print(f"   🖼️ {len(image_list)}개 이미지 발견")
            stats['total_images'] += len(image_list)
            
            for img_index, img in enumerate(image_list):
                try:
                    # 이미지를 메모리에서 직접 처리
                    xref = img[0]
                    pix = fitz.Pixmap(doc, xref)
                    
                    if pix.n - pix.alpha < 4:  # GRAY or RGB
                        # numpy 배열로 변환
                        img_data = pix.tobytes("png")
                        nparr = np.frombuffer(img_data, np.uint8)
                        cv_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                        
                        if cv_image is not None:
                            h, w = cv_image.shape[:2]
                            print(f"      이미지 {img_index+1}: {w}x{h}")
                            
                            # 충분히 큰 이미지만 처리 (작은 아이콘 제외)
                            if w >= 300 and h >= 200:
                                # 과실비율 표 OCR 처리
                                table_result = process_fault_ratio_image_direct(
                                    cv_image, page_num+1, img_index+1
                                )
                                
                                if table_result:
                                    all_extracted_tables.append(table_result)
                                    stats['successful_extractions'] += 1
                                    print(f"         ✅ 과실비율 표 추출 성공!")
                                else:
                                    stats['failed_extractions'] += 1
                                    print(f"         ❌ 추출 실패")
                            else:
                                print(f"         ⚠️ 너무 작은 이미지 (아이콘으로 추정)")
                    
                    pix = None
                    
                except Exception as e:
                    stats['failed_extractions'] += 1
                    print(f"      ❌ 이미지 {img_index+1} 처리 오류: {e}")
        
        doc.close()
        
        # 전체 결과 저장
        save_all_fault_ratio_tables(all_extracted_tables, stats)
        
        print(f"\n🎉 전체 처리 완료!")
        print(f"📊 최종 통계:")
        print(f"   - 전체 이미지: {stats['total_images']}")
        print(f"   - 성공 추출: {stats['successful_extractions']}")
        print(f"   - 실패: {stats['failed_extractions']}")
        
        if stats['total_images'] > 0:
            success_rate = (stats['successful_extractions'] / stats['total_images']) * 100
            print(f"   - 성공률: {success_rate:.1f}%")
        
    except Exception as e:
        print(f"❌ PDF 처리 오류: {e}")

def process_fault_ratio_image_direct(cv_image, page_num, img_num):
    """이미지를 직접 과실비율 표로 처리"""
    
    try:
        h, w = cv_image.shape[:2]
        
        # 1. 표 영역 추출 (오른쪽 부분)
        table_region = extract_table_region(cv_image)
        
        # 2. 여러 OCR 방법 시도
        ocr_results = []
        
        # 방법 1: 기본 전처리
        result1 = process_with_basic_preprocessing(table_region)
        if result1 and len(result1.strip()) > 10:
            ocr_results.append(("기본", result1))
        
        # 방법 2: 색상 분리
        result2 = process_with_color_separation(table_region)
        if result2 and len(result2.strip()) > 10:
            ocr_results.append(("색상분리", result2))
        
        # 방법 3: 고해상도
        result3 = process_with_enhancement(table_region)
        if result3 and len(result3.strip()) > 10:
            ocr_results.append(("고해상도", result3))
        
        if not ocr_results:
            return None
        
        # 3. 가장 좋은 결과 선택
        best_result = select_best_ocr_result(ocr_results)
        
        # 4. 과실비율 관련 내용인지 확인
        if not is_fault_ratio_content(best_result):
            return None
        
        # 5. 표 형태로 포맷팅
        formatted_result = format_as_fault_ratio_table_simple(best_result, page_num, img_num)
        
        return {
            'page': page_num,
            'image': img_num,
            'content': formatted_result,
            'raw_text': best_result
        }
        
    except Exception as e:
        return None

def is_fault_ratio_content(text):
    """과실비율 관련 내용인지 판단"""
    
    if not text or len(text.strip()) < 10:
        return False
    
    # 과실비율 관련 키워드 확인
    keywords = ['과실', '비율', '보행자', '차량', '가산', '감산', '기본']
    keyword_count = sum(1 for keyword in keywords if keyword in text)
    
    # 숫자나 퍼센트 포함 확인
    has_numbers = bool(re.search(r'\d+', text))
    
    # 최소 조건: 키워드 1개 이상 + 숫자 포함
    return keyword_count >= 1 and has_numbers

def format_as_fault_ratio_table_simple(raw_text, page_num, img_num):
    """간단한 과실비율 표 포맷팅"""
    
    result = f"## 페이지 {page_num}, 이미지 {img_num}\n\n"
    
    lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
    
    # 제목 찾기
    title_line = ""
    content_lines = lines
    
    if lines:
        first_line = lines[0]
        if any(word in first_line for word in ['보행자', '차량', '횡단', '사고']):
            title_line = first_line
            content_lines = lines[1:]
    
    if title_line:
        result += f"**{title_line}**\n\n"
    
    # 표 데이터 추출
    table_data = []
    
    for line in content_lines:
        if any(keyword in line for keyword in ['과실', '비율', '가산', '감산', '%']) or re.search(r'\d+', line):
            # 구분자로 분리
            if 'ㅣ' in line:
                parts = [p.strip() for p in line.split('ㅣ') if p.strip()]
            elif '|' in line:
                parts = [p.strip() for p in line.split('|') if p.strip()]
            else:
                parts = [line.strip()]
            
            if parts:
                table_data.append(parts)
    
    if table_data:
        # 간단한 리스트 형태로 표시
        result += "**추출된 내용:**\n\n"
        for row in table_data:
            if len(row) == 1:
                result += f"- {row[0]}\n"
            else:
                result += f"- {' | '.join(row)}\n"
        result += "\n"
    
    result += f"**원본 텍스트:**\n```\n{raw_text[:200]}...\n```\n\n"
    result += "---\n\n"
    
    return result

def save_all_fault_ratio_tables(all_tables, stats):
    """모든 과실비율 표를 하나의 파일로 저장"""
    
    output_file = "과실비율_전체표_OCR추출.md"
    
    content = f"""# 과실비율 PDF 전체 표 OCR 추출 결과

> 추출 일시: {__import__('datetime').datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
> 전체 이미지: {stats['total_images']}개
> 성공 추출: {stats['successful_extractions']}개
> 실패: {stats['failed_extractions']}개
> 성공률: {(stats['successful_extractions']/stats['total_images']*100) if stats['total_images'] > 0 else 0:.1f}%

---

"""
    
    if all_tables:
        for i, table in enumerate(all_tables, 1):
            content += f"# 표 {i}\n\n"
            content += table['content']
    else:
        content += "## ⚠️ 추출된 과실비율 표가 없습니다\n\n"
        content += "모든 이미지가 작은 아이콘이거나 과실비율과 관련없는 내용이었습니다.\n"
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 전체 결과 저장: {output_file}")

if __name__ == "__main__":
    process_full_pdf_with_specialized_ocr()

🚀 PDF 전체 과실비율 표 OCR 처리
📄 전체 페이지: 600
🎯 39페이지부터 처리 시작...

📄 페이지 39 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x310
📊 표 영역 추출: 187 x 310
   기본 전처리: 79자 추출
   색상 분리: 79자 추출
   고해상도 강화: 0자 추출
   기본: 점수 12.2
   색상분리: 점수 25.0
✅ 최종 선택: 색상분리 (점수: 25.0)
         ❌ 추출 실패

📄 페이지 40 처리 중...
   ℹ️ 이미지 없음

📄 페이지 41 처리 중...
   ℹ️ 이미지 없음

📄 페이지 42 처리 중...
   ℹ️ 이미지 없음

📄 페이지 43 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 64자 추출
   색상 분리: 93자 추출
   고해상도 강화: 0자 추출
   기본: 점수 6.2
   색상분리: 점수 11.7
✅ 최종 선택: 색상분리 (점수: 11.7)
         ❌ 추출 실패

📄 페이지 44 처리 중...
   ℹ️ 이미지 없음

📄 페이지 45 처리 중...
   ℹ️ 이미지 없음

📄 페이지 46 처리 중...
   ℹ️ 이미지 없음

📄 페이지 47 처리 중...
   🖼️ 2개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 53자 추출
   색상 분리: 66자 추출
   고해상도 강화: 0자 추출
   기본: 점수 9.9
   색상분리: 점수 13.4
✅ 최종 선택: 색상분리 (점수: 13.4)
         ❌ 추출 실패
      이미지 2: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 6자 추출
   색상 분리: 86자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 13.3
✅ 최종 선택: 색상분리 (점수: 13.3)
         ❌ 추출 실

   기본 전처리: 6자 추출
   색상 분리: 286자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 23.6
✅ 최종 선택: 색상분리 (점수: 23.6)
         ❌ 추출 실패

📄 페이지 120 처리 중...
   ℹ️ 이미지 없음

📄 페이지 121 처리 중...
   🖼️ 2개 이미지 발견
      이미지 1: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 6자 추출
   색상 분리: 11자 추출
   고해상도 강화: 0자 추출
         ❌ 추출 실패
      이미지 2: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 14자 추출
   색상 분리: 11자 추출
   고해상도 강화: 0자 추출
   기본: 점수 5.0
✅ 최종 선택: 기본 (점수: 5.0)
         ❌ 추출 실패

📄 페이지 122 처리 중...
   ℹ️ 이미지 없음

📄 페이지 123 처리 중...
   ℹ️ 이미지 없음

📄 페이지 124 처리 중...
   ℹ️ 이미지 없음

📄 페이지 125 처리 중...
   ℹ️ 이미지 없음

📄 페이지 126 처리 중...
   ℹ️ 이미지 없음

📄 페이지 127 처리 중...
   ℹ️ 이미지 없음

📄 페이지 128 처리 중...
   ℹ️ 이미지 없음

📄 페이지 129 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 150x150
         ⚠️ 너무 작은 이미지 (아이콘으로 추정)

📄 페이지 130 처리 중...
   ℹ️ 이미지 없음

📄 페이지 131 처리 중...
   ℹ️ 이미지 없음

📄 페이지 132 처리 중...
   ℹ️ 이미지 없음

📄 페이지 133 처리 중...
   ℹ️ 이미지 없음

📄 페이지 134 처리 중...
   ℹ️ 이미지 없음

📄 페이지 135 처리 중...
   ℹ️ 이미지 없음

📄 페이지 136 처리 중...
   ℹ️ 이미지 없음

📄 페이지 137 처리 중...
   ℹ️ 

   색상 분리: 12자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 0.0
✅ 최종 선택:  (점수: 0.0)
         ❌ 추출 실패

📄 페이지 231 처리 중...
   ℹ️ 이미지 없음

📄 페이지 232 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 4자 추출
   색상 분리: 22자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 5.0
✅ 최종 선택: 색상분리 (점수: 5.0)
         ❌ 추출 실패

📄 페이지 233 처리 중...
   ℹ️ 이미지 없음

📄 페이지 234 처리 중...
   ℹ️ 이미지 없음

📄 페이지 235 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 33자 추출
   색상 분리: 21자 추출
   고해상도 강화: 0자 추출
   기본: 점수 5.1
   색상분리: 점수 4.1
✅ 최종 선택: 기본 (점수: 5.1)
         ❌ 추출 실패

📄 페이지 236 처리 중...
   ℹ️ 이미지 없음

📄 페이지 237 처리 중...
   ℹ️ 이미지 없음

📄 페이지 238 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 7자 추출
   색상 분리: 13자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 0.2
✅ 최종 선택: 색상분리 (점수: 0.2)
         ❌ 추출 실패

📄 페이지 239 처리 중...
   ℹ️ 이미지 없음

📄 페이지 240 처리 중...
   ℹ️ 이미지 없음

📄 페이지 241 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 5자 추출
   색상 분리: 34자 추출
   고해상

   색상 분리: 16자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 7.0
✅ 최종 선택: 색상분리 (점수: 7.0)
         ❌ 추출 실패

📄 페이지 326 처리 중...
   ℹ️ 이미지 없음

📄 페이지 327 처리 중...
   ℹ️ 이미지 없음

📄 페이지 328 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 13자 추출
   색상 분리: 2자 추출
   고해상도 강화: 0자 추출
   기본: 점수 2.1
✅ 최종 선택: 기본 (점수: 2.1)
         ❌ 추출 실패

📄 페이지 329 처리 중...
   ℹ️ 이미지 없음

📄 페이지 330 처리 중...
   ℹ️ 이미지 없음

📄 페이지 331 처리 중...
   ℹ️ 이미지 없음

📄 페이지 332 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 18자 추출
   색상 분리: 6자 추출
   고해상도 강화: 0자 추출
   기본: 점수 6.0
✅ 최종 선택: 기본 (점수: 6.0)
         ❌ 추출 실패

📄 페이지 333 처리 중...
   ℹ️ 이미지 없음

📄 페이지 334 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 2자 추출
   색상 분리: 4자 추출
   고해상도 강화: 0자 추출
         ❌ 추출 실패

📄 페이지 335 처리 중...
   ℹ️ 이미지 없음

📄 페이지 336 처리 중...
   ℹ️ 이미지 없음

📄 페이지 337 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 24자 추출
   색상 분리: 17자 추출
   고해상도 강화: 0자 추출
   기본: 점수 0.0

   색상 분리: 7자 추출
   고해상도 강화: 0자 추출
   기본: 점수 9.0
✅ 최종 선택: 기본 (점수: 9.0)
         ❌ 추출 실패

📄 페이지 424 처리 중...
   ℹ️ 이미지 없음

📄 페이지 425 처리 중...
   ℹ️ 이미지 없음

📄 페이지 426 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 121자 추출
   색상 분리: 90자 추출
   고해상도 강화: 0자 추출
   기본: 점수 15.5
   색상분리: 점수 8.6
✅ 최종 선택: 기본 (점수: 15.5)
         ❌ 추출 실패

📄 페이지 427 처리 중...
   ℹ️ 이미지 없음

📄 페이지 428 처리 중...
   ℹ️ 이미지 없음

📄 페이지 429 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 63자 추출
   색상 분리: 13자 추출
   고해상도 강화: 8자 추출
   기본: 점수 9.5
   색상분리: 점수 5.0
✅ 최종 선택: 기본 (점수: 9.5)
         ❌ 추출 실패

📄 페이지 430 처리 중...
   ℹ️ 이미지 없음

📄 페이지 431 처리 중...
   ℹ️ 이미지 없음

📄 페이지 432 처리 중...
   ℹ️ 이미지 없음

📄 페이지 433 처리 중...
   🖼️ 1개 이미지 발견
      이미지 1: 311x311
📊 표 영역 추출: 187 x 311
   기본 전처리: 19자 추출
   색상 분리: 17자 추출
   고해상도 강화: 0자 추출
   기본: 점수 5.1
   색상분리: 점수 7.0
✅ 최종 선택: 색상분리 (점수: 7.0)
         ❌ 추출 실패

📄 페이지 434 처리 중...
   ℹ️ 이미지 없음

📄 페이지 435 처리 중...
   ℹ️ 이미지 없음

📄 페이지 436 처리 중...
   🖼

   색상 분리: 20자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 0.2
✅ 최종 선택: 색상분리 (점수: 0.2)
         ❌ 추출 실패
      이미지 2: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 46자 추출
   색상 분리: 43자 추출
   고해상도 강화: 0자 추출
   기본: 점수 5.3
   색상분리: 점수 9.2
✅ 최종 선택: 색상분리 (점수: 9.2)
         ❌ 추출 실패

📄 페이지 520 처리 중...
   ℹ️ 이미지 없음

📄 페이지 521 처리 중...
   ℹ️ 이미지 없음

📄 페이지 522 처리 중...
   🖼️ 2개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 2자 추출
   색상 분리: 12자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 4.2
✅ 최종 선택: 색상분리 (점수: 4.2)
         ❌ 추출 실패
      이미지 2: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 14자 추출
   색상 분리: 16자 추출
   고해상도 강화: 0자 추출
   기본: 점수 0.0
   색상분리: 점수 2.2
✅ 최종 선택: 색상분리 (점수: 2.2)
         ❌ 추출 실패

📄 페이지 523 처리 중...
   ℹ️ 이미지 없음

📄 페이지 524 처리 중...
   ℹ️ 이미지 없음

📄 페이지 525 처리 중...
   🖼️ 2개 이미지 발견
      이미지 1: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 6자 추출
   색상 분리: 18자 추출
   고해상도 강화: 0자 추출
   색상분리: 점수 0.1
✅ 최종 선택: 색상분리 (점수: 0.1)
         ❌ 추출 실패
      이미지 2: 312x312
📊 표 영역 추출: 188 x 312
   기본 전처리: 2자 추출
   색상 분리: 19자 추출


      이미지 1: 242x242
         ⚠️ 너무 작은 이미지 (아이콘으로 추정)
      이미지 2: 242x240
         ⚠️ 너무 작은 이미지 (아이콘으로 추정)
      이미지 3: 242x242
         ⚠️ 너무 작은 이미지 (아이콘으로 추정)

📄 페이지 585 처리 중...
   ℹ️ 이미지 없음

📄 페이지 586 처리 중...
   ℹ️ 이미지 없음

📄 페이지 587 처리 중...
   ℹ️ 이미지 없음

📄 페이지 588 처리 중...
   ℹ️ 이미지 없음

📄 페이지 589 처리 중...
   ℹ️ 이미지 없음

📄 페이지 590 처리 중...
   ℹ️ 이미지 없음

📄 페이지 591 처리 중...
   ℹ️ 이미지 없음

📄 페이지 592 처리 중...
   ℹ️ 이미지 없음

📄 페이지 593 처리 중...
   ℹ️ 이미지 없음

📄 페이지 594 처리 중...
   ℹ️ 이미지 없음

📄 페이지 595 처리 중...
   ℹ️ 이미지 없음

📄 페이지 596 처리 중...
   ℹ️ 이미지 없음

📄 페이지 597 처리 중...
   ℹ️ 이미지 없음

📄 페이지 598 처리 중...
   ℹ️ 이미지 없음

📄 페이지 599 처리 중...
   ℹ️ 이미지 없음

📄 페이지 600 처리 중...
   ℹ️ 이미지 없음
📄 전체 결과 저장: 과실비율_전체표_OCR추출.md

🎉 전체 처리 완료!
📊 최종 통계:
   - 전체 이미지: 218
   - 성공 추출: 2
   - 실패: 200
   - 성공률: 0.9%


In [13]:
import fitz  # PyMuPDF
import cv2
import numpy as np
import pytesseract
import os
import re
from datetime import datetime

# Tesseract 경로 설정
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def hybrid_pdf_extraction():
    """하이브리드 방식: 직접 텍스트 + 이미지 OCR 결합"""
    
    print("🔄 하이브리드 PDF 추출 (직접 텍스트 + 이미지 OCR)")
    print("=" * 60)
    
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\과실비율 원본 PDF\231107_과실비율인정기준_온라인용.pdf"
    
    if not os.path.exists(pdf_path):
        print(f"❌ PDF 파일을 찾을 수 없습니다: {pdf_path}")
        return
    
    try:
        doc = fitz.open(pdf_path)
        total_pages = len(doc)
        
        print(f"📄 전체 페이지: {total_pages}")
        
        all_content = []
        stats = {
            'total_pages': total_pages,
            'direct_text_pages': 0,
            'image_ocr_pages': 0,
            'fault_ratio_sections': 0,
            'total_chars': 0
        }
        
        for page_num in range(total_pages):
            if page_num % 50 == 0:
                progress = (page_num + 1) / total_pages * 100
                print(f"📈 진행률: {page_num+1}/{total_pages} ({progress:.1f}%)")
            
            page = doc.load_page(page_num)
            page_content = extract_page_hybrid(page, page_num + 1, stats)
            
            if page_content:
                all_content.append(page_content)
        
        doc.close()
        
        # 결과 저장
        save_hybrid_results(all_content, stats)
        
        print(f"\n🎉 하이브리드 추출 완료!")
        print(f"📊 최종 통계:")
        print(f"   - 전체 페이지: {stats['total_pages']}")
        print(f"   - 직접 텍스트: {stats['direct_text_pages']} 페이지")
        print(f"   - 이미지 OCR: {stats['image_ocr_pages']} 페이지") 
        print(f"   - 과실비율 섹션: {stats['fault_ratio_sections']}개")
        print(f"   - 총 글자 수: {stats['total_chars']:,}")
        
    except Exception as e:
        print(f"❌ 처리 오류: {e}")
        import traceback
        traceback.print_exc()

def extract_page_hybrid(page, page_num, stats):
    """페이지별 하이브리드 추출"""
    
    try:
        # 1. 직접 텍스트 추출 시도
        direct_text = extract_direct_text_from_page(page)
        
        # 2. 이미지 영역 확인
        image_list = page.get_images()
        
        page_content = f"\n\n--- 페이지 {page_num} ---\n\n"
        content_added = False
        
        # 3. 직접 텍스트가 충분하면 사용
        if direct_text and len(direct_text.strip()) > 50:
            formatted_direct = format_direct_text(direct_text, page_num)
            if formatted_direct:
                page_content += formatted_direct
                stats['direct_text_pages'] += 1
                stats['total_chars'] += len(formatted_direct)
                content_added = True
                
                # 과실비율 관련 내용인지 확인
                if is_fault_ratio_content_text(formatted_direct):
                    stats['fault_ratio_sections'] += 1
        
        # 4. 이미지가 있고 직접 텍스트가 부족하면 OCR 시도
        if image_list and (not content_added or len(direct_text.strip()) < 100):
            ocr_results = extract_images_from_page(page, page_num)
            
            if ocr_results:
                if not content_added:
                    page_content += "**이미지에서 추출된 내용:**\n\n"
                else:
                    page_content += "\n**추가 이미지 내용:**\n\n"
                
                for ocr_result in ocr_results:
                    page_content += ocr_result + "\n\n"
                
                stats['image_ocr_pages'] += 1
                content_added = True
        
        if content_added:
            return page_content
        else:
            return None
            
    except Exception as e:
        return None

def extract_direct_text_from_page(page):
    """페이지에서 직접 텍스트 추출 (레이아웃 보존)"""
    
    try:
        # 방법 1: 블록 기반 추출
        blocks = page.get_text("dict")
        if blocks and "blocks" in blocks:
            text_elements = []
            
            for block in blocks["blocks"]:
                if block.get("type") == 0:  # 텍스트 블록
                    for line in block.get("lines", []):
                        line_text = ""
                        line_spans = []
                        
                        for span in line.get("spans", []):
                            text = span.get("text", "").strip()
                            if text:
                                line_spans.append({
                                    'text': text,
                                    'x': span.get("bbox", [0, 0, 0, 0])[0]
                                })
                        
                        if line_spans:
                            # X 좌표로 정렬
                            line_spans.sort(key=lambda s: s['x'])
                            
                            # 적절한 간격으로 연결
                            prev_x = 0
                            line_parts = []
                            
                            for span in line_spans:
                                if prev_x > 0 and span['x'] - prev_x > 20:
                                    line_parts.append("    ")  # 큰 간격
                                elif prev_x > 0 and span['x'] - prev_x > 10:
                                    line_parts.append("  ")   # 작은 간격
                                
                                line_parts.append(span['text'])
                                prev_x = span['x'] + len(span['text']) * 6  # 대략적인 글자 폭
                            
                            line_text = "".join(line_parts)
                        
                        if line_text.strip():
                            y_pos = line.get("bbox", [0, 0, 0, 0])[1]
                            text_elements.append((y_pos, line_text.strip()))
            
            # Y 좌표로 정렬
            text_elements.sort()
            return "\n".join([text for y, text in text_elements])
        
        # 방법 2: 기본 텍스트 추출
        return page.get_text()
        
    except Exception as e:
        return ""

def format_direct_text(text, page_num):
    """직접 텍스트를 표 형태로 포맷팅"""
    
    if not text or not text.strip():
        return ""
    
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    
    if not lines:
        return ""
    
    # 과실비율 관련 라인 찾기
    fault_lines = []
    other_lines = []
    
    for line in lines:
        if (any(keyword in line for keyword in ['과실', '비율', '가산', '감산', '보행자', '차량']) or
            re.search(r'\d+%?', line)):
            fault_lines.append(line)
        else:
            other_lines.append(line)
    
    result = ""
    
    # 과실비율 표가 있으면 표 형태로 변환
    if fault_lines and len(fault_lines) >= 2:
        result += "**과실비율 표:**\n\n"
        
        table_md = convert_lines_to_table(fault_lines)
        if table_md:
            result += table_md + "\n\n"
        else:
            for line in fault_lines:
                result += f"- {line}\n"
            result += "\n"
    
    # 일반 텍스트
    if other_lines:
        important_lines = [line for line in other_lines 
                          if len(line) > 10 and not line.isdigit()][:10]  # 처음 10개만
        
        if important_lines:
            result += "**문서 내용:**\n\n"
            for line in important_lines:
                result += f"{line}\n\n"
    
    return result

def convert_lines_to_table(lines):
    """텍스트 라인들을 마크다운 표로 변환"""
    
    table_data = []
    
    for line in lines:
        # 여러 공백이나 탭으로 분할
        parts = re.split(r'\s{3,}|\t+', line)
        
        if len(parts) == 1:
            # 다른 구분 방법 시도
            if ':' in line:
                parts = [p.strip() for p in line.split(':', 1)]
            elif re.search(r'(\d+)%?', line):
                # 숫자 앞에서 분리
                match = re.search(r'(.+?)(\d+%?)(.*)$', line)
                if match:
                    parts = [match.group(1).strip(), match.group(2), match.group(3).strip()]
                    parts = [p for p in parts if p]
        
        clean_parts = [part.strip() for part in parts if part.strip()]
        
        if len(clean_parts) >= 2:
            table_data.append(clean_parts)
        elif len(clean_parts) == 1 and clean_parts[0]:
            table_data.append([clean_parts[0], ""])
    
    if not table_data or len(table_data) < 2:
        return None
    
    # 마크다운 표 생성
    max_cols = max(len(row) for row in table_data)
    
    if max_cols == 2:
        headers = ["구분", "내용"]
    elif max_cols == 3:
        headers = ["구분", "내용", "비율"]
    else:
        headers = ["구분"] + [f"항목{i}" for i in range(2, max_cols + 1)]
    
    markdown_lines = []
    markdown_lines.append("| " + " | ".join(headers) + " |")
    markdown_lines.append("|" + "|".join([" --- " for _ in headers]) + "|")
    
    for row in table_data:
        padded_row = row + [""] * (max_cols - len(row))
        padded_row = padded_row[:max_cols]
        markdown_lines.append("| " + " | ".join(padded_row) + " |")
    
    return "\n".join(markdown_lines)

def extract_images_from_page(page, page_num):
    """페이지의 이미지에서 OCR 추출"""
    
    try:
        image_list = page.get_images()
        ocr_results = []
        
        for img_index, img in enumerate(image_list):
            try:
                xref = img[0]
                pix = fitz.Pixmap(doc, xref)
                
                if pix.n - pix.alpha < 4:
                    img_data = pix.tobytes("png")
                    nparr = np.frombuffer(img_data, np.uint8)
                    cv_image = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
                    
                    if cv_image is not None:
                        h, w = cv_image.shape[:2]
                        
                        # 충분히 큰 이미지만 처리
                        if w >= 200 and h >= 150:
                            ocr_text = simple_ocr_extraction(cv_image)
                            
                            if ocr_text and len(ocr_text.strip()) > 20:
                                formatted = f"**이미지 {img_index+1} ({w}x{h}):**\n\n```\n{ocr_text.strip()}\n```"
                                ocr_results.append(formatted)
                
                pix = None
                
            except Exception as e:
                continue
        
        return ocr_results
        
    except Exception as e:
        return []

def simple_ocr_extraction(img):
    """간단한 OCR 추출"""
    
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # 크기 확대
        h, w = gray.shape
        if w < 600:
            scale = 600 / w
            new_w = int(w * scale)
            new_h = int(h * scale)
            gray = cv2.resize(gray, (new_w, new_h), interpolation=cv2.INTER_CUBIC)
        
        # 기본 OCR
        config = r'--oem 3 --psm 6 -l kor+eng'
        result = pytesseract.image_to_string(gray, config=config)
        
        return result
        
    except Exception as e:
        return ""

def is_fault_ratio_content_text(text):
    """텍스트가 과실비율 관련인지 판단"""
    
    if not text:
        return False
    
    keywords = ['과실', '비율', '보행자', '차량', '가산', '감산']
    keyword_count = sum(1 for keyword in keywords if keyword in text)
    
    return keyword_count >= 2

def save_hybrid_results(all_content, stats):
    """하이브리드 결과 저장"""
    
    output_file = "과실비율_하이브리드추출.md"
    
    content = f"""# 과실비율 PDF 하이브리드 추출 결과

> 추출 일시: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
> 추출 방식: 직접 텍스트 + 이미지 OCR 결합

## 📊 처리 통계

- **전체 페이지**: {stats['total_pages']}
- **직접 텍스트 추출**: {stats['direct_text_pages']} 페이지
- **이미지 OCR 보완**: {stats['image_ocr_pages']} 페이지
- **과실비율 섹션**: {stats['fault_ratio_sections']}개
- **총 추출 글자**: {stats['total_chars']:,}자

---

## 📄 추출된 내용

"""
    
    for page_content in all_content:
        content += page_content
    
    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(content)
    
    print(f"📄 하이브리드 결과 저장: {output_file}")

if __name__ == "__main__":
    hybrid_pdf_extraction()

🔄 하이브리드 PDF 추출 (직접 텍스트 + 이미지 OCR)
📄 전체 페이지: 600
📈 진행률: 1/600 (0.2%)
📈 진행률: 51/600 (8.5%)
📈 진행률: 101/600 (16.8%)
📈 진행률: 151/600 (25.2%)
📈 진행률: 201/600 (33.5%)
📈 진행률: 251/600 (41.8%)
📈 진행률: 301/600 (50.2%)
📈 진행률: 351/600 (58.5%)
📈 진행률: 401/600 (66.8%)
📈 진행률: 451/600 (75.2%)
📈 진행률: 501/600 (83.5%)
📈 진행률: 551/600 (91.8%)
📄 하이브리드 결과 저장: 과실비율_하이브리드추출.md

🎉 하이브리드 추출 완료!
📊 최종 통계:
   - 전체 페이지: 600
   - 직접 텍스트: 593 페이지
   - 이미지 OCR: 0 페이지
   - 과실비율 섹션: 593개
   - 총 글자 수: 684,142
